In [1]:
import pandas as pd
import numpy as np
import time, json, requests 
from datetime import datetime, timedelta, date

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import FinanceDataReader as fdr

import pymysql # python에서 mysql을 사용하는 패키지
import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

# 1. 종목

In [2]:
class STOCK:
    stock_list = None # 종목리스트 저장될 변수
    
    # 종목리스트 초기화
    def __init__(self):
        self.stock_list = pd.DataFrame()
    
    # 종목 수집하기
    def setStocks(self, date):
        # header 선언
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                        AppleWebKit/537.36 (KHTML, like Gecko)\
                        Chrome/102.0.5005.63 Safari/537.36',
            'X-Requested-With': 'XMLHttpRequest'
        }

        # 데이터 url 설정
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

        # payload 데이터 설정 (데이터 속성)
        p_data = {
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501', # 화면번호
            'locale': 'ko_KR',
            'mktId': 'ALL', # 전체 조회
            'trdDd': date, # 조회 날짜
            'share': '1',
            'money': '1', # 금액 단위
            'csvxls_isNo': 'false'
        }

        # HTTP 서버에 데이터 요청
        res = requests.post(url, headers=headers, data=p_data)

        # 요청한 데이터 json 형식으로 html_json에 할당
        html_json = json.loads(res.content)

        # json 형태의 결과값 DataFrame 형식으로 df에 할당
        df = pd.json_normalize(html_json['OutBlock_1'])

        # df 중 필요한 데이터만 클래스의 stock_list에 할당
        self.stock_list[['code', 'name']] = df[['ISU_SRT_CD', 'ISU_ABBRV']] # 종목 코드, 이름
        self.stock_list['date'] = datetime.strptime(date[0:4]+"-"+date[4:6]+"-"+date[6:8], '%Y-%m-%d')
        self.stock_list[['exchange', 'mktcap', 'shares']] = df[['MKT_NM', 'MKTCAP', 'LIST_SHRS']]
        
        # 시가총액, 상장주식수 int 형변환
        for i in range(0, len(self.stock_list)):
            self.stock_list['mktcap'][i] = int(self.stock_list['mktcap'][i].replace(',', ''))
            self.stock_list['shares'][i] = int(self.stock_list['shares'][i].replace(',', ''))
        
        # sqlalchemy의 create_engine을 이용해 DB 연결
        server = '127.0.0.1' # local server
        user = 'root' # user name
        password = '0000' # 개인 password
        db = 'finalterm' # DB 이름
        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
        
        # 종목리스트 stocks DB에 저장
        self.stock_list.to_sql(name='stocks', con=engine, if_exists='append', index=False,
                dtype = {
                    'code' : sqlalchemy.types.VARCHAR(10), # 종목코드
                    'name' : sqlalchemy.types.TEXT(), # 종목명
                    'date' : sqlalchemy.types.DATE(), # 수집날짜
                    'exchange' : sqlalchemy.types.TEXT(), # KOSPI / KOSDAQ / KONEX
                    'mktcap' : sqlalchemy.types.BIGINT(), # 시가총액
                    'shares': sqlalchemy.types.BIGINT() # 상장주식수
                })
        
        # DB 연결 해제
        engine.dispose()
        
    # 모든 종목 불러오기
    def getAllStocks(self):
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        sql = "SELECT * FROM stocks"
        cursor.execute(sql)

        self.stock_list = pd.DataFrame(cursor.fetchall())
        self.stock_list.columns = [col[0] for col in cursor.description]

        # DB 연결 해제
        cursor.close()
        conn.close()
        
        return self.stock_list
        
    # KOSPI / KOSDAQ / KONEX 선택해 불러오기
    def getStocks(self, exchange):
        # DB 연결 & 특정 exchange 종목만 가져오기
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        sql = "SELECT * FROM stocks WHERE exchange='{}'".format(exchange.upper())
        cursor.execute(sql)

        # stock_list에 DB 데이터 할당
        self.stock_list = pd.DataFrame(cursor.fetchall())
        self.stock_list.columns = [col[0] for col in cursor.description] # column명 지정

        # DB 연결 해제
        cursor.close()
        conn.close()
        
        return self.stock_list
    
    # DB에 저장되어 있는 종목 코드 가져오기
    def getStockCode(self):
        # DB 연결 & 특정 exchange 종목만 가져오기
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        sql = "SELECT code FROM stocks";
        cursor.execute(sql)
        
        # stock_list에 DB 데이터 할당
        self.stock_list = pd.DataFrame(cursor.fetchall())
        self.stock_list.columns = [col[0] for col in cursor.description] # column명 지정
        
        # DB 연결 해제
        cursor.close()
        conn.close()
        
        return self.stock_list

### STOCK 클래스 사용 예시

In [158]:
# 주식 종목 클래스 생성 & set
s = STOCK()

# 2022년 1분기 기준 종목 수집 > DB 저장
s.setStocks('20220331')

# 주식 종목 클래스 생성 & get
g = STOCK()

# DB에 저장된 주식 종목 불러오기
stocks = g.getAllStocks() # 전체
stocks = g.getStocks('KOSPI') # KOSPI 종목만
stocks = g.getStocks('KOSDAQ') # KOSDAQ 종목만
stocks = g.getStocks('KONEX') # KONEX 종목만
stocks = g.getStockCode() # 종목 코드 가져오기
stocks

,code
0,060310
1,095570
2,006840
3,054620
4,265520
...,...
2620,000547
2621,000545
2622,003280
2623,037440


# 2. 수정주가

In [3]:
class PRICE:
    price_list = None # 수정주가 리스트 저장될 변수
    
    # 수정주가 리스트 초기화
    def __init__(self):
        self.price_list = pd.DataFrame()
        
    # 수정주가 수집하기
    def setPrices(self):
        # 수정주가 수집 날짜 설정
        start_date = datetime.strptime('2010-01-01', '%Y-%m-%d')
        end_date = datetime.now() # 오늘까지 data 수집
        
        # DB 연결하기
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        # prices 테이블 존재 여부 체크하기
        cursor.execute("SHOW TABLES LIKE 'prices'")
        check = cursor.fetchall()
        
        # prices 테이블 존재하는 경우
        if len(check) != 0:
            sql = "SELECT max(date) from prices" # 가장 최신 날짜 조회하기
            df = pd.read_sql(sql, conn)
            
            # prices 테이블에 데이터 존재하는 경우
            if df.loc[0][0] is not None:
                max_date = df.loc[0][0].strftime('%Y-%m-%d')
                max_date = datetime.strptime(max_date, '%Y-%m-%d')
            
                # max_date까지는 수정주가 데이터 존재
                print(max_date.strftime('%Y-%m-%d'), "exist ...")
                start_date = max_date + timedelta(days=1) # 주가 수집 시작 날짜 변경 (max_date + 1)

        # 이미 오늘 수정주가까지 수집 완료한 경우
        if start_date > end_date:
            print("Already Exist")
            return 

        # 상장 종목 리스트 가져오기
        stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체
        stock_list = fdr.StockListing('KRX').dropna()
        
        # sqlalchemy의 create_engine을 이용해 DB 연결
        server = '127.0.0.1' # local server
        user = 'root' # user name
        password = '0000' # 개인 password
        db = 'finalterm' # DB 이름
        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))
        
        # 수정주가 수집해 저장할 DataFrame 변수
        daily = pd.DataFrame()

        # 모든 종목 코드에 대해 수정주가 수집
        for code in stock_list['Symbol'].values:
            ohlcv = fdr.DataReader(code, start=start_date, end=end_date) # start~end 수정 주가 수집
            ohlcv['code'] = code # 종목코드 추가
            daily = pd.concat([daily, ohlcv]) # daily와 ohlcv 연결
            daily['date'] = daily.index # 날짜 추가
            
            print("("+code+") ... OK") # 해당 종목 수집 완료

        # 수집한 수정주가 DB에 저장
        daily.to_sql(name='prices', con=engine, if_exists='append', index=False,
                    dtype = {
                        'code' : sqlalchemy.types.VARCHAR(10), # 종목코드
                        'date' : sqlalchemy.types.DATE(), # 날짜
                        'Open': sqlalchemy.types.BIGINT(), # 시가
                        'High' : sqlalchemy.types.BIGINT(), # 고가
                        'Low' : sqlalchemy.types.BIGINT(), # 저가
                        'Close' : sqlalchemy.types.BIGINT(), # 종가
                        'Volume' : sqlalchemy.types.BIGINT(), # 거래량
                    })
        
        # DB 연결 해제
        cursor.close()
        conn.close()
        engine.dispose()
        
        print(start_date, " ~ ", end_date, " 수집 완료")
    
    # 수정주가 불러오기 (전체 날짜)
    def getPrices(self):
        # DB 연결 & 전체 수정주가 불러오기
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        sql = "SELECT * FROM prices"
        cursor.execute(sql)

        # DB 불러온 데이터 price_list DataFrame에 할당
        self.price_list = pd.DataFrame(cursor.fetchall())
        self.price_list.columns = [col[0] for col in cursor.description]

        # DB 연결 해제
        cursor.close()
        conn.close()
        
        return self.price_list
    
    # 해당 분기 수정주가 불러오기
    def getTermPrices(self, term):
        # term > 분기별 시작/종료날짜 설정
        start_date, end_date = self.termToDate(term)
        
        # DB 연결 & 해당 분기 수정주가 불러오기
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()
        
        sql = "SELECT * FROM prices WHERE DATE(date) BETWEEN '{}' AND '{}'".format(start_date, end_date)
        cursor.execute(sql)

        # DB 불러온 데이터 price_list DataFrame에 할당
        self.price_list = pd.DataFrame(cursor.fetchall())
        self.price_list.columns = [col[0] for col in cursor.description]

        # DB 연결 해제
        cursor.close()
        conn.close()
        
        return self.price_list
    
    # 분기별 OHLCV 구하기 (분기별 시가, 종가 등)
    def getTermOHLCV(self, term):
        # term으로 입력된 값 period로 변환하기
        period = termToPeriod(term)
        
        # 해당 분기 주가 데이터 불러오기
        df_is = self.getTermPrices(term)
        df_is = df_is.sort_values(by=['Code', 'Date'], axis=0) # code, date 기준 정렬
        
        #결측치 처리
        df_is[['Open', 'High', 'Low', 'Close']] = df_is[['Open', 'High', 'Low', 'Close']].replace(0, np.nan)
        
        df_is['Open'] = np.where(pd.notnull(df_is['Open']) == True, df_is['Open'], df_is['Close'])
        df_is['High'] = np.where(pd.notnull(df_is['High']) == True, df_is['High'], df_is['Close'])
        df_is['Low'] = np.where(pd.notnull(df_is['Low']) == True, df_is['Low'], df_is['Close'])
        df_is['Close'] = np.where(pd.notnull(df_is['Close']) == True, df_is['Close'], df_is['Close'])
        
        # 컬럼명 맞춰주기
        df_is.rename(columns={'Code':'code', 'Date':'date'}, inplace = True)
        
        # stock_code 별로 통계 모으기
        groups = df_is.groupby('code')
        
        # 분기별 OHLCV 구하기
        df_ohlcv = pd.DataFrame()
        
        df_ohlcv['High'] = groups.max()['High'] # 분기별 고가
        df_ohlcv['Low'] = groups.min()['Low'] # 분기별 저가
        df_ohlcv['Period'] = period # 분기 이름 설정
        df_ohlcv['Open'], df_ohlcv['Close'], df_ohlcv['Volume'] = np.nan, np.nan, np.nan # 분기별 시가, 종가, 거래량 (아래에서 구할 것)
        
        df_ohlcv['code'] = df_ohlcv.index # 종목 코드
        df_ohlcv = df_ohlcv.reset_index(drop=True) # index 재설정
        
        for i in range(len(df_ohlcv)):
            df_ohlcv['Open'][i] = float(df_is[df_is['code']==df_ohlcv['code'][i]].head(1)['Open']) # 분기별 시가
            df_ohlcv['Close'][i] = float(df_is[df_is['code']==df_ohlcv['code'][i]].tail(1)['Close']) # 분기별 저가
            df_ohlcv['Volume'][i] = float(df_is[df_is['code']==df_ohlcv['code'][i]].tail(1)['Volume']) # 분기별 거래량
        
        # 분기별 시가총액, 상장주식수 구하기
        df_stock = STOCK().getAllStocks()
        
        # OHLCV와 시가총액, 상장주식수 결합하기
        df_ohlcv = pd.merge(df_ohlcv, df_stock, how='left', on=['code']) # 종목 코드 기준 결합
        
        return df_ohlcv
        
    
    # 분기별 날짜 계산 함수
    def termToDate(self, term):
        if term[5] == '1': # 1분기 (작년 4월~3월)
            start_date = str(int(term[0:4])-1) + '-04-01'
            end_date = term[0:4] + '-03-31'

        elif term[5] == '2': # 2분기 (작년 7월~6월)
            start_date = str(int(term[0:4])-1) + '-07-01'
            end_date = term[0:4] + '-06-30'

        elif term[5] == '3': # 3분기 (작년 10월~9월)
            start_date = str(int(term[0:4])-1) + '-09-01'
            end_date = term[0:4] + '-09-30'

        elif term[5] == '4': # 4분기 (1월~12월)
            start_date = term[0:4] + '-01-01'
            end_date = term[0:4] + '-12-31'
        
        return start_date, end_date

### PRICE 클래스 사용 예시

In [159]:
# 수정주가 클래스 생성 & set
s = PRICE()
s.setPrices()

# 수정주가 클래스 생성 & get
g = PRICE()
list = g.getPrices() # 전체 수정주가 불러오기
list = g.getTermPrices('2022Q1') # 2022 1분기 수정주가만 불러오기
list = g.getTermOHLCV('2022Q1') # 2022 1분기 OHLCV 불러오기 (분기별 시가, 종가, 고가, 저가, 거래량 + 시가총액, 상장주식수)
list

,High,Low,Period,Open,Close,Volume,code,name,date,exchange,mktcap,shares
0,21550.0,11250.0,2022/03,14200.0,13650.0,214548.0,000020,동화약품,2022-03-31,KOSPI,381264565500,27931470
1,1470.0,720.0,2022/03,1340.0,889.0,417614.0,000040,KR모터스,2022-03-31,KOSPI,85466120152,96137368
2,16000.0,12650.0,2022/03,13200.0,15950.0,42169.0,000050,경방,2022-03-31,KOSPI,437273556500,27415270
3,53500.0,17150.0,2022/03,19350.0,47450.0,298202.0,000060,메리츠화재,2022-03-31,KOSPI,5723656250000,120625000
4,147500.0,79900.0,2022/03,93800.0,86100.0,13667.0,000070,삼양홀딩스,2022-03-31,KOSPI,737383733100,8564271
...,...,...,...,...,...,...,...,...,...,...,...,...
2326,6870.0,2700.0,2022/03,5150.0,4585.0,63761.0,950170,JTC,2022-03-31,KOSDAQ,160500295445,35005517
2327,26500.0,16950.0,2022/03,24300.0,20300.0,45327.0,950190,미투젠,2022-03-31,KOSDAQ,275671807600,13579892
2328,17300.0,6520.0,2022/03,12550.0,7290.0,23924.0,950200,소마젠,2022-03-31,KOSDAQ,139483601370,19133553
2329,40850.0,12850.0,2022/03,28100.0,20300.0,511003.0,950210,프레스티지바이오파마,2022-03-31,KOSPI,1219951946500,60096155


# 3. 재무제표

### period 계산 함수

In [4]:
# 분기별 period 설정
def termToPeriod(term):
    if term[5] == '1': # 1분기 (1~3)
        period = term[0:4] + '/03' # 3월
    elif term[5] == '2': # 2분기 (4~6)
        period = term[0:4] + '/06' # 6월
    elif term[5] == '3': # 3분기 (7~9)
        period = term[0:4] + '/09' # 9월
    elif term[5] == '4': # 4분기 (10~12)
        period = term[0:4] + '/12' # 12월
        
    return period

## 3-1. 손익계산서

In [32]:
class INCOMES:
    is_list = None # 손익계산서 리스트 저장될 변수
    
    # 손익게산서 리스트 초기화
    def __init__(self):
        self.is_list = pd.DataFrame()
    
    # 손익계산서 수집하기
    def setIncomes(self, code, rpt_type, freq):
        # sqlalchemy의 create_engine을 이용해 DB 연결
        server = '127.0.0.1' # local server
        user = 'root' # user name
        password = '0000' # 개인 password
        db = 'finalterm' # DB 이름
        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

        items_en = ['rev', 'cgs', 'gross', 'sga', 'sga1', 'sga2', 'sga3', 'sga4', 'sga5', 'sga6', 'sga7', 'sga8', # 12
                    'opr', 'opr_', 'fininc', 'fininc1', 'fininc2', 'fininc3', 'fininc4', 'fininc5', 'fininc6', # 9
                    'fininc7', 'fininc8', 'fininc9', 'fininc10', 'fininc11', 'fincost', 'fincost1', 'fincost2', # 8
                    'fincost3', 'fincost4', 'fincost5', 'fincost6', 'fincost7', 'fincost8', 'fincost9', 'fincost10', # 8
                    'otherrev', 'otherrev1', 'otherrev2', 'otherrev3', 'otherrev4', 'otherrev5', 'otherrev6', # 7
                    'otherrev7', 'otherrev8', 'otherrev9', 'otherrev10', 'otherrev11', 'otherrev12', 'otherrev13', # 7
                    'otherrev14', 'otherrev15', 'otherrev16', 'othercost', 'othercost1', 'othercost2', 'othercost3', # 7
                    'othercost4', 'othercost5', 'othercost6', 'othercost7', 'othercost8', 'othercost9', 'othercost10', # 7
                    'othercost11', 'othercost12', 'otherpl', 'otherpl1', 'otherpl2', 'otherpl3', 'otherpl4', # 7
                    'ebit', 'tax', 'contop', 'discontop', 'netinc'] # 5

        # 손익계산서 종류 (연간/분기)
        if rpt_type.upper() == 'CONSOLIDATED': # 연결 손익계산서 (ReportGB=D)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'.format(code)
            items_en = items_en + ['netinc1', 'netinc2']
        else: # 별도 손익계산서 (ReportGB=B)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701'.format(code)
        
        # Header 설정
        headers = {
            "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
        }
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')

        # 손익계산서 종류 (연간/분기)
        if freq.upper() == 'A': # 연간 손익계산서 영역 추출
            is_a = soup.find(id = 'divSonikY')
            num_col = 3 # 최근 3개년간 데이터
        else: # 분기 손익계산서 영역 추출 (freq.upper() == 'Q')
            is_a = soup.find(id = 'divSonikQ')
            num_col = 4 # 최근 4개 분기 데이터
        
        # 손익계산서 존재하지 않는 종목
        if is_a is None:
            return

        is_a = is_a.find_all(['tr'])

        # 손익계산서 자료 수집
        period = [is_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
        
        # 항목별 값 불러오기
        for item, i in zip(items_en, range(1, len(is_a))):
            temps = []
            for j in range(0, num_col):
                temp = [float(is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                       if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                       else (0 if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                           else 0)]

                temps.append(temp[0])

            globals()[item] = temps # item_en 내 각 항목을 global 변수로 지정하고 값 저장

        # 지배/비지배 항목 처리
        if rpt_type.upper() == 'CONSOLIDATED': # 연결 연간 손익계산서는 아무 것도 하지 않음
            pass
        else: # 별도 연간 손익계산서 해당 항목을 Null값으로 채움
            globals()['netinc1'], globals()['netinc2'] = [np.NaN]*num_col, [np.NaN]*num_col  
        
        # 데이터프레임으로 변환
        is_domestic = pd.DataFrame({
            'code': code,
            'period': period,
            'rev': rev, # 매출액
            'cgs': cgs, # 매출원가
            'gross': gross, # 매출총이익
            'sga': sga, # 판매비와관리비
            'opr': opr, # 영업이익
            'fininc': fininc, # 금융이익
            'fincost': fincost, # 금융원가
            'otherrev': otherrev, # 기타수익
            'othercost': othercost, # 기타비용
            'otherpl': otherpl, # 종속기업, 공동지배기업및관계기업관련손익
            'ebit': ebit, # 세전계속사업이익
            'tax': tax, # 법인세비용
            'contop': contop, # 계속영업이익
            'discontop': discontop, # 중단영업이익
            'netinc': netinc, # 당기순이익
            })

        is_domestic['rpt_type'] = rpt_type+'_'+freq.upper() # 손익계산서 종류

        # DB 중복 데이터 있는지 체크 (종목코드, 기간, 손익계산서 종류)
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # incomes 테이블 존재 여부 체크 (없는 경우 바로 DB에 저장)
        cursor.execute("SHOW TABLES LIKE 'incomes'")
        result = cursor.fetchall()

        # incomes 테이블 존재하는 경우
        if len(result)!=0:
            r = rpt_type+'_'+freq.upper() # 손익계산서 종류
            
            # 존재하는 data인지 체크
            for i in range(0, len(is_domestic)): # 손익계산서 개수만큼
                sql = "SELECT * FROM incomes WHERE code='{}' and period='{}' and rpt_type='{}'".format(code, is_domestic.loc[i]['period'], r)
                df = pd.read_sql(sql, conn)
                
                if not df.empty: # db에 존재하는 경우
                    print(code, is_domestic.loc[i]['period'], r, "exist ... ")
                    is_domestic = is_domestic.drop([i], axis=0) # 삭제

        # DB에 저장
        is_domestic.to_sql(name='incomes', con=engine, if_exists='append', index=False,
                    dtype = {
                        'code' : sqlalchemy.types.VARCHAR(10),
                        'rpt_type': sqlalchemy.types.TEXT(),
                        'period' : sqlalchemy.types.TEXT(),
                        'rev': sqlalchemy.types.FLOAT(),
                        'cgs': sqlalchemy.types.FLOAT(),
                        'gross': sqlalchemy.types.FLOAT(),
                        'sga': sqlalchemy.types.FLOAT(),
                        'opr': sqlalchemy.types.FLOAT(),
                        'fininc': sqlalchemy.types.FLOAT(),
                        'fincost': sqlalchemy.types.FLOAT(),
                        'otherrev': sqlalchemy.types.FLOAT(),
                        'othercost': sqlalchemy.types.FLOAT(),
                        'otherpl': sqlalchemy.types.FLOAT(),
                        'ebit': sqlalchemy.types.FLOAT(),
                        'tax': sqlalchemy.types.FLOAT(),
                        'contop': sqlalchemy.types.FLOAT(),
                        'discontop': sqlalchemy.types.FLOAT(),
                        'netinc': sqlalchemy.types.FLOAT()
                    })

        # DB 연결 해제
        cursor.close()
        conn.close()
        engine.dispose()

        print("(Incomes "+code+") "+rpt_type+"_"+freq.upper()+" ... OK")
    
    # 손익계산서 불러오기
    def getIncomes(self):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 손익계산서 항목 불러와 is_list에 할당
        sql = "SELECT * FROM incomes"
        cursor.execute(sql)
        self.is_list = pd.DataFrame(cursor.fetchall())

        # DB에 손익계산서 존재하지 않는 경우
        if len(self.is_list) == 0:
            print("Empty Incomes")
            return
        
        # 손익계산서 존재하는 경우 column명 수정
        self.is_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.is_list
    
    # 해당 분기의 손익계산서 불러오기
    def getTermIncomes(self, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 분기 동일한 손익계산서 항목 불러와 is_list에 할당
        sql = "SELECT * FROM incomes WHERE period='{}' AND rpt_type='Consolidated_Q'".format(period)
        cursor.execute(sql)
        self.is_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 분기 손익계산서 존재하지 않는 경우
        if len(self.is_list) == 0:
            print(period, ": Empty Incomes")
            return
        
        # 손익계산서 존재하는 경우 column명 수정
        self.is_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.is_list
        
    # 해당 종목, 해당 분기의 손익계산서 불러오기
    def getStockIncomes(self, code, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 종목, 분기 동일한 손익계산서 항목 불러와 is_list에 할당
        sql = "SELECT * FROM incomes WHERE code='{}' AND period='{}' AND rpt_type='Consolidated_Q'".format(code, period)
        cursor.execute(sql)
        self.is_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 종목 손익계산서 존재하지 않는 경우
        if len(self.is_list) == 0:
            print(code, ": Empty Incomes")
            return
        
        # 손익계산서 존재하는 경우 column명 수정
        self.is_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.is_list
    
    # 트레일링 데이터 만들기
    def getTrailingIncomes(self, code, period):
        df_quat = {}
        period_quat = period # 기간 보정 변수
        
        # 4분기 데이터 불러오기
        for i in range(4):
            df_is = self.getStockIncomes(code, period_quat) # 포괄손익계산서
            
            # 분기 데이터 존재하지 않는 경우 트레일링 데이터 만들지 않음
            if df_is is None:
                return
            
            df_quat[i] = df_is

            if int(period_quat[5:7])-3 > 0: # period 마지막 2개 : 06, 09, 12 (2~4분기)
                period_quat = period_quat[0:4] + '/' + str(int(period_quat[5:7])-3).zfill(2) # 이전 분기로 만들기
            else: # 03 (1분기)
                period_quat = str(int(period_quat[0:4])-1) + '/12' # 이전 분기로 만들기 (작년 12)
        
        df_trailing = pd.DataFrame(df_quat[0], columns=['code', 'period', 'rpt_type'])
        df_trailing[df_quat[0].columns[2:-1]] = 0

        # 모든 데이터 더해 트레일링 데이터로 만들기
        for i in range(len(df_quat)):
            df_trailing[df_quat[0].columns[2:-1]] += df_quat[i][df_quat[i].columns[2:-1]]

        return df_trailing

### INCOMES 클래스 사용 예시

In [171]:
# 손익계산서 클래스 생성 & set
s = INCOMES()

# 수집된 모든 주식 종목 코드에 대해 손익계산서 수집
stock_code = STOCK().getStockCode()
for code in stock_code['code']:
    s.setIncomes(code, 'Consolidated', 'Q') # 연결/분기
    s.setIncomes(code, 'Consolidated', 'A') # 연결/연간
    s.setIncomes(code, 'Unconsolidated', 'Q') # 별도/분기
    s.setIncomes(code, 'Unconsolidated', 'A') # 별도/연간

# 손익계산서 클래스 생성 & get
g = INCOMES()

list = g.getIncomes() # 전체 손익계산서 가져오기
list = g.getTermIncomes('2022/03') # 해당 분기의 모든 종목 손익계산서 가져오기
list = g.getStockIncomes('005930', '2022/03') # 해당 종목코드, 분기의 손익계산서 가져오기
list = g.getTrailingIncomes('005930', '2022/03') # 해당 종목코드, 분기의 트레일링 데이터 가져오기
list

,code,period,rpt_type,rev,cgs,gross,sga,opr,fininc,fincost,otherrev,othercost,otherpl,ebit,tax,contop,discontop,netinc
0,005930,2022/03,Consolidated_Q,2919978.0,1719835.0,1200144.0,636420.0,563723.0,98067.4,87440.0,26061.46,21848.41,8145.72,586710.0,145806.3,440902.9,0.0,440902.9


## 3-2. 재무상태표

In [6]:
class BALANCES:
    bs_list = None # 재무상태표 리스트 저장될 변수
    
    # 재무상태표 리스트 초기화
    def __init__(self):
        self.bs_list = pd.DataFrame()
    
    # 재무상태표 수집하기
    def setBalances(self, code, rpt_type, freq):
        # sqlalchemy의 create_engine을 이용해 DB 연결
        server = '127.0.0.1' # local server
        user = 'root' # user name
        password = '0000' # 개인 password
        db = 'finalterm' # DB 이름

        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

        items_en = ['asset', 'c_asset', 'inv', 'curassets2', 'curassets3', 'rec', 'curassets5', # 7
               'curassets6', 'curassets7', 'curassets8', 'curassets9', 'cash', 'curassets11', # 6
               'l_asset', 'ltassets1', 'ltassets2', 'ltassets3', 'ltassets4', 'ltassets5', 'ltassets6', 'ltassets7', # 8
                'ltassets8', 'ltassets9', 'ltassets10', 'ltassets11', 'ltassets12', 'ltassets13','finassets', # 7
                'liab', 'c_liab', 'curliab1', 'c_debt', 'curliab3', 'curliab4', 'curliab5', # 7
                'curliab6', 'curliab7', 'curliab8', 'curliab9', 'curliab10', 'curliab11', 'curliab12', 'curliab13', # 8
                'l_liab', 'ltliab1', 'l_debt', 'ltliab3', 'ltliab4', 'ltliab5', 'ltliab6', # 7
                'ltliab7', 'ltliab8', 'ltliab9', 'ltliab10', 'ltliab11', 'ltliab12', 'finliab', # 7
                'equity', 'equity1', 'equity2', 'equity3', 'equity4', 'equity5', 'equity6', 'equity7', 'equity8'] # 9

        # 재무상태표 종류 (연결/별도)
        if rpt_type.upper() == 'CONSOLIDATED': # 연결 재무상태표 (ReportGB=D)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'.format(code)
        else: # 별도 재무상태표 (ReportGB=B)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701'.format(code)
            items_en = [item for item in items_en if item not in ['equity1', 'equity8']]

        # Header 설정
        headers = {
            "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
        }
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')

        # 재무상태표 종류 (연간/분기)
        if freq.upper() == 'A': # 연간
            bs_a = soup.find(id = 'divDaechaY')
            num_col = 3 # 최근 3개 연간 데이터
        else: # 분기 (freq.upper() == 'Q')
            bs_a = soup.find(id = 'divDaechaQ')
            num_col = 4 # 최근 4개 분기 데이터

        # 재무상태표 존재하지 않는 종목
        if bs_a is None:
            return
            
        bs_a = bs_a.find_all(['tr'])

        # 최근 재무상태표 자료 수집
        period = [bs_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]

        # 항목별 값 불러오기
        for item, i in zip(items_en, range(1, len(bs_a))):
            temps = []
            for j in range(0, num_col):
                temp = [float(bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                       if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                       else (0 if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                           else 0)]
                temps.append(temp[0])

            globals()[item] = temps # item_en 내 각 항목을 global 변수로 지정하고 값 저장


        # 지배/비지배 항목 처리
        if rpt_type.upper() == 'CONSOLIDATED': # 연결 연간 재무상태표는 아무 것도 하지 않음
            pass
        else: # 별도 연간 재무상태표는 연간에만 존재하는 항목을 Null값으로 채움
            globals()['equity1'], globals()['equity8'] = [np.NaN]*num_col, [np.NaN]*num_col

        bs_domestic = pd.DataFrame({
            'code':code,
            'period':period,
            'asset':asset, # 자산
            'curasset':c_asset, # 유동자산
            'inv':inv, # 재고자산
            'rec':rec, # 매출채권
            'cash':cash, #현금 및 현금성자산
            'longasset':l_asset, # 비유동자산
            'otherasset': finassets, # 기타금융업자산
            'liab':liab, # 부채
            'curliab':c_liab, # 유동부채
            'curdebt':c_debt, # 단기차입금
            'longliab':l_liab, # 비유동부채
            'longdebt':l_debt, # 장기차입금
            'otherliab': ltliab6, # 기타금융업부채
            'equity': equity, # 자본
            'equity2': equity2, # 자본금
            'equity3': equity3, # 신종자본증권
            'equity4': equity4, # 자본잉여금
            'equity5': equity5, # 기타자본
            'equity6': equity6, # 기타포괄손익누계액
            'equity7': equity7 # 이익잉여금(결손금)
        })

        # 재무제표 종료 컬럼 추가
        bs_domestic['rpt_type'] = rpt_type+'_'+freq.upper()

        # DB 중복 데이터 있는지 체크 (종목코드, 기간, 재무상태표 종류)
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # balances 테이블 존재 여부 체크 (없는 경우 바로 DB에 저장)
        cursor.execute("SHOW TABLES LIKE 'balances'")
        check = cursor.fetchall()

        if len(check)!=0: # balances 테이블이 존재하는 경우 : 존재하는 데이터인지 체크
            c = code # 종목코드
            r = rpt_type+'_'+freq.upper() # 재무상태표 종류

            for i in range(0, len(bs_domestic)): # 재무상태표 기간
                sql = "SELECT * from balances where code='{}' and period='{}' and rpt_type='{}'".format(c, bs_domestic.loc[i]['period'], r)
                df = pd.read_sql(sql, conn)
                if not df.empty: # db에 존재하는 경우
                    print(code, bs_domestic.loc[i]['period'], r, "exist ... ")
                    bs_domestic = bs_domestic.drop([i], axis=0) # 삭제

        # DB에 저장
        bs_domestic.to_sql(name='balances', con=engine, if_exists='append', index=False,
                    dtype = {
                        'code' : sqlalchemy.types.VARCHAR(10),
                        'name' : sqlalchemy.types.TEXT(),
                        'rpt_type': sqlalchemy.types.TEXT(),
                        'period' : sqlalchemy.types.TEXT(),
                        'asset': sqlalchemy.types.FLOAT(),
                        'curasset': sqlalchemy.types.FLOAT(),
                        'inv':sqlalchemy.types.FLOAT(),
                        'rec':sqlalchemy.types.FLOAT(),
                        'cash':sqlalchemy.types.FLOAT(),
                        'longasset':sqlalchemy.types.FLOAT(),
                        'otherasset': sqlalchemy.types.FLOAT(),
                        'liab': sqlalchemy.types.FLOAT(),
                        'curliab': sqlalchemy.types.FLOAT(),
                        'curdebt': sqlalchemy.types.FLOAT(),
                        'longliab': sqlalchemy.types.FLOAT(),
                        'longdebt': sqlalchemy.types.FLOAT(),
                        'otherliab': sqlalchemy.types.FLOAT(),
                        'equity': sqlalchemy.types.FLOAT(),
                        'equity2': sqlalchemy.types.FLOAT(),
                        'equity3': sqlalchemy.types.FLOAT(),
                        'equity4': sqlalchemy.types.FLOAT(),
                        'equity5': sqlalchemy.types.FLOAT(),
                        'equity6': sqlalchemy.types.FLOAT(),
                        'equity7': sqlalchemy.types.FLOAT(),
                    })

        # DB 접속 해제
        cursor.close()
        conn.close()
        engine.dispose()

        print("(Balances "+code+") "+rpt_type+"_"+freq.upper()+" ... OK")
    
    # 재무상태표 불러오기
    def getBalances(self):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 전체 재무상태표 항목 불러와 bs_list에 할당
        sql = "SELECT * FROM balances"
        cursor.execute(sql)
        self.bs_list = pd.DataFrame(cursor.fetchall())

        # DB에 재무상태표 존재하지 않는 경우
        if len(self.bs_list) == 0:
            print("Empty Balances")
            return
        
        # 재무상태표 존재하는 경우 column명 수정
        self.bs_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.bs_list
    
    # 해당 분기의 재무상태표 불러오기
    def getTermBalances(self, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 분기 동일한 재무상태표 항목 불러와 bs_list에 할당
        sql = "SELECT * FROM balances WHERE period='{}' AND rpt_type='Consolidated_Q'".format(period)
        cursor.execute(sql)
        self.bs_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 분기 재무상태표 존재하지 않는 경우
        if len(self.bs_list) == 0:
            print(period, ": Empty Balances")
            return
        
        # 재무상태표 존재하는 경우 column명 수정
        self.bs_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.bs_list
        
    # 해당 종목, 해당 분기의 재무상태표 불러오기
    def getStockBalances(self, code, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 종목, 분기 동일한 재무상태표 항목 불러와 bs_list에 할당
        sql = "SELECT * FROM balances WHERE code='{}' AND period='{}' AND rpt_type='Consolidated_Q'".format(code, period)
        cursor.execute(sql)
        self.bs_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 종목 재무상태표 존재하지 않는 경우
        if len(self.bs_list) == 0:
            print(code, ": Empty Balances")
            return
        
        # 재무상태표 존재하는 경우 column명 수정
        self.bs_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.bs_list
    
    # 트레일링 데이터 만들기
    def getTrailingBalances(self, code, period):
        df_quat = {}
        period_quat = period # 기간 보정 변수

        # 4분기 데이터 불러오기
        for i in range(4):
            df_bs = self.getStockBalances(code, period_quat) # 재무상태표
            
            # 분기 데이터 존재하지 않는 경우 트레일링 데이터 만들지 않음
            if df_bs is None:
                return
            
            df_quat[i] = df_bs

            if int(period_quat[5:7])-3 > 0: # period 마지막 2개 : 06, 09, 12 (2~4분기)
                period_quat = period_quat[0:4] + '/' + str(int(period_quat[5:7])-3).zfill(2) # 이전 분기로 만들기
            else: # 03 (1분기)
                period_quat = str(int(period_quat[0:4])-1) + '/12' # 이전 분기로 만들기 (작년 12)
        
        df_trailing = pd.DataFrame(df_quat[0], columns=['code', 'period', 'rpt_type'])
        df_trailing[df_quat[0].columns[2:-1]] = 0

        # 모든 데이터 더해 트레일링 데이터로 만들기
        for i in range(len(df_quat)):
            df_trailing[df_quat[0].columns[2:-1]] += df_quat[i][df_quat[i].columns[2:-1]]

        return df_trailing

### BALANCES 클래스 사용 예시

In [207]:
# 재무상태표 클래스 생성 & set
s = BALANCES()

# 수집된 모든 주식 종목 코드에 대해 재무상태표 수집
stock_code = STOCK().getStockCode()
for code in stock_code['code']:
    s.setBalances(code, 'Consolidated', 'Q') # 연결/분기
    s.setBalances(code, 'Consolidated', 'A') # 연결/연간
    s.setBalances(code, 'Unconsolidated', 'Q') # 별도/분기
    s.setBalances(code, 'Unconsolidated', 'A') # 별도/연간

# 재무상태표 클래스 생성 & get
g = BALANCES()

list = g.getBalances() # 전체 재무상태표 가져오기
list = g.getTermBalances('2022/03') # 해당 분기의 모든 종목 재무상태표 가져오기
list = g.getStockBalances('352820', '2022/03') # 해당 종목코드, 분기의 재무상태표 가져오기
list = g.getTrailingBalances('352820', '2022/03') # 해당 종목코드, 분기의 트레일링 데이터 가져오기
list

,code,period,rpt_type,asset,curasset,inv,rec,cash,longasset,otherasset,...,longliab,longdebt,otherliab,equity,equity2,equity3,equity4,equity5,equity6,equity7
0,352820,2022/03,Consolidated_Q,166596.5,68139.9,3421.1,7088.64,25019.82,98456.5,0.0,...,42184.01,23851.91,636.4,100722.5,803.8,0.0,87563.5,193.92,2076.78,6050.28


## 3-3. 현금흐름표

In [7]:
class CASHFLOW:
    cf_list = None # 현금흐름표 리스트 저장될 변수
    
    # 현금흐름표 리스트 초기화
    def __init__(self):
        self.cf_list = pd.DataFrame()
    
    # 현금흐름표 수집하기
    def setCashflow(self, code, rpt_type, freq):
        # sqlalchemy의 create_engine을 이용해 DB 연결
        server = '127.0.0.1' # local server
        user = 'root' # user name
        password = '0000' # 개인 password
        db = 'finalterm' # DB 이름

        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

        items_en = ['cfo', 'cfo1', 'cfo2', 'cfo3', 'cfo4', 'cfo5', 'cfo6', 'cfo7', 'cfo8', #9
                    'cfi', 'cfi1', 'cfi2', 'cfi3', 'cff', 'cff1', 'cff2', 'cff3', # 8
                    'cff4', 'cff5', 'cff6', 'cff7', 'cff8', 'cff9'] #6

        # 현금흐름표 종류 (연결/별도)
        if rpt_type.upper() == 'CONSOLIDATED': # 연결 현금흐름표 (ReportGB=D)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'.format(code)
            items_en = items_en + ['netinc1', 'netinc2']
        else: # 별도 현금흐름표 (ReportGB=B)
            url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701'.format(code)

        # Header 설정
        headers = {
            "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
        }
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')

        # 현금흐름표 종류 (연간/분기)
        if freq.upper() == 'A': # 연간 현금흐름표 영역 추출
            cf_a = soup.find(id = 'divCashY')
            num_col = 3 # 최근 3개년간 데이터
        else: # 분기 현금흐름표 영역 추출 (freq.upper() == 'Q')
            cf_a = soup.find(id = 'divCashQ')
            num_col = 4 # 최근 4개 분기 데이터
        
        # 현금흐름표 존재하지 않는 종목코드 체크
        if cf_a is None:
            return

        cf_a = cf_a.find_all(['tr'])
        
        # 현금흐름표 항목 개수 맞지 않는 종목코드 체크
        if len(cf_a) < 158:
            return

        # 현금흐름표 자료 수집
        period = [cf_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]

        # 항목별 값 불러오기 (수집할 인덱스 값 미리 설정)
        idx = [1,2,3,4,9,39,70,75,76,84,85,99,113,121,122,134,145,153,154,155,156,157,158]
        for item, i in zip(items_en, idx):
            temps = []
            for j in range(0, num_col):
                temp = [float(cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                       if cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                       else (0 if cf_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                           else 0)]

                temps.append(temp[0])

            globals()[item] = temps # item_en 내 각 항목을 global 변수로 지정하고 값 저장

            items_en = ['cfo', 'cfo1', 'cfo2', 'cfo3', 'cfo4', 'cfo5', 'cfo6', 'cfo7', 'cfo8', #9
                    'cfi', 'cfi1', 'cfi2', 'cfi3', 'cff', 'cff1', 'cff2', 'cff3', # 8
                    'cff4', 'cff5', 'cff6', 'cff7', 'cff8', 'cff9'] #6
            
            
        # 현금흐름표 주요컬럼값만 추출하여 데이터프레임으로 변환
        cf_domestic = pd.DataFrame({
            'code':code,
            'period':period,
            'cfo':cfo, # 영업현금흐름
            'cfo1': cfo1, # 당기순손익
            'cfo2': cfo2, # 법인세비용차감전계속사업이익
            'cfo3': cfo3, # 현금유출이없는비용등가산
            'cfo4':cfo4, # 감가상각비
            'cfo5': cfo5, # 현금유입이없는수익등차감
            'cfo6': cfo6, # 영업활동으로인한자산부채변동(운전자본변동)
            'cfo7': cfo7, # 영업에서창출된현금흐름
            'cfo8': cfo8, # 기타영업활동으로인한현금흐름
            'cfi': cfi, # 투자활동으로인한현금흐름
            'cfi1': cfi1, # 투자활동으로인한현금유입액
            'cfi2': cfi2, # 투자활동으로인한현금유출액
            'cfi3': cfi3, # 기타투자활동으로인한현금흐름
            'cff': cff, # 재무활동으로인한현금흐름
            'cff1': cff1, # 재무활동으로인한현금유입액
            'cff2': cff2, # 재무활동으로인한현금유출액
            'cff3': cff3, # 기타재무활동으로인한현금흐름
            'cff4': cff4, # 영업투자재무활동기타현금흐름
            'cff5': cff5, # 연결범위변동으로인한현금의증가
            'cff6': cff6, # 환율변동효과
            'cff7': cff7, # 현금및현금성자산의증가
            'cff8': cff8, # 기초현금및현금성자산
            'cff9': cff9 # 기말현금및현금성자산
            })

        cf_domestic['rpt_type'] = rpt_type+'_'+freq.upper() # 손익계산서 종류

        # DB 중복 데이터 있는지 체크 (종목코드, 기간, 현금흐름표 종류)
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # cashflow 테이블 존재 여부 체크 (없는 경우 바로 DB에 저장)
        cursor.execute("SHOW TABLES LIKE 'cashflow'")
        result = cursor.fetchall()

        if len(result)!=0: # cashflow 테이블이 존재하는 경우 : 존재하는 데이터인지 체크
            r = rpt_type+'_'+freq.upper() # 현금흐름표 종류

            for i in range(0, len(cf_domestic)): # 현금흐름표 기간
                sql = "SELECT * from cashflow where code='{}' and period='{}' and rpt_type='{}'".format(code, cf_domestic.loc[i]['period'], r)
                df = pd.read_sql(sql, conn)
                if not df.empty: # db에 존재하는 경우
                    print(code, cf_domestic.loc[i]['period'], r, "exist ... ")
                    cf_domestic = cf_domestic.drop([i], axis=0) # 삭제

        # DB에 저장
        cf_domestic.to_sql(name='cashflow', con=engine, if_exists='append', index=False,
                    dtype = {
                        'code' : sqlalchemy.types.VARCHAR(10),
                        'name' : sqlalchemy.types.TEXT(),
                        'rpt_type': sqlalchemy.types.TEXT(),
                        'period' : sqlalchemy.types.TEXT(),
                        'cfo': sqlalchemy.types.FLOAT(),
                        'cfo1': sqlalchemy.types.FLOAT(),
                        'cfo2': sqlalchemy.types.FLOAT(),
                        'cfo3': sqlalchemy.types.FLOAT(),
                        'cfo4': sqlalchemy.types.FLOAT(),
                        'cfo5': sqlalchemy.types.FLOAT(),
                        'cfo6': sqlalchemy.types.FLOAT(),
                        'cfo7': sqlalchemy.types.FLOAT(),
                        'cfo8': sqlalchemy.types.FLOAT(),
                        'cfi': sqlalchemy.types.FLOAT(),
                        'cfi1': sqlalchemy.types.FLOAT(),
                        'cfi2': sqlalchemy.types.FLOAT(),
                        'cfi3': sqlalchemy.types.FLOAT(),
                        'cff': sqlalchemy.types.FLOAT(),
                        'cff1': sqlalchemy.types.FLOAT(),
                        'cff2': sqlalchemy.types.FLOAT(),
                        'cff3': sqlalchemy.types.FLOAT(),
                        'cff4': sqlalchemy.types.FLOAT(),
                        'cff5': sqlalchemy.types.FLOAT(),
                        'cff6': sqlalchemy.types.FLOAT(),
                        'cff7': sqlalchemy.types.FLOAT(),
                        'cff8': sqlalchemy.types.FLOAT(),
                        'cff9': sqlalchemy.types.FLOAT()
                    })

        # DB 접속 해제
        cursor.close()
        conn.close()
        engine.dispose()

        print("(Cashflow "+code+") "+rpt_type+"_"+freq.upper()+" ... OK")
    
    # 현금흐름표 불러오기
    def getCashflow(self):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 모든 현금흐름표 항목 불러와 cf_list에 할당
        sql = "SELECT * FROM cashflow"
        cursor.execute(sql)
        self.cf_list = pd.DataFrame(cursor.fetchall())

        # DB에 현금흐름표 존재하지 않는 경우
        if len(self.cf_list) == 0:
            print("Empty Cashflow")
            return
        
        # 현금흐름표 존재하는 경우 column명 수정
        self.cf_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.cf_list
    
    # 해당 분기의 현금흐름표 불러오기
    def getTermCashflow(self, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 분기 동일한 현금흐름표 항목 불러와 cf_list에 할당
        sql = "SELECT * FROM cashflow WHERE period='{}' AND rpt_type='Consolidated_Q'".format(period)
        cursor.execute(sql)
        self.cf_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 분기 현금흐름표 존재하지 않는 경우
        if len(self.cf_list) == 0:
            print(period, ": Empty Cashflow")
            return
        
        # 현금흐름표 존재하는 경우 column명 수정
        self.cf_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.cf_list
        
    # 해당 종목, 해당 분기의 현금흐름표 불러오기
    def getStockCashflow(self, code, period):
        # db에 연결
        conn = pymysql.connect(host='localhost', user='root', password='0000', db='finalterm', charset='utf8')
        cursor = conn.cursor()

        # 종목, 분기 동일한 현금흐름표 항목 불러와 cf_list에 할당
        sql = "SELECT * FROM cashflow WHERE code='{}' AND period='{}' AND rpt_type='Consolidated_Q'".format(code, period)
        cursor.execute(sql)
        self.cf_list = pd.DataFrame(cursor.fetchall())

        # DB에 해당 종목 현금흐름표 존재하지 않는 경우
        if len(self.cf_list) == 0:
            print(code, ": Empty Cashflow")
            return
        
        # 현금흐름표 존재하는 경우 column명 수정
        self.cf_list.columns = [col[0] for col in cursor.description]

        # db 연결 해제
        cursor.close()
        conn.close()

        return self.cf_list
    
    # 트레일링 데이터 만들기
    def getTrailingCashflow(self, code, period):
        df_quat = {}
        period_quat = period # 기간 보정 변수

        # 4분기 데이터 불러오기
        for i in range(4):
            df_cf = self.getStockCashflow(code, period_quat) # 현금흐름표
            
            # 분기 데이터 존재하지 않는 경우 트레일링 데이터 만들지 않음
            if df_cf is None:
                return
            
            df_quat[i] = df_cf

            if int(period_quat[5:7])-3 > 0: # period 마지막 2개 : 06, 09, 12 (2~4분기)
                period_quat = period_quat[0:4] + '/' + str(int(period_quat[5:7])-3).zfill(2) # 이전 분기로 만들기
            else: # 03 (1분기)
                period_quat = str(int(period_quat[0:4])-1) + '/12' # 이전 분기로 만들기 (작년 12)
        
        df_trailing = pd.DataFrame(df_quat[0], columns=['code', 'period', 'rpt_type'])
        df_trailing[df_quat[0].columns[2:-1]] = 0

        # 모든 데이터 더해 트레일링 데이터로 만들기
        for i in range(len(df_quat)):
            df_trailing[df_quat[0].columns[2:-1]] += df_quat[i][df_quat[i].columns[2:-1]]

        return df_trailing

### CASHFLOW 클래스 사용 예시

In [213]:
# 현금흐름표 클래스 생성 & set
s = CASHFLOW()

# 수집된 모든 주식 종목 코드에 대해 현금흐름표 수집
stock_code = STOCK().getStockCode()
for code in stock_code['code']:
    s.setCashflow(code, 'Consolidated', 'Q') # 연결/분기
    s.setCashflow(code, 'Consolidated', 'A') # 연결/연간
    s.setCashflow(code, 'Unconsolidated', 'Q') # 별도/분기
    s.setCashflow(code, 'Unconsolidated', 'A') # 별도/연간

# 현금흐름표 클래스 생성 & get
g = CASHFLOW()

list = g.getCashflow() # 전체 현금흐름표 가져오기
list = g.getTermCashflow('2022/03') # 해당 분기의 모든 종목 현금흐름표 가져오기
list = g.getStockCashflow('035720', '2022/03') # 해당 종목코드, 분기의 현금흐름표 가져오기
list = g.getTrailingCashflow('035720', '2022/03') # 해당 종목코드, 분기의 트레일링 데이터 가져오기
list

,code,period,rpt_type,cfo,cfo1,cfo2,cfo3,cfo4,cfo5,cfo6,...,cff,cff1,cff2,cff3,cff4,cff5,cff6,cff7,cff8,cff9
0,035720,2022/03,Consolidated_Q,11157.88,0.0,37345.85,19569.88,2695.24,38765.02,-5238.37,...,38788.26,46874.44,7398.61,-687.43,0.0,0.0,-82.46,19551.72,146980.5,166532.2


# 4. 가치주 종목 추출

## 가치주 지표 계산 클래스

In [123]:
class VALUATION:
    period = None # 분기 할당 (str)
    stocks = None # 수집된 종목 할당 (DataFrame)
    ohlcv = None # 분기별 수정주가 할당 (DataFrame)
    fin_unit = 0 # 재무제표 단위 설정
    
    def __init__(self, term):
        print("init 실행 중", datetime.now())
        
        self.stocks = STOCK().getAllStocks() # 수집된 종목 가져오기
        self.period = termToPeriod(term) # 분기 설정
        self.ohlcv = PRICE().getTermOHLCV(term) # 해당 분기 수정주가 가져오기
        self.ohlcv = self.ohlcv.set_index('code') # 종목 코드를 인덱스로 사용
        self.fin_unit = 100000000 # 재무제표 단위 : 억원
        
        print("init 실행 완료", datetime.now())
        
    # 이익 대비 저평가 종목 (주가 수익 비율)
    def getPER(self):
        # PER 계산 후 저장할 DataFrame 생성
        df_PER = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)

            # 손익계산서 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass

            # 2. EPS = 당기순이익/상장주식수 
            # 당기순이익: incomes > netinc
            EPS = float((is_trailing['netinc']*self.fin_unit) / self.ohlcv.loc[code]['shares']) # ohlcv : 해당 종목 코드의 상장주식수
            
            # 3. PER = 주가/EPS 계산
            PER = float(self.ohlcv.loc[code]['Close'] / EPS)

            # 4. 결측치 처리
            if float(is_trailing['netinc']) == 0 or self.ohlcv.loc[code]['shares'] == 0 or EPS == 0: # 0으로 나누는 경우
                PER = np.NaN # np.nan 처리
            if PER < 0: # PER 값이 음수인 경우
                PER = 0 # 0으로 처리

            # 5. 계산한 결과 DataFrame에 추가
            df_PER = df_PER.append({
                'code': code,
                'name': name,
                'exchange': exc,
                'period': self.period, # 기간
                'mktcap': self.ohlcv.loc[code]['mktcap'], # 시가총액
                'PER':PER }, ignore_index=True)
            
            print(code, ": Calculate PER ... OK")
            
        return df_PER
        
    # 장부가치 대비 저평가 종목 (주가 순자산 비율)
    def getPBR(self):
        # PBR 계산 후 저장할 DataFrame 생성
        df_PBR = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 재무상태표 가져오기
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무상태표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
                
            # 2. BPS = 자본/상장주식수 계산
            # 자본: balances > equity
            BPS = float((bs_trailing['equity']*self.fin_unit) / self.ohlcv.loc[code]['shares']) # ohlcv : 해당 종목 코드의 상장주식수
            
            # 3. PBR = 주가/BPS 계산
            PBR = float(self.ohlcv.loc[code]['Close'] / BPS)
            
            # 4. 결측치 처리
            if float(bs_trailing['equity']) == 0 or self.ohlcv.loc[code]['shares'] == 0 or BPS == 0: # 0으로 나누는 경우
                PBR = np.NaN # np.nan 처리
            if PBR < 0: # PBR 값이 음수인 경우
                PBR = 0 # 0으로 처리
                
            # 5. 계산한 결과 DataFrame에 추가
            df_PBR = df_PBR.append({
                'code': code,
                'name': name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'PBR':PBR }, ignore_index=True)
            
            print(code, ": Calculate PBR ... OK")
        
        return df_PBR
    
    # 매출 대비 저평가 종목 (주가 매출 비율)
    def getPSR(self):
        # PSR 계산 후 저장할 DataFrame 생성
        df_PSR = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            
            # 손익계산서 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
                
            # 2. SPS = 매출액/상장주식수 계산
            # 매출액: incomes > rev
            SPS = float((is_trailing['rev']*self.fin_unit) / self.ohlcv.loc[code]['shares']) # ohlcv : 해당 종목 코드의 상장주식수
            
            # 3. PSR = 주가/SPS 계산
            PSR = float(self.ohlcv.loc[code]['Close'] / SPS)
            
            # 4. 결측치 처리
            if float(is_trailing['rev']) == 0 or self.ohlcv.loc[code]['shares'] == 0 or SPS == 0: # 0으로 나누는 경우
                PSR = np.NaN # np.nan 처리
            if PSR < 0: # PSR 값이 음수인 경우
                PSR = 0 # 0으로 처리
                
            # 5. 계산한 결과 DataFrame에 추가
            df_PSR = df_PSR.append({
                'code': code,
                'name': name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'PSR':PSR }, ignore_index=True)
            
            print(code, ": Calculate PSR ... OK")
        
        return df_PSR
        
    # 현금흐름 대비 저평가 종목 (주가 현금흐름 비율)
    def getPCR(self):
        # PCR 계산 후 저장할 DataFrame 생성
        df_PCR = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 현금흐름표 가져오기
            cf_trailing = CASHFLOW().getTrailingCashflow(code, self.period)
            
            # 현금흐름표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if cf_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
                
            # 2. CFPS = 영업현금흐름/상장주식수 계산
            # 영업현금흐름: cashflow > cfo
            CFPS = float((cf_trailing['cfo']*self.fin_unit) / self.ohlcv.loc[code]['shares']) # ohlcv : 해당 종목 코드의 상장주식수
            
            # 3. PCR = 주가/CFPS 계산
            PCR = float(self.ohlcv.loc[code]['Close'] / CFPS)
            
            # 4. 결측치 처리
            if float(cf_trailing['cfo']) == 0 or self.ohlcv.loc[code]['shares'] == 0 or CFPS == 0: # 0으로 나누는 경우
                PCR = np.NaN # np.nan 처리
            if PCR < 0: # PCR 값이 음수인 경우
                PCR = 0 # 0으로 처리
                
            # 5. 계산한 결과 DataFrame에 추가
            df_PCR = df_PCR.append({
                'code': code,
                'name': name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'PCR':PCR }, ignore_index=True)
            
            print(code, ": Calculate PCR ... OK")
        
        return df_PCR
    
    # 실적 대비 기업가치 (EV/EBITDA)
    def getEVEBITDA(self):
        # EV/EBITDA 계산 후 저장할 DataFrame 생성
        df_EE = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서 & 재무상태표 & 현금흐름표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            cf_trailing = CASHFLOW().getTrailingCashflow(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or cf_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
        
            # 2. EV=(시가총액 + 차입금 - 현금성자산) 계산
            # 시가총액: ohlcv > mktcap
            # 차입금: balances > longdebt + curdebt (장기차입금 + 단기차입금)
            # 현금성자산: balances > cash
            EV = float(self.ohlcv.loc[code]['mktcap'] + (bs_trailing['longdebt'] + bs_trailing['curdebt'] - bs_trailing['cash'])*self.fin_unit)

            # 3. EBITDA=(영업이익 + 감가상각비) 계산
            # 영업이익: incomes > opr
            # 감가상각비: cashflow > cfo4
            EBITDA = float(is_trailing['opr'] + cf_trailing['cfo4']*self.fin_unit)

            # 4. EV/EBITDA 계산
            if EV <= 0 or EBITDA <= 0: # EV 음수 or EBITDA 음수인 경우
                EE = np.NaN
            else: # 계산 가능한 경우
                EE = float(EV / EBITDA)
                
            # 5. 계산한 결과 DataFrame에 추가
            df_EE = df_EE.append({
                'code': code,
                'name': name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'EV/EBITDA': EE }, ignore_index=True)
            
            print(code, ": Calculate EV/EBITDA ... OK")
        
        return df_EE
    
    # 실적 대비 기업가치 (EV/Sales)
    def getEVSales(self):
        # EV/Sales 계산 후 저장할 DataFrame 생성
        df_ES = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서 & 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
        
            # 2. EV=(시가총액 + 차입금 - 현금성자산) 계산
            # 시가총액: stocks > mktcap
            # 차입금: balances > longdebt + curdebt (장기차입금 + 단기차입금)
            # 현금성자산: balances > cash
            EV = float(self.ohlcv.loc[code]['mktcap'] + (bs_trailing['longdebt'] + bs_trailing['curdebt'] - bs_trailing['cash'])*self.fin_unit)

            # 3. EV/Sales(매출액) 계산
            # 매출액: incomes > rev
            ES = float(EV / (is_trailing['rev']*self.fin_unit))
            
            # 4. 결측치 처리
            if EV <= 0 or float(is_trailing['rev']) <= 0: # EV 음수 or 매출액 음수인 경우
                ES = np.NaN
                
            # 5. 계산한 결과 DataFrame에 추가
            df_ES = df_ES.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'EV/Sales': ES }, ignore_index=True)
            
            print(code, ": Calculate EV/Sales ... OK")
        
        return df_ES
    
    # 안전마진이 있는 그레이엄의 NCAV(청산가치) 투자법
    def getNCAV(self):
        # NCAV 계산 후 저장할 DataFrame 생성
        df_NCAV = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 재무상태표 가져오기
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
        
            # 2. NCAV (유동자산 - 총부채) 계산
            # 유동자산: balances > curasset
            # 총 부채: balances > liab
            NCAV = float(bs_trailing['curasset'] - bs_trailing['liab']) * self.fin_unit

            # 3. 안전마진(NCAV - (시가총액*1.5)) 계산
            # 시가총액: ohlcv > mktcap
            SafetyMargin = NCAV - (self.ohlcv.loc[code]['mktcap'] * 1.5)
                
            # 4. 계산한 결과 DataFrame에 추가
            df_NCAV = df_NCAV.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'NCAV': SafetyMargin }, ignore_index=True)
            
            print(code, ": Calculate NCAV ... OK")
        
        return df_NCAV
    
    # 주가수익 성장비율
    def getPEG(self):
        # PEG 계산 후 저장할 DataFrame 생성
        df_PEG = pd.DataFrame
        
        # PER 계산하기
        df_PER = self.getPER()
        df_PER = df_PER.set_index('code') # 종목 코드를 인덱스로 사용
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 해당 종목 분기별 손익계산서 가져오기
            # > 4분기 전 트레일링 데이터 존재하지 않으므로 현재 분기(2022/03)와 3분기 전 (2021/06) 데이터 사용
            df_is = INCOMES().getStockIncomes(code, self.period)
            df_is4 = INCOMES().getStockIncomes(code, '2021/06')
            
            # 재무제표 데이터 충분하지 않은 경우
            #   또는 PER, ohlcv 데이터가 존재하지 않는 종목인 경우
            if df_is is None or df_is4 is None or code not in df_PER.index or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
            
            # 2. EPS 성장률 계산
            # (현재 EPS - 4분기 전 EPS) / 4분기 전 EPS * 100
            # EPS = 당기순이익/상장주식수
            # 당기순이익: incomes > netinc
            EPS = float((df_is['netinc'] * self.fin_unit) / self.ohlcv.loc[code]['shares']) # 현재 EPS
            EPS_4 = float((df_is4['netinc'] * self.fin_unit) / self.ohlcv.loc[code]['shares']) # 4분기 전 EPS
            EPS_Growth = (EPS - EPS_4) / EPS_4 * 100
                
            # 3. PEG 계산
            # PER / EPS 성장률
            if df_PER.loc[code]['PER'] == 0 or df_PER.loc[code]['PER'] == np.NaN or EPS_Growth <= 0:
                PEG = np.NaN # 결측치 처리
            else:
                PEG = df_PER.loc[code]['PER'] / EPS_Growth
                
            # 4. 계산한 결과 DataFrame에 추가
            df_PEG = df_PEG.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'PEG': PEG }, ignore_index=True)
            
            print(code, ": Calculate PEG ... OK")
        
        return df_PEG

## 종목 선택 함수

In [126]:
def stock_select(df_factor, mktcap_top, exchange, n, factor_list):
    basic_list = ['code', 'name', 'exchange', 'period', 'mktcap'] # 종목 코드, 이름, 주식 시장, 기간, 시가총액

    basic_list.extend(factor_list) # basic_list에 factor_list 요소 넣기

    df_select = df_factor.copy() # df_select에 df_factor 복사하기
    df_select = df_select[basic_list] # df_select에 df_select의 basic_list 요소 넣기

    df_select['score'] = 0 # df_select에 score 항목 추가 & 0 초기화

    # 시가총액 기준 내림차순 정렬 후, 상위 mktcap_top% 산출
    df_select = df_select.sort_values(by=['mktcap'], ascending=False).head(int(len(df_select) * mktcap_top))
    df_select = df_select.dropna()
    
    # 해당 exchange 종목만 골라내기
    if exchange.upper() != 'ALL':
        df_select = df_select[df_select['exchange']==exchange]
    
    # 최저값 이상 종목 추출
    floor = 0.1
    df_range = df_select.copy()
    
    for i in range(len(factor_list)):
        # 슬기로운 퀀트 투자 기준, 각 factor 최저값 설정
        if factor_list[i] == 'PER': # PER = 1
            floor = 1
        elif factor_list[i] == 'PBR' or factor_list[i] == 'PSR' or factor_list[i] == 'PCR': # PBR, PSR, PCR = 0.1
            floor = 0.1
        else: # 나머지 = 0
            floor = 0

        # 최저값 이상 종목만 추출
        df_range = df_range[df_range[factor_list[i]] >= floor]
        
    df_select = df_range.copy()

    # 팩터간의 점수 계산
    sort_desc = ['NCAV', 'ROA', 'ROE', 'GP/A', 'ASS turnover', 'REC turnover', 'INV turnover'] # 클수록 높은 점수 팩터 리스트
    
    for i in range(len(factor_list)):
        if factor_list[i] in sort_desc: # 클수록 높은 점수 팩터인 경우
            # (factor 값 - factor 최소값) / factor 최대값
            df_select[factor_list[i] + '_score'] = (df_select[factor_list[i]] - min(df_select[factor_list[i]]))
            df_select[factor_list[i] + '_score'] = df_select[factor_list[i] + '_score'] / max(df_select[factor_list[i] + '_score'])

            # 점수 = 위 계산 값 / factor 개수 * 100 > (0~100점으로 맞추기)
            df_select['score'] += (df_select[factor_list[i] + '_score'] / len(factor_list)) * 100
            
        else: # 작을수록 높은 점수
            # (factor 값 - factor 최대값) / factor 최소값
            df_select[factor_list[i] + '_score'] = (df_select[factor_list[i]] - max(df_select[factor_list[i]]))
            df_select[factor_list[i] + '_score'] = df_select[factor_list[i] + '_score']/min(df_select[factor_list[i] + '_score'])

            # 점수 = 위 계산 값 / factor 개수 * 100 > (0~100점으로 맞추기)
            df_select['score'] += (df_select[factor_list[i] + '_score'] / len(factor_list)) * 100

    # 상위 n개 종목 추출
    df_select = df_select.sort_values(by=['score'], ascending=False).head(n)

    # 종목 선택    
    stock_select = df_select[['code', 'name']]
    
    # 각 팩터 점수 추가
    for i in range(len(factor_list)):
        stock_select[factor_list[i]] = df_select[factor_list[i]]
        
    # 점수 합계
    stock_select['score'] = df_select['score']
    
    stock_select.reset_index(drop=True, inplace=True) # index 재설정
    stock_select.index = stock_select.index+1 # index 1부터 시작 (~30)

    return stock_select

### VALUATION 클래스 사용 예시

In [124]:
v = VALUATION('2022Q1')

init 실행 중 2022-06-17 01:02:33.914118
init 실행 완료 2022-06-17 01:12:27.197056


#### PER

In [44]:
# PER 구하기
per_list = v.getPER()
per_list

Empty DB Incomes
060310 : Not enough data to calculate ...
095570 : Calculate PER ... OK
006840 : Calculate PER ... OK
054620 : Calculate PER ... OK
265520 : Calculate PER ... OK
211270 : Calculate PER ... OK
027410 : Calculate PER ... OK
282330 : Calculate PER ... OK
032790 : Calculate PER ... OK
138930 : Calculate PER ... OK
001460 : Calculate PER ... OK
Empty DB Incomes
001465 : Not enough data to calculate ...
013720 : Calculate PER ... OK
001040 : Calculate PER ... OK
079160 : Calculate PER ... OK
035760 : Calculate PER ... OK
311690 : Calculate PER ... OK
Empty DB Incomes
00104K : Not enough data to calculate ...
000120 : Calculate PER ... OK
011150 : Calculate PER ... OK
Empty DB Incomes
011155 : Not enough data to calculate ...
Empty DB Incomes
001045 : Not enough data to calculate ...
097950 : Calculate PER ... OK
Empty DB Incomes
097955 : Not enough data to calculate ...
051500 : Calculate PER ... OK
058820 : Calculate PER ... OK
023460 : Calculate PER ... OK
056730 : Calcula

000660 : Calculate PER ... OK
048550 : Calculate PER ... OK
063440 : Calculate PER ... OK
Empty DB Incomes
950180 : Not enough data to calculate ...
064960 : Calculate PER ... OK
100840 : Calculate PER ... OK
003570 : Calculate PER ... OK
036530 : Calculate PER ... OK
005610 : Calculate PER ... OK
011810 : Calculate PER ... OK
077970 : Calculate PER ... OK
071970 : Calculate PER ... OK
002820 : Calculate PER ... OK
Empty DB Incomes
289080 : Not enough data to calculate ...
084870 : Not enough data to calculate ...
002710 : Calculate PER ... OK
089230 : Calculate PER ... OK
161570 : Calculate PER ... OK
192410 : Calculate PER ... OK
032540 : Calculate PER ... OK
048770 : Calculate PER ... OK
246690 : Calculate PER ... OK
317240 : Not enough data to calculate ...
002900 : Calculate PER ... OK
038340 : Calculate PER ... OK
073570 : Calculate PER ... OK
024070 : Calculate PER ... OK
057030 : Calculate PER ... OK
037270 : Calculate PER ... OK
040300 : Calculate PER ... OK
051390 : Calculate

020400 : Calculate PER ... OK
008830 : Calculate PER ... OK
048470 : Calculate PER ... OK
Empty DB Incomes
008110 : Not enough data to calculate ...
004780 : Calculate PER ... OK
005750 : Calculate PER ... OK
017650 : Calculate PER ... OK
006570 : Calculate PER ... OK
007720 : Calculate PER ... OK
317850 : Not enough data to calculate ...
290670 : Calculate PER ... OK
078140 : Calculate PER ... OK
001680 : Calculate PER ... OK
Empty DB Incomes
001685 : Not enough data to calculate ...
084690 : Not enough data to calculate ...
Empty DB Incomes
084695 : Not enough data to calculate ...
036480 : Calculate PER ... OK
128820 : Calculate PER ... OK
117580 : Calculate PER ... OK
025440 : Calculate PER ... OK
027830 : Calculate PER ... OK
104040 : Calculate PER ... OK
016710 : Calculate PER ... OK
387310 : Not enough data to calculate ...
397500 : Not enough data to calculate ...
336570 : Not enough data to calculate ...
020180 : Calculate PER ... OK
003540 : Calculate PER ... OK
Empty DB Inco

108490 : Not enough data to calculate ...
900260 : Not enough data to calculate ...
Empty DB Incomes
223220 : Not enough data to calculate ...
067730 : Calculate PER ... OK
071280 : Calculate PER ... OK
032350 : Calculate PER ... OK
089860 : Calculate PER ... OK
330590 : Calculate PER ... OK
000400 : Calculate PER ... OK
023530 : Calculate PER ... OK
004000 : Calculate PER ... OK
286940 : Calculate PER ... OK
280360 : Calculate PER ... OK
004990 : Calculate PER ... OK
Empty DB Incomes
00499K : Not enough data to calculate ...
005300 : Calculate PER ... OK
Empty DB Incomes
005305 : Not enough data to calculate ...
011170 : Calculate PER ... OK
002270 : Calculate PER ... OK
071840 : Calculate PER ... OK
038060 : Calculate PER ... OK
Empty DB Incomes
162120 : Not enough data to calculate ...
Empty DB Incomes
253610 : Not enough data to calculate ...
085370 : Calculate PER ... OK
Empty DB Incomes
08537M : Not enough data to calculate ...
060240 : Calculate PER ... OK
058470 : Calculate PER

007160 : Calculate PER ... OK
014710 : Calculate PER ... OK
006090 : Calculate PER ... OK
100090 : Calculate PER ... OK
122350 : Calculate PER ... OK
014970 : Calculate PER ... OK
018310 : Calculate PER ... OK
053700 : Calculate PER ... OK
009620 : Calculate PER ... OK
023600 : Calculate PER ... OK
001470 : Calculate PER ... OK
006400 : Calculate PER ... OK
Empty DB Incomes
006405 : Not enough data to calculate ...
006660 : Calculate PER ... OK
380320 : Not enough data to calculate ...
028260 : Calculate PER ... OK
Empty DB Incomes
02826K : Not enough data to calculate ...
207940 : Calculate PER ... OK
032830 : Calculate PER ... OK
377630 : Not enough data to calculate ...
018260 : Calculate PER ... OK
028050 : Calculate PER ... OK
009150 : Calculate PER ... OK
Empty DB Incomes
009155 : Not enough data to calculate ...
005930 : Calculate PER ... OK
Empty DB Incomes
005935 : Not enough data to calculate ...
001360 : Calculate PER ... OK
010140 : Calculate PER ... OK
Empty DB Incomes
010

...

313760 : Calculate PER ... OK
008600 : Calculate PER ... OK
900340 : Calculate PER ... OK
033270 : Calculate PER ... OK
064510 : Calculate PER ... OK
014830 : Calculate PER ... OK
036200 : Calculate PER ... OK
018000 : Calculate PER ... OK
000910 : Calculate PER ... OK
047400 : Calculate PER ... OK
203450 : Calculate PER ... OK
011330 : Calculate PER ... OK
077500 : Calculate PER ... OK
011320 : Calculate PER ... OK
086390 : Calculate PER ... OK
241690 : Calculate PER ... OK
142210 : Calculate PER ... OK
Empty DB Incomes
121060 : Not enough data to calculate ...
048430 : Calculate PER ... OK
206650 : Calculate PER ... OK
089850 : Calculate PER ... OK
Empty DB Incomes
084440 : Not enough data to calculate ...
032620 : Calculate PER ... OK
264450 : Calculate PER ... OK
078070 : Calculate PER ... OK
002920 : Calculate PER ... OK
024800 : Calculate PER ... OK
000700 : Calculate PER ... OK
054930 : Calculate PER ... OK
069330 : Calculate PER ... OK
049520 : Calculate PER ... OK
336060 : Not

276730 : Calculate PER ... OK
080220 : Calculate PER ... OK
006220 : Calculate PER ... OK
089590 : Calculate PER ... OK
216080 : Calculate PER ... OK
229000 : Calculate PER ... OK
Empty DB Incomes
248020 : Not enough data to calculate ...
082270 : Calculate PER ... OK
064800 : Calculate PER ... OK
041590 : Calculate PER ... OK
044060 : Calculate PER ... OK
004910 : Calculate PER ... OK
004700 : Calculate PER ... OK
001550 : Calculate PER ... OK
000480 : Calculate PER ... OK
120030 : Calculate PER ... OK
034940 : Calculate PER ... OK
067000 : Calculate PER ... OK
018470 : Calculate PER ... OK
002600 : Calculate PER ... OK
185750 : Calculate PER ... OK
063160 : Calculate PER ... OK
001630 : Calculate PER ... OK
033340 : Calculate PER ... OK
036930 : Calculate PER ... OK
044380 : Calculate PER ... OK
239340 : Calculate PER ... OK
051980 : Calculate PER ... OK
072020 : Calculate PER ... OK
000440 : Calculate PER ... OK
078650 : Calculate PER ... OK
228760 : Calculate PER ... OK
314130 : Ca

003240 : Calculate PER ... OK
011280 : Calculate PER ... OK
053620 : Calculate PER ... OK
004100 : Calculate PER ... OK
Empty DB Incomes
004105 : Not enough data to calculate ...
Empty DB Incomes
116100 : Not enough data to calculate ...
009410 : Calculate PER ... OK
Empty DB Incomes
009415 : Not enough data to calculate ...
044490 : Calculate PER ... OK
124560 : Calculate PER ... OK
001420 : Calculate PER ... OK
007980 : Calculate PER ... OK
191420 : Calculate PER ... OK
073640 : Calculate PER ... OK
Empty DB Incomes
182690 : Not enough data to calculate ...
066700 : Calculate PER ... OK
095610 : Calculate PER ... OK
131970 : Calculate PER ... OK
055490 : Calculate PER ... OK
Empty DB Incomes
308700 : Not enough data to calculate ...
089030 : Calculate PER ... OK
Empty DB Incomes
258050 : Not enough data to calculate ...
054450 : Calculate PER ... OK
091440 : Calculate PER ... OK
078000 : Calculate PER ... OK
200230 : Calculate PER ... OK
214420 : Calculate PER ... OK
Empty DB Incomes

000970 : Calculate PER ... OK
104700 : Calculate PER ... OK
017960 : Calculate PER ... OK
023760 : Calculate PER ... OK
054040 : Calculate PER ... OK
050540 : Calculate PER ... OK
161890 : Not enough data to calculate ...
024720 : Calculate PER ... OK
021650 : Calculate PER ... OK
161390 : Calculate PER ... OK
053590 : Calculate PER ... OK
034830 : Calculate PER ... OK
007280 : Calculate PER ... OK
032300 : Calculate PER ... OK
Empty DB Incomes
168490 : Not enough data to calculate ...
037230 : Calculate PER ... OK
010100 : Calculate PER ... OK
047810 : Calculate PER ... OK
123690 : Calculate PER ... OK
003350 : Calculate PER ... OK
030520 : Calculate PER ... OK
052600 : Calculate PER ... OK
011500 : Calculate PER ... OK
002390 : Calculate PER ... OK
256150 : Calculate PER ... OK
014790 : Calculate PER ... OK
092460 : Calculate PER ... OK
060980 : Calculate PER ... OK
053690 : Calculate PER ... OK
042700 : Calculate PER ... OK
008930 : Calculate PER ... OK
128940 : Not enough data to c

,code,name,exchange,period,mktcap,PER
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,3.499855
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.000000
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,12.243959
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,6.467678
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,27.084721
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,38.218970
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,19.427660


In [147]:
# PER 종목 선정
df_factor = per_list.copy()
factor_list = ['PER']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PER, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PER, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # PER, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,PER,score
1,950130,엑세스바이오,1.850054,100.000000
2,003380,하림지주,1.945864,99.980362
3,205470,휴마시스,2.058360,99.957305
4,096530,씨젠,4.951057,99.364409
5,049070,인탑스,5.454860,99.261148
6,090460,비에이치,6.851420,98.974904
7,108320,LX세미콘,6.892517,98.966481
8,086520,에코프로,7.108488,98.922215
9,121800,비덴트,7.802206,98.780028
10,036930,주성엔지니어링,8.104694,98.718029


#### PBR

In [72]:
# PBR 구하기
pbr_list = v.getPBR()
pbr_list

Empty DB Balances
060310 : Not enough data to calculate ...
095570 : Calculate PBR ... OK
006840 : Calculate PBR ... OK
054620 : Calculate PBR ... OK
265520 : Calculate PBR ... OK
211270 : Calculate PBR ... OK
027410 : Calculate PBR ... OK
282330 : Calculate PBR ... OK
032790 : Calculate PBR ... OK
138930 : Calculate PBR ... OK
001460 : Calculate PBR ... OK
Empty DB Balances
001465 : Not enough data to calculate ...
013720 : Calculate PBR ... OK
001040 : Calculate PBR ... OK
079160 : Calculate PBR ... OK
035760 : Calculate PBR ... OK
311690 : Calculate PBR ... OK
Empty DB Balances
00104K : Not enough data to calculate ...
000120 : Calculate PBR ... OK
011150 : Calculate PBR ... OK
Empty DB Balances
011155 : Not enough data to calculate ...
Empty DB Balances
001045 : Not enough data to calculate ...
097950 : Calculate PBR ... OK
Empty DB Balances
097955 : Not enough data to calculate ...
051500 : Calculate PBR ... OK
058820 : Calculate PBR ... OK
023460 : Calculate PBR ... OK
056730 : C

017670 : Calculate PBR ... OK
000660 : Calculate PBR ... OK
048550 : Calculate PBR ... OK
063440 : Calculate PBR ... OK
Empty DB Balances
950180 : Not enough data to calculate ...
064960 : Calculate PBR ... OK
100840 : Calculate PBR ... OK
003570 : Calculate PBR ... OK
036530 : Calculate PBR ... OK
005610 : Calculate PBR ... OK
011810 : Calculate PBR ... OK
077970 : Calculate PBR ... OK
071970 : Calculate PBR ... OK
002820 : Calculate PBR ... OK
Empty DB Balances
289080 : Not enough data to calculate ...
084870 : Not enough data to calculate ...
002710 : Calculate PBR ... OK
089230 : Calculate PBR ... OK
161570 : Calculate PBR ... OK
192410 : Calculate PBR ... OK
032540 : Calculate PBR ... OK
048770 : Calculate PBR ... OK
246690 : Calculate PBR ... OK
317240 : Not enough data to calculate ...
002900 : Calculate PBR ... OK
038340 : Calculate PBR ... OK
073570 : Calculate PBR ... OK
024070 : Calculate PBR ... OK
057030 : Calculate PBR ... OK
037270 : Calculate PBR ... OK
040300 : Calcula

020400 : Calculate PBR ... OK
008830 : Calculate PBR ... OK
048470 : Calculate PBR ... OK
Empty DB Balances
008110 : Not enough data to calculate ...
004780 : Calculate PBR ... OK
005750 : Calculate PBR ... OK
017650 : Calculate PBR ... OK
006570 : Calculate PBR ... OK
007720 : Calculate PBR ... OK
317850 : Not enough data to calculate ...
290670 : Calculate PBR ... OK
078140 : Calculate PBR ... OK
001680 : Calculate PBR ... OK
Empty DB Balances
001685 : Not enough data to calculate ...
084690 : Not enough data to calculate ...
Empty DB Balances
084695 : Not enough data to calculate ...
036480 : Calculate PBR ... OK
128820 : Calculate PBR ... OK
117580 : Calculate PBR ... OK
025440 : Calculate PBR ... OK
027830 : Calculate PBR ... OK
104040 : Calculate PBR ... OK
016710 : Calculate PBR ... OK
387310 : Not enough data to calculate ...
397500 : Not enough data to calculate ...
336570 : Not enough data to calculate ...
020180 : Calculate PBR ... OK
003540 : Calculate PBR ... OK
Empty DB B

108490 : Not enough data to calculate ...
900260 : Not enough data to calculate ...
Empty DB Balances
223220 : Not enough data to calculate ...
067730 : Calculate PBR ... OK
071280 : Calculate PBR ... OK
032350 : Calculate PBR ... OK
089860 : Calculate PBR ... OK
330590 : Calculate PBR ... OK
000400 : Calculate PBR ... OK
023530 : Calculate PBR ... OK
004000 : Calculate PBR ... OK
286940 : Calculate PBR ... OK
280360 : Calculate PBR ... OK
004990 : Calculate PBR ... OK
Empty DB Balances
00499K : Not enough data to calculate ...
005300 : Calculate PBR ... OK
Empty DB Balances
005305 : Not enough data to calculate ...
011170 : Calculate PBR ... OK
002270 : Calculate PBR ... OK
071840 : Calculate PBR ... OK
038060 : Calculate PBR ... OK
Empty DB Balances
162120 : Not enough data to calculate ...
Empty DB Balances
253610 : Not enough data to calculate ...
085370 : Calculate PBR ... OK
Empty DB Balances
08537M : Not enough data to calculate ...
060240 : Calculate PBR ... OK
058470 : Calcula

003960 : Calculate PBR ... OK
008040 : Calculate PBR ... OK
007160 : Calculate PBR ... OK
014710 : Calculate PBR ... OK
006090 : Calculate PBR ... OK
100090 : Calculate PBR ... OK
122350 : Calculate PBR ... OK
014970 : Calculate PBR ... OK
018310 : Calculate PBR ... OK
053700 : Calculate PBR ... OK
009620 : Calculate PBR ... OK
023600 : Calculate PBR ... OK
001470 : Calculate PBR ... OK
006400 : Calculate PBR ... OK
Empty DB Balances
006405 : Not enough data to calculate ...
006660 : Calculate PBR ... OK
380320 : Not enough data to calculate ...
028260 : Calculate PBR ... OK
Empty DB Balances
02826K : Not enough data to calculate ...
207940 : Calculate PBR ... OK
032830 : Calculate PBR ... OK
377630 : Not enough data to calculate ...
018260 : Calculate PBR ... OK
028050 : Calculate PBR ... OK
009150 : Calculate PBR ... OK
Empty DB Balances
009155 : Not enough data to calculate ...
005930 : Calculate PBR ... OK
Empty DB Balances
005935 : Not enough data to calculate ...
001360 : Calcula

...

136540 : Calculate PBR ... OK
320000 : Calculate PBR ... OK
097800 : Calculate PBR ... OK
192390 : Calculate PBR ... OK
313760 : Calculate PBR ... OK
008600 : Calculate PBR ... OK
900340 : Calculate PBR ... OK
033270 : Calculate PBR ... OK
064510 : Calculate PBR ... OK
014830 : Calculate PBR ... OK
036200 : Calculate PBR ... OK
018000 : Calculate PBR ... OK
000910 : Calculate PBR ... OK
047400 : Calculate PBR ... OK
203450 : Calculate PBR ... OK
011330 : Calculate PBR ... OK
077500 : Calculate PBR ... OK
011320 : Calculate PBR ... OK
086390 : Calculate PBR ... OK
241690 : Calculate PBR ... OK
142210 : Calculate PBR ... OK
Empty DB Balances
121060 : Not enough data to calculate ...
048430 : Calculate PBR ... OK
206650 : Calculate PBR ... OK
089850 : Calculate PBR ... OK
Empty DB Balances
084440 : Not enough data to calculate ...
032620 : Calculate PBR ... OK
264450 : Calculate PBR ... OK
078070 : Calculate PBR ... OK
002920 : Calculate PBR ... OK
024800 : Calculate PBR ... OK
000700 : C

001560 : Calculate PBR ... OK
199820 : Calculate PBR ... OK
023440 : Calculate PBR ... OK
038010 : Calculate PBR ... OK
002620 : Calculate PBR ... OK
276730 : Calculate PBR ... OK
080220 : Calculate PBR ... OK
006220 : Calculate PBR ... OK
089590 : Calculate PBR ... OK
216080 : Calculate PBR ... OK
229000 : Calculate PBR ... OK
Empty DB Balances
248020 : Not enough data to calculate ...
082270 : Calculate PBR ... OK
064800 : Calculate PBR ... OK
041590 : Calculate PBR ... OK
044060 : Calculate PBR ... OK
004910 : Calculate PBR ... OK
004700 : Calculate PBR ... OK
001550 : Calculate PBR ... OK
000480 : Calculate PBR ... OK
120030 : Calculate PBR ... OK
034940 : Calculate PBR ... OK
067000 : Calculate PBR ... OK
018470 : Calculate PBR ... OK
002600 : Calculate PBR ... OK
185750 : Calculate PBR ... OK
063160 : Calculate PBR ... OK
001630 : Calculate PBR ... OK
033340 : Calculate PBR ... OK
036930 : Calculate PBR ... OK
044380 : Calculate PBR ... OK
239340 : Calculate PBR ... OK
051980 : C

065130 : Calculate PBR ... OK
134580 : Calculate PBR ... OK
014580 : Calculate PBR ... OK
015890 : Calculate PBR ... OK
006890 : Calculate PBR ... OK
023160 : Calculate PBR ... OK
003240 : Calculate PBR ... OK
011280 : Calculate PBR ... OK
053620 : Calculate PBR ... OK
004100 : Calculate PBR ... OK
Empty DB Balances
004105 : Not enough data to calculate ...
Empty DB Balances
116100 : Not enough data to calculate ...
009410 : Calculate PBR ... OK
Empty DB Balances
009415 : Not enough data to calculate ...
044490 : Calculate PBR ... OK
124560 : Calculate PBR ... OK
001420 : Calculate PBR ... OK
007980 : Calculate PBR ... OK
191420 : Calculate PBR ... OK
073640 : Calculate PBR ... OK
Empty DB Balances
182690 : Not enough data to calculate ...
066700 : Calculate PBR ... OK
095610 : Calculate PBR ... OK
131970 : Calculate PBR ... OK
055490 : Calculate PBR ... OK
Empty DB Balances
308700 : Not enough data to calculate ...
089030 : Calculate PBR ... OK
Empty DB Balances
258050 : Not enough da

053300 : Calculate PBR ... OK
025770 : Calculate PBR ... OK
409570 : Not enough data to calculate ...
009540 : Calculate PBR ... OK
023350 : Calculate PBR ... OK
025890 : Calculate PBR ... OK
000970 : Calculate PBR ... OK
104700 : Calculate PBR ... OK
017960 : Calculate PBR ... OK
023760 : Calculate PBR ... OK
054040 : Calculate PBR ... OK
050540 : Calculate PBR ... OK
161890 : Not enough data to calculate ...
024720 : Calculate PBR ... OK
021650 : Calculate PBR ... OK
161390 : Calculate PBR ... OK
053590 : Calculate PBR ... OK
034830 : Calculate PBR ... OK
007280 : Calculate PBR ... OK
032300 : Calculate PBR ... OK
Empty DB Balances
168490 : Not enough data to calculate ...
037230 : Calculate PBR ... OK
010100 : Calculate PBR ... OK
047810 : Calculate PBR ... OK
123690 : Calculate PBR ... OK
003350 : Calculate PBR ... OK
030520 : Calculate PBR ... OK
052600 : Calculate PBR ... OK
011500 : Calculate PBR ... OK
002390 : Calculate PBR ... OK
256150 : Calculate PBR ... OK
014790 : Calcula

,code,name,exchange,period,mktcap,PBR
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.207604
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.063523
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.348534
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.496519
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,2.845191
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.496487
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,0.829170
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,1.538758
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.545216


In [148]:
# PBR 종목 선정
df_factor = pbr_list.copy()
factor_list = ['PBR']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PBR, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PBR, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # PBR, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,PBR,score
1,036830,솔브레인홀딩스,0.143742,100.000000
2,035760,CJ ENM,0.173449,99.870179
3,038500,삼표시멘트,0.218696,99.672447
4,178320,서진시스템,0.241380,99.573319
5,046890,서울반도체,0.243720,99.563091
6,049070,인탑스,0.247395,99.547031
7,034230,파라다이스,0.253088,99.522154
8,056190,에스에프에이,0.271509,99.441651
9,030520,한글과컴퓨터,0.272344,99.438005
10,078340,컴투스,0.274395,99.429040


#### PER + PBR 결합

In [150]:
# PER, PBR 결합 종목 선정
factor_list = ['PER', 'PBR']
df_factor = pd.merge(per_list, pbr_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PER+PBR, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # PER+PBR, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PER+PBR, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,PER,PBR,score
1,001230,동국제강,1.989159,0.147859,99.999796
2,001120,LX인터내셔널,2.005910,0.146550,99.999791
3,011200,HMM,1.714468,0.388620,99.999512
4,005880,대한해운,2.689223,0.156941,99.999454
5,298020,효성티앤씨,2.156513,0.327424,99.999411
6,011780,금호석유,2.520174,0.244289,99.999383
7,036460,한국가스공사,3.083438,0.103828,99.999361
8,042670,현대두산인프라코어,2.882788,0.184595,99.999317
9,018670,SK가스,3.161570,0.125308,99.999288
10,005490,POSCO홀딩스,3.207656,0.118080,99.999279


#### PSR

In [95]:
# PSR 구하기
psr_list = v.getPSR()
psr_list

Empty DB Incomes
060310 : Not enough data to calculate ...
095570 : Calculate PSR ... OK
006840 : Calculate PSR ... OK
054620 : Calculate PSR ... OK
265520 : Calculate PSR ... OK
211270 : Calculate PSR ... OK
027410 : Calculate PSR ... OK
282330 : Calculate PSR ... OK
032790 : Calculate PSR ... OK
138930 : Calculate PSR ... OK
001460 : Calculate PSR ... OK
Empty DB Incomes
001465 : Not enough data to calculate ...
013720 : Calculate PSR ... OK
001040 : Calculate PSR ... OK
079160 : Calculate PSR ... OK
035760 : Calculate PSR ... OK
311690 : Calculate PSR ... OK
Empty DB Incomes
00104K : Not enough data to calculate ...
000120 : Calculate PSR ... OK
011150 : Calculate PSR ... OK
Empty DB Incomes
011155 : Not enough data to calculate ...
Empty DB Incomes
001045 : Not enough data to calculate ...
097950 : Calculate PSR ... OK
Empty DB Incomes
097955 : Not enough data to calculate ...
051500 : Calculate PSR ... OK
058820 : Calculate PSR ... OK
023460 : Calculate PSR ... OK
056730 : Calcula

000660 : Calculate PSR ... OK
048550 : Calculate PSR ... OK
063440 : Calculate PSR ... OK
Empty DB Incomes
950180 : Not enough data to calculate ...
064960 : Calculate PSR ... OK
100840 : Calculate PSR ... OK
003570 : Calculate PSR ... OK
036530 : Calculate PSR ... OK
005610 : Calculate PSR ... OK
011810 : Calculate PSR ... OK
077970 : Calculate PSR ... OK
071970 : Calculate PSR ... OK
002820 : Calculate PSR ... OK
Empty DB Incomes
289080 : Not enough data to calculate ...
084870 : Not enough data to calculate ...
002710 : Calculate PSR ... OK
089230 : Calculate PSR ... OK
161570 : Calculate PSR ... OK
192410 : Calculate PSR ... OK
032540 : Calculate PSR ... OK
048770 : Calculate PSR ... OK
246690 : Calculate PSR ... OK
317240 : Not enough data to calculate ...
002900 : Calculate PSR ... OK
038340 : Calculate PSR ... OK
073570 : Calculate PSR ... OK
024070 : Calculate PSR ... OK
057030 : Calculate PSR ... OK
037270 : Calculate PSR ... OK
040300 : Calculate PSR ... OK
051390 : Calculate

020400 : Calculate PSR ... OK
008830 : Calculate PSR ... OK
048470 : Calculate PSR ... OK
Empty DB Incomes
008110 : Not enough data to calculate ...
004780 : Calculate PSR ... OK
005750 : Calculate PSR ... OK
017650 : Calculate PSR ... OK
006570 : Calculate PSR ... OK
007720 : Calculate PSR ... OK
317850 : Not enough data to calculate ...
290670 : Calculate PSR ... OK
078140 : Calculate PSR ... OK
001680 : Calculate PSR ... OK
Empty DB Incomes
001685 : Not enough data to calculate ...
084690 : Not enough data to calculate ...
Empty DB Incomes
084695 : Not enough data to calculate ...
036480 : Calculate PSR ... OK
128820 : Calculate PSR ... OK
117580 : Calculate PSR ... OK
025440 : Calculate PSR ... OK
027830 : Calculate PSR ... OK
104040 : Calculate PSR ... OK
016710 : Calculate PSR ... OK
387310 : Not enough data to calculate ...
397500 : Not enough data to calculate ...
336570 : Not enough data to calculate ...
020180 : Calculate PSR ... OK
003540 : Calculate PSR ... OK
Empty DB Inco

108490 : Not enough data to calculate ...
900260 : Not enough data to calculate ...
Empty DB Incomes
223220 : Not enough data to calculate ...
067730 : Calculate PSR ... OK
071280 : Calculate PSR ... OK
032350 : Calculate PSR ... OK
089860 : Calculate PSR ... OK
330590 : Calculate PSR ... OK
000400 : Calculate PSR ... OK
023530 : Calculate PSR ... OK
004000 : Calculate PSR ... OK
286940 : Calculate PSR ... OK
280360 : Calculate PSR ... OK
004990 : Calculate PSR ... OK
Empty DB Incomes
00499K : Not enough data to calculate ...
005300 : Calculate PSR ... OK
Empty DB Incomes
005305 : Not enough data to calculate ...
011170 : Calculate PSR ... OK
002270 : Calculate PSR ... OK
071840 : Calculate PSR ... OK
038060 : Calculate PSR ... OK
Empty DB Incomes
162120 : Not enough data to calculate ...
Empty DB Incomes
253610 : Not enough data to calculate ...
085370 : Calculate PSR ... OK
Empty DB Incomes
08537M : Not enough data to calculate ...
060240 : Calculate PSR ... OK
058470 : Calculate PSR

007160 : Calculate PSR ... OK
014710 : Calculate PSR ... OK
006090 : Calculate PSR ... OK
100090 : Calculate PSR ... OK
122350 : Calculate PSR ... OK
014970 : Calculate PSR ... OK
018310 : Calculate PSR ... OK
053700 : Calculate PSR ... OK
009620 : Calculate PSR ... OK
023600 : Calculate PSR ... OK
001470 : Calculate PSR ... OK
006400 : Calculate PSR ... OK
Empty DB Incomes
006405 : Not enough data to calculate ...
006660 : Calculate PSR ... OK
380320 : Not enough data to calculate ...
028260 : Calculate PSR ... OK
Empty DB Incomes
02826K : Not enough data to calculate ...
207940 : Calculate PSR ... OK
032830 : Calculate PSR ... OK
377630 : Not enough data to calculate ...
018260 : Calculate PSR ... OK
028050 : Calculate PSR ... OK
009150 : Calculate PSR ... OK
Empty DB Incomes
009155 : Not enough data to calculate ...
005930 : Calculate PSR ... OK
Empty DB Incomes
005935 : Not enough data to calculate ...
001360 : Calculate PSR ... OK
010140 : Calculate PSR ... OK
Empty DB Incomes
010

...

313760 : Calculate PSR ... OK
008600 : Calculate PSR ... OK
900340 : Calculate PSR ... OK
033270 : Calculate PSR ... OK
064510 : Calculate PSR ... OK
014830 : Calculate PSR ... OK
036200 : Calculate PSR ... OK
018000 : Calculate PSR ... OK
000910 : Calculate PSR ... OK
047400 : Calculate PSR ... OK
203450 : Calculate PSR ... OK
011330 : Calculate PSR ... OK
077500 : Calculate PSR ... OK
011320 : Calculate PSR ... OK
086390 : Calculate PSR ... OK
241690 : Calculate PSR ... OK
142210 : Calculate PSR ... OK
Empty DB Incomes
121060 : Not enough data to calculate ...
048430 : Calculate PSR ... OK
206650 : Calculate PSR ... OK
089850 : Calculate PSR ... OK
Empty DB Incomes
084440 : Not enough data to calculate ...
032620 : Calculate PSR ... OK
264450 : Calculate PSR ... OK
078070 : Calculate PSR ... OK
002920 : Calculate PSR ... OK
024800 : Calculate PSR ... OK
000700 : Calculate PSR ... OK
054930 : Calculate PSR ... OK
069330 : Calculate PSR ... OK
049520 : Calculate PSR ... OK
336060 : Not

276730 : Calculate PSR ... OK
080220 : Calculate PSR ... OK
006220 : Calculate PSR ... OK
089590 : Calculate PSR ... OK
216080 : Calculate PSR ... OK
229000 : Calculate PSR ... OK
Empty DB Incomes
248020 : Not enough data to calculate ...
082270 : Calculate PSR ... OK
064800 : Calculate PSR ... OK
041590 : Calculate PSR ... OK
044060 : Calculate PSR ... OK
004910 : Calculate PSR ... OK
004700 : Calculate PSR ... OK
001550 : Calculate PSR ... OK
000480 : Calculate PSR ... OK
120030 : Calculate PSR ... OK
034940 : Calculate PSR ... OK
067000 : Calculate PSR ... OK
018470 : Calculate PSR ... OK
002600 : Calculate PSR ... OK
185750 : Calculate PSR ... OK
063160 : Calculate PSR ... OK
001630 : Calculate PSR ... OK
033340 : Calculate PSR ... OK
036930 : Calculate PSR ... OK
044380 : Calculate PSR ... OK
239340 : Calculate PSR ... OK
051980 : Calculate PSR ... OK
072020 : Calculate PSR ... OK
000440 : Calculate PSR ... OK
078650 : Calculate PSR ... OK
228760 : Calculate PSR ... OK
314130 : Ca

003240 : Calculate PSR ... OK
011280 : Calculate PSR ... OK
053620 : Calculate PSR ... OK
004100 : Calculate PSR ... OK
Empty DB Incomes
004105 : Not enough data to calculate ...
Empty DB Incomes
116100 : Not enough data to calculate ...
009410 : Calculate PSR ... OK
Empty DB Incomes
009415 : Not enough data to calculate ...
044490 : Calculate PSR ... OK
124560 : Calculate PSR ... OK
001420 : Calculate PSR ... OK
007980 : Calculate PSR ... OK
191420 : Calculate PSR ... OK
073640 : Calculate PSR ... OK
Empty DB Incomes
182690 : Not enough data to calculate ...
066700 : Calculate PSR ... OK
095610 : Calculate PSR ... OK
131970 : Calculate PSR ... OK
055490 : Calculate PSR ... OK
Empty DB Incomes
308700 : Not enough data to calculate ...
089030 : Calculate PSR ... OK
Empty DB Incomes
258050 : Not enough data to calculate ...
054450 : Calculate PSR ... OK
091440 : Calculate PSR ... OK
078000 : Calculate PSR ... OK
200230 : Calculate PSR ... OK
214420 : Calculate PSR ... OK
Empty DB Incomes

000970 : Calculate PSR ... OK
104700 : Calculate PSR ... OK
017960 : Calculate PSR ... OK
023760 : Calculate PSR ... OK
054040 : Calculate PSR ... OK
050540 : Calculate PSR ... OK
161890 : Not enough data to calculate ...
024720 : Calculate PSR ... OK
021650 : Calculate PSR ... OK
161390 : Calculate PSR ... OK
053590 : Calculate PSR ... OK
034830 : Calculate PSR ... OK
007280 : Calculate PSR ... OK
032300 : Calculate PSR ... OK
Empty DB Incomes
168490 : Not enough data to calculate ...
037230 : Calculate PSR ... OK
010100 : Calculate PSR ... OK
047810 : Calculate PSR ... OK
123690 : Calculate PSR ... OK
003350 : Calculate PSR ... OK
030520 : Calculate PSR ... OK
052600 : Calculate PSR ... OK
011500 : Calculate PSR ... OK
002390 : Calculate PSR ... OK
256150 : Calculate PSR ... OK
014790 : Calculate PSR ... OK
092460 : Calculate PSR ... OK
060980 : Calculate PSR ... OK
053690 : Calculate PSR ... OK
042700 : Calculate PSR ... OK
008930 : Calculate PSR ... OK
128940 : Not enough data to c

,code,name,exchange,period,mktcap,PSR
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.267959
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.087992
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,6.429352
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.731407
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,2.017709
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,7.794003
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.616329


In [151]:
# PSR 종목 선정
df_factor = psr_list.copy()
factor_list = ['PSR']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PSR, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PSR, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # PSR, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,PSR,score
1,091700,파트론,0.477121,100.000000
2,049070,인탑스,0.531317,99.997909
3,950130,엑세스바이오,0.602210,99.995173
4,178320,서진시스템,0.605168,99.995059
5,035600,KG이니시스,0.629129,99.994135
6,046890,서울반도체,0.643174,99.993593
7,090460,비에이치,0.652457,99.993234
8,035760,CJ ENM,0.775059,99.988504
9,033290,코웰패션,0.825607,99.986553
10,056190,에스에프에이,0.909761,99.983306


#### PCR

In [100]:
# PCR 구하기
pcr_list = v.getPCR()
pcr_list

Empty DB Cashflow
060310 : Not enough data to calculate ...
095570 : Calculate PCR ... OK
006840 : Calculate PCR ... OK
054620 : Calculate PCR ... OK
265520 : Calculate PCR ... OK
211270 : Calculate PCR ... OK
027410 : Calculate PCR ... OK
282330 : Calculate PCR ... OK
032790 : Calculate PCR ... OK
138930 : Calculate PCR ... OK
001460 : Calculate PCR ... OK
Empty DB Cashflow
001465 : Not enough data to calculate ...
013720 : Calculate PCR ... OK
001040 : Calculate PCR ... OK
079160 : Calculate PCR ... OK
035760 : Calculate PCR ... OK
311690 : Calculate PCR ... OK
Empty DB Cashflow
00104K : Not enough data to calculate ...
000120 : Calculate PCR ... OK
011150 : Calculate PCR ... OK
Empty DB Cashflow
011155 : Not enough data to calculate ...
Empty DB Cashflow
001045 : Not enough data to calculate ...
097950 : Calculate PCR ... OK
Empty DB Cashflow
097955 : Not enough data to calculate ...
051500 : Calculate PCR ... OK
058820 : Calculate PCR ... OK
023460 : Calculate PCR ... OK
056730 : C

017670 : Calculate PCR ... OK
000660 : Calculate PCR ... OK
048550 : Calculate PCR ... OK
063440 : Calculate PCR ... OK
Empty DB Cashflow
950180 : Not enough data to calculate ...
064960 : Calculate PCR ... OK
100840 : Calculate PCR ... OK
003570 : Calculate PCR ... OK
036530 : Calculate PCR ... OK
005610 : Calculate PCR ... OK
011810 : Calculate PCR ... OK
077970 : Calculate PCR ... OK
071970 : Calculate PCR ... OK
002820 : Calculate PCR ... OK
Empty DB Cashflow
289080 : Not enough data to calculate ...
084870 : Not enough data to calculate ...
002710 : Calculate PCR ... OK
089230 : Calculate PCR ... OK
161570 : Calculate PCR ... OK
192410 : Calculate PCR ... OK
032540 : Calculate PCR ... OK
048770 : Calculate PCR ... OK
246690 : Calculate PCR ... OK
317240 : Not enough data to calculate ...
002900 : Calculate PCR ... OK
038340 : Calculate PCR ... OK
073570 : Calculate PCR ... OK
024070 : Calculate PCR ... OK
057030 : Calculate PCR ... OK
037270 : Calculate PCR ... OK
040300 : Calcula

020400 : Calculate PCR ... OK
008830 : Calculate PCR ... OK
048470 : Calculate PCR ... OK
Empty DB Cashflow
008110 : Not enough data to calculate ...
004780 : Calculate PCR ... OK
005750 : Calculate PCR ... OK
017650 : Calculate PCR ... OK
006570 : Calculate PCR ... OK
007720 : Calculate PCR ... OK
317850 : Not enough data to calculate ...
290670 : Calculate PCR ... OK
078140 : Calculate PCR ... OK
001680 : Calculate PCR ... OK
Empty DB Cashflow
001685 : Not enough data to calculate ...
084690 : Not enough data to calculate ...
Empty DB Cashflow
084695 : Not enough data to calculate ...
036480 : Calculate PCR ... OK
128820 : Calculate PCR ... OK
117580 : Calculate PCR ... OK
025440 : Calculate PCR ... OK
027830 : Calculate PCR ... OK
104040 : Calculate PCR ... OK
016710 : Calculate PCR ... OK
387310 : Not enough data to calculate ...
397500 : Not enough data to calculate ...
336570 : Not enough data to calculate ...
020180 : Calculate PCR ... OK
003540 : Calculate PCR ... OK
Empty DB C

108490 : Not enough data to calculate ...
900260 : Not enough data to calculate ...
Empty DB Cashflow
223220 : Not enough data to calculate ...
067730 : Calculate PCR ... OK
071280 : Calculate PCR ... OK
032350 : Calculate PCR ... OK
089860 : Calculate PCR ... OK
330590 : Calculate PCR ... OK
000400 : Calculate PCR ... OK
023530 : Calculate PCR ... OK
004000 : Calculate PCR ... OK
286940 : Calculate PCR ... OK
280360 : Calculate PCR ... OK
004990 : Calculate PCR ... OK
Empty DB Cashflow
00499K : Not enough data to calculate ...
005300 : Calculate PCR ... OK
Empty DB Cashflow
005305 : Not enough data to calculate ...
011170 : Calculate PCR ... OK
002270 : Calculate PCR ... OK
071840 : Calculate PCR ... OK
038060 : Calculate PCR ... OK
Empty DB Cashflow
162120 : Not enough data to calculate ...
Empty DB Cashflow
253610 : Not enough data to calculate ...
085370 : Calculate PCR ... OK
Empty DB Cashflow
08537M : Not enough data to calculate ...
060240 : Calculate PCR ... OK
058470 : Calcula

003960 : Calculate PCR ... OK
008040 : Calculate PCR ... OK
007160 : Calculate PCR ... OK
014710 : Calculate PCR ... OK
006090 : Calculate PCR ... OK
100090 : Calculate PCR ... OK
122350 : Calculate PCR ... OK
014970 : Calculate PCR ... OK
018310 : Calculate PCR ... OK
053700 : Calculate PCR ... OK
009620 : Calculate PCR ... OK
023600 : Calculate PCR ... OK
001470 : Calculate PCR ... OK
006400 : Calculate PCR ... OK
Empty DB Cashflow
006405 : Not enough data to calculate ...
006660 : Calculate PCR ... OK
380320 : Not enough data to calculate ...
028260 : Calculate PCR ... OK
Empty DB Cashflow
02826K : Not enough data to calculate ...
207940 : Calculate PCR ... OK
032830 : Calculate PCR ... OK
377630 : Not enough data to calculate ...
018260 : Calculate PCR ... OK
028050 : Calculate PCR ... OK
009150 : Calculate PCR ... OK
Empty DB Cashflow
009155 : Not enough data to calculate ...
005930 : Calculate PCR ... OK
Empty DB Cashflow
005935 : Not enough data to calculate ...
001360 : Calcula

...

136540 : Calculate PCR ... OK
320000 : Calculate PCR ... OK
097800 : Calculate PCR ... OK
192390 : Calculate PCR ... OK
313760 : Calculate PCR ... OK
008600 : Calculate PCR ... OK
900340 : Calculate PCR ... OK
033270 : Calculate PCR ... OK
064510 : Calculate PCR ... OK
014830 : Calculate PCR ... OK
036200 : Calculate PCR ... OK
018000 : Calculate PCR ... OK
000910 : Calculate PCR ... OK
047400 : Calculate PCR ... OK
203450 : Calculate PCR ... OK
011330 : Calculate PCR ... OK
077500 : Calculate PCR ... OK
011320 : Calculate PCR ... OK
086390 : Calculate PCR ... OK
241690 : Calculate PCR ... OK
142210 : Calculate PCR ... OK
Empty DB Cashflow
121060 : Not enough data to calculate ...
048430 : Calculate PCR ... OK
206650 : Calculate PCR ... OK
089850 : Calculate PCR ... OK
Empty DB Cashflow
084440 : Not enough data to calculate ...
032620 : Calculate PCR ... OK
264450 : Calculate PCR ... OK
078070 : Calculate PCR ... OK
002920 : Calculate PCR ... OK
024800 : Calculate PCR ... OK
000700 : C

001560 : Calculate PCR ... OK
199820 : Calculate PCR ... OK
023440 : Calculate PCR ... OK
038010 : Calculate PCR ... OK
002620 : Calculate PCR ... OK
276730 : Calculate PCR ... OK
080220 : Calculate PCR ... OK
006220 : Calculate PCR ... OK
089590 : Calculate PCR ... OK
216080 : Calculate PCR ... OK
229000 : Calculate PCR ... OK
Empty DB Cashflow
248020 : Not enough data to calculate ...
082270 : Calculate PCR ... OK
064800 : Calculate PCR ... OK
041590 : Calculate PCR ... OK
044060 : Calculate PCR ... OK
004910 : Calculate PCR ... OK
004700 : Calculate PCR ... OK
001550 : Calculate PCR ... OK
000480 : Calculate PCR ... OK
120030 : Calculate PCR ... OK
034940 : Calculate PCR ... OK
067000 : Calculate PCR ... OK
018470 : Calculate PCR ... OK
002600 : Calculate PCR ... OK
185750 : Calculate PCR ... OK
063160 : Calculate PCR ... OK
001630 : Calculate PCR ... OK
033340 : Calculate PCR ... OK
036930 : Calculate PCR ... OK
044380 : Calculate PCR ... OK
239340 : Calculate PCR ... OK
051980 : C

065130 : Calculate PCR ... OK
134580 : Calculate PCR ... OK
014580 : Calculate PCR ... OK
015890 : Calculate PCR ... OK
006890 : Calculate PCR ... OK
023160 : Calculate PCR ... OK
003240 : Calculate PCR ... OK
011280 : Calculate PCR ... OK
053620 : Calculate PCR ... OK
004100 : Calculate PCR ... OK
Empty DB Cashflow
004105 : Not enough data to calculate ...
Empty DB Cashflow
116100 : Not enough data to calculate ...
009410 : Calculate PCR ... OK
Empty DB Cashflow
009415 : Not enough data to calculate ...
044490 : Calculate PCR ... OK
124560 : Calculate PCR ... OK
001420 : Calculate PCR ... OK
007980 : Calculate PCR ... OK
191420 : Calculate PCR ... OK
073640 : Calculate PCR ... OK
Empty DB Cashflow
182690 : Not enough data to calculate ...
066700 : Calculate PCR ... OK
095610 : Calculate PCR ... OK
131970 : Calculate PCR ... OK
055490 : Calculate PCR ... OK
Empty DB Cashflow
308700 : Not enough data to calculate ...
089030 : Calculate PCR ... OK
Empty DB Cashflow
258050 : Not enough da

053300 : Calculate PCR ... OK
025770 : Calculate PCR ... OK
409570 : Not enough data to calculate ...
009540 : Calculate PCR ... OK
023350 : Calculate PCR ... OK
025890 : Calculate PCR ... OK
000970 : Calculate PCR ... OK
104700 : Calculate PCR ... OK
017960 : Calculate PCR ... OK
023760 : Calculate PCR ... OK
054040 : Calculate PCR ... OK
050540 : Calculate PCR ... OK
161890 : Not enough data to calculate ...
024720 : Calculate PCR ... OK
021650 : Calculate PCR ... OK
161390 : Calculate PCR ... OK
053590 : Calculate PCR ... OK
034830 : Calculate PCR ... OK
007280 : Calculate PCR ... OK
032300 : Calculate PCR ... OK
Empty DB Cashflow
168490 : Not enough data to calculate ...
037230 : Calculate PCR ... OK
010100 : Calculate PCR ... OK
047810 : Calculate PCR ... OK
123690 : Calculate PCR ... OK
003350 : Calculate PCR ... OK
030520 : Calculate PCR ... OK
052600 : Calculate PCR ... OK
011500 : Calculate PCR ... OK
002390 : Calculate PCR ... OK
256150 : Calculate PCR ... OK
014790 : Calcula

,code,name,exchange,period,mktcap,PCR
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.000000
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.000000
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.000000
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,5.332948
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,7.892564
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,0.000000
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,7.803157


In [ ]:
# PCR 종목 선정
df_factor = pcr_list.copy()
factor_list = ['PCR']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PCR, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PCR, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # PCR, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

#### 4대장 결합 (PER + PBR + PSR + PCR)

In [152]:
# 4대장 결합 종목 선정
factor_list = ['PER', 'PBR', 'PSR', 'PCR']

# 결합하기
df_factor = pd.merge(per_list, pbr_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])
df_factor = pd.merge(df_factor, psr_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])
df_factor = pd.merge(df_factor, pcr_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PER+PBR, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PER+PBR, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,PER,PBR,PSR,PCR,score
1,003070,코오롱글로벌,3.844197,0.231416,0.116367,1.858922,99.987166
2,001740,SK네트웍스,17.357580,0.123329,0.108819,2.216611,99.986252
3,001230,동국제강,1.989159,0.147859,0.200418,4.020328,99.892514
4,049770,동원F&B,11.508513,0.218263,0.192383,4.445341,99.892003
5,034220,LG디스플레이,6.587205,0.127632,0.250753,1.346730,99.880496
6,298020,효성티앤씨,2.156513,0.327424,0.215951,4.484545,99.873984
7,097950,CJ제일제당,6.036377,0.129005,0.205327,5.739855,99.867547
8,298040,효성중공업,11.413018,0.139740,0.183432,7.211524,99.867471
9,034310,NICE,7.328878,0.123571,0.252046,2.815003,99.862218
10,007070,GS리테일,3.844619,0.188247,0.289239,4.006598,99.818141


#### EV/EBITDA

In [132]:
# EV/EBITDA 구하기
EE_list = v.getEVEBITDA()
EE_list

Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
060310 : Not enough data to calculate ...
095570 : Calculate EV/EBITDA ... OK
006840 : Calculate EV/EBITDA ... OK
054620 : Calculate EV/EBITDA ... OK
265520 : Calculate EV/EBITDA ... OK
211270 : Calculate EV/EBITDA ... OK
027410 : Calculate EV/EBITDA ... OK
282330 : Calculate EV/EBITDA ... OK
032790 : Calculate EV/EBITDA ... OK
138930 : Calculate EV/EBITDA ... OK
001460 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
001465 : Not enough data to calculate ...
013720 : Calculate EV/EBITDA ... OK
001040 : Calculate EV/EBITDA ... OK
079160 : Calculate EV/EBITDA ... OK
035760 : Calculate EV/EBITDA ... OK
311690 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
00104K : Not enough data to calculate ...
000120 : Calculate EV/EBITDA ... OK
011150 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
011155 : Not enough data to calculate ...
Empty DB

024940 : Calculate EV/EBITDA ... OK
005490 : Calculate EV/EBITDA ... OK
218410 : Calculate EV/EBITDA ... OK
327260 : Calculate EV/EBITDA ... OK
091340 : Calculate EV/EBITDA ... OK
010950 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
010955 : Not enough data to calculate ...
019550 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
950110 : Not enough data to calculate ...
034120 : Calculate EV/EBITDA ... OK
046140 : Calculate EV/EBITDA ... OK
036120 : Calculate EV/EBITDA ... OK
099220 : Calculate EV/EBITDA ... OK
036540 : Calculate EV/EBITDA ... OK
255220 : Calculate EV/EBITDA ... OK
040610 : Calculate EV/EBITDA ... OK
049470 : Calculate EV/EBITDA ... OK
184230 : Calculate EV/EBITDA ... OK
005090 : Calculate EV/EBITDA ... OK
016250 : Calculate EV/EBITDA ... OK
001380 : Calculate EV/EBITDA ... OK
004060 : Calculate EV/EBITDA ... OK
001770 : Calculate EV/EBITDA ... OK
002360 : Calculate EV/EBITDA ... OK
009160 : Calculate

Empty DB Balances
Empty DB Cashflow
257990 : Not enough data to calculate ...
138610 : Not enough data to calculate ...
130580 : Calculate EV/EBITDA ... OK
036800 : Calculate EV/EBITDA ... OK
267320 : Calculate EV/EBITDA ... OK
001260 : Calculate EV/EBITDA ... OK
008350 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
008355 : Not enough data to calculate ...
004270 : Calculate EV/EBITDA ... OK
003920 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
003925 : Not enough data to calculate ...
025860 : Calculate EV/EBITDA ... OK
111710 : Calculate EV/EBITDA ... OK
091590 : Calculate EV/EBITDA ... OK
168330 : Calculate EV/EBITDA ... OK
094860 : Calculate EV/EBITDA ... OK
253590 : Calculate EV/EBITDA ... OK
212560 : Calculate EV/EBITDA ... OK
095660 : Calculate EV/EBITDA ... OK
042420 : Calculate EV/EBITDA ... OK
950220 : Calculate EV/EBITDA ... OK
311390 : Calculate EV/EBITDA ... OK
085910 : Calculate EV/EBITDA ... OK
092730

192080 : Not enough data to calculate ...
299170 : Not enough data to calculate ...
012510 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
302920 : Not enough data to calculate ...
213420 : Calculate EV/EBITDA ... OK
317330 : Calculate EV/EBITDA ... OK
077360 : Calculate EV/EBITDA ... OK
004830 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
004835 : Not enough data to calculate ...
090410 : Calculate EV/EBITDA ... OK
024900 : Calculate EV/EBITDA ... OK
263600 : Calculate EV/EBITDA ... OK
194480 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
348840 : Not enough data to calculate ...
263800 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
199150 : Not enough data to calculate ...
206560 : Calculate EV/EBITDA ... OK
261200 : Calculate EV/EBITDA ... OK
145720 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
227420 : Not en

060240 : Calculate EV/EBITDA ... OK
058470 : Calculate EV/EBITDA ... OK
039980 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
019570 : Not enough data to calculate ...
016100 : Calculate EV/EBITDA ... OK
012700 : Calculate EV/EBITDA ... OK
302550 : Calculate EV/EBITDA ... OK
377450 : Calculate EV/EBITDA ... OK
277070 : Calculate EV/EBITDA ... OK
042500 : Calculate EV/EBITDA ... OK
219420 : Calculate EV/EBITDA ... OK
027740 : Calculate EV/EBITDA ... OK
195500 : Calculate EV/EBITDA ... OK
222810 : Calculate EV/EBITDA ... OK
305090 : Calculate EV/EBITDA ... OK
098120 : Calculate EV/EBITDA ... OK
147760 : Calculate EV/EBITDA ... OK
377480 : Calculate EV/EBITDA ... OK
038290 : Calculate EV/EBITDA ... OK
204320 : Calculate EV/EBITDA ... OK
001080 : Calculate EV/EBITDA ... OK
220630 : Not enough data to calculate ...
267980 : Calculate EV/EBITDA ... OK
005990 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
127160 : Not enoug

...

123330 : Calculate EV/EBITDA ... OK
159580 : Calculate EV/EBITDA ... OK
147830 : Calculate EV/EBITDA ... OK
033100 : Calculate EV/EBITDA ... OK
079370 : Calculate EV/EBITDA ... OK
054950 : Calculate EV/EBITDA ... OK
090470 : Calculate EV/EBITDA ... OK
287410 : Calculate EV/EBITDA ... OK
137950 : Calculate EV/EBITDA ... OK
033320 : Calculate EV/EBITDA ... OK
348950 : Calculate EV/EBITDA ... OK
204270 : Not enough data to calculate ...
194370 : Calculate EV/EBITDA ... OK
026040 : Calculate EV/EBITDA ... OK
126880 : Calculate EV/EBITDA ... OK
322510 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
254160 : Not enough data to calculate ...
033050 : Calculate EV/EBITDA ... OK
094970 : Not enough data to calculate ...
058420 : Calculate EV/EBITDA ... OK
025620 : Calculate EV/EBITDA ... OK
036420 : Calculate EV/EBITDA ... OK
089790 : Calculate EV/EBITDA ... OK
030000 : Calculate EV/EBITDA ... OK
052670 : Calculate EV/EBITDA ... OK
271980 : Calculate EV/EBITDA 

222040 : Calculate EV/EBITDA ... OK
241710 : Calculate EV/EBITDA ... OK
005070 : Calculate EV/EBITDA ... OK
005420 : Calculate EV/EBITDA ... OK
069110 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
169670 : Not enough data to calculate ...
071950 : Calculate EV/EBITDA ... OK
166480 : Calculate EV/EBITDA ... OK
045970 : Calculate EV/EBITDA ... OK
196450 : Calculate EV/EBITDA ... OK
029960 : Calculate EV/EBITDA ... OK
002020 : Calculate EV/EBITDA ... OK
003070 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
003075 : Not enough data to calculate ...
102940 : Not enough data to calculate ...
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
002025 : Not enough data to calculate ...
120110 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
120115 : Not enough data to calculate ...
950160 : Calculate EV/EBITDA ... OK
138490 : Calculate EV/EBITDA ... OK
021240 : Calculate EV/EBITDA ... OK
03

189690 : Calculate EV/EBITDA ... OK
318020 : Calculate EV/EBITDA ... OK
256630 : Calculate EV/EBITDA ... OK
331380 : Calculate EV/EBITDA ... OK
141020 : Calculate EV/EBITDA ... OK
041020 : Calculate EV/EBITDA ... OK
114630 : Calculate EV/EBITDA ... OK
123010 : Calculate EV/EBITDA ... OK
290720 : Not enough data to calculate ...
005670 : Calculate EV/EBITDA ... OK
094940 : Calculate EV/EBITDA ... OK
007330 : Calculate EV/EBITDA ... OK
017810 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
093380 : Not enough data to calculate ...
023900 : Calculate EV/EBITDA ... OK
103140 : Not enough data to calculate ...
005810 : Calculate EV/EBITDA ... OK
371950 : Calculate EV/EBITDA ... OK
370090 : Calculate EV/EBITDA ... OK
220100 : Not enough data to calculate ...
035200 : Calculate EV/EBITDA ... OK
334970 : Calculate EV/EBITDA ... OK
950210 : Calculate EV/EBITDA ... OK
203690 : Calculate EV/EBITDA ... OK
321260 : Not enough data to calculate ...
Empty DB Incomes


002320 : Calculate EV/EBITDA ... OK
003480 : Calculate EV/EBITDA ... OK
180640 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
18064K : Not enough data to calculate ...
005110 : Calculate EV/EBITDA ... OK
043090 : Calculate EV/EBITDA ... OK
079170 : Calculate EV/EBITDA ... OK
009460 : Calculate EV/EBITDA ... OK
086960 : Calculate EV/EBITDA ... OK
372910 : Calculate EV/EBITDA ... OK
054920 : Calculate EV/EBITDA ... OK
002680 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
066110 : Not enough data to calculate ...
000880 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
00088K : Not enough data to calculate ...
088350 : Calculate EV/EBITDA ... OK
000370 : Calculate EV/EBITDA ... OK
009830 : Calculate EV/EBITDA ... OK
Empty DB Incomes
Empty DB Balances
Empty DB Cashflow
009835 : Not enough data to calculate ...
272210 : Calculate EV/EBITDA ... OK
012450 : Calculate EV/EBITDA ... OK
Empty DB

,code,name,exchange,period,mktcap,EV/EBITDA
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,8.879163
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,24.892799
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,91.825158
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,27.008583
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,42.306290
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,42.677096
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,26.261981


In [153]:
# EV/EBITDA 종목 선정
df_factor = EE_list.copy()
factor_list = ['EV/EBITDA']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # EV/EBITDA, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # EV/EBITDA, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # EV/EBITDA, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,EV/EBITDA,score
1,030520,한글과컴퓨터,5.594059,100.000000
2,049070,인탑스,10.145944,99.840674
3,033640,네패스,10.203910,99.838645
4,035600,KG이니시스,12.062348,99.773595
5,095660,네오위즈,13.014478,99.740268
6,091700,파트론,15.281617,99.660913
7,056190,에스에프에이,19.733836,99.505076
8,215200,메가스터디교육,20.061508,99.493606
9,067310,하나마이크론,20.651306,99.472962
10,215000,골프존,21.221381,99.453008


#### EV/Sales

In [142]:
# EV/Sales 구하기
ES_list = v.getEVSales()
ES_list

Empty DB Incomes
Empty DB Balances
060310 : Not enough data to calculate ...
095570 : Calculate EV/Sales ... OK
006840 : Calculate EV/Sales ... OK
054620 : Calculate EV/Sales ... OK
265520 : Calculate EV/Sales ... OK
211270 : Calculate EV/Sales ... OK
027410 : Calculate EV/Sales ... OK
282330 : Calculate EV/Sales ... OK
032790 : Calculate EV/Sales ... OK
138930 : Calculate EV/Sales ... OK
001460 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
001465 : Not enough data to calculate ...
013720 : Calculate EV/Sales ... OK
001040 : Calculate EV/Sales ... OK
079160 : Calculate EV/Sales ... OK
035760 : Calculate EV/Sales ... OK
311690 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
00104K : Not enough data to calculate ...
000120 : Calculate EV/Sales ... OK
011150 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
011155 : Not enough data to calculate ...
Empty DB Incomes
Empty DB Balances
001045 : Not enough data to calculate ...
097950 : Calculate E

040610 : Calculate EV/Sales ... OK
049470 : Calculate EV/Sales ... OK
184230 : Calculate EV/Sales ... OK
005090 : Calculate EV/Sales ... OK
016250 : Calculate EV/Sales ... OK
001380 : Calculate EV/Sales ... OK
004060 : Calculate EV/Sales ... OK
001770 : Calculate EV/Sales ... OK
002360 : Calculate EV/Sales ... OK
009160 : Calculate EV/Sales ... OK
123700 : Not enough data to calculate ...
025530 : Calculate EV/Sales ... OK
034730 : Calculate EV/Sales ... OK
337450 : Not enough data to calculate ...
340350 : Not enough data to calculate ...
011790 : Calculate EV/Sales ... OK
018670 : Calculate EV/Sales ... OK
001740 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
001745 : Not enough data to calculate ...
006120 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
006125 : Not enough data to calculate ...
210980 : Calculate EV/Sales ... OK
068400 : Calculate EV/Sales ... OK
395400 : Calculate EV/Sales ... OK
302440 : Calculate EV/Sales ... OK
326030 : Not enough 

086220 : Not enough data to calculate ...
033640 : Calculate EV/Sales ... OK
330860 : Calculate EV/Sales ... OK
005720 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
005725 : Not enough data to calculate ...
002350 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
002355 : Not enough data to calculate ...
089140 : Calculate EV/Sales ... OK
065170 : Calculate EV/Sales ... OK
003580 : Calculate EV/Sales ... OK
137940 : Not enough data to calculate ...
348210 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
041140 : Not enough data to calculate ...
217270 : Calculate EV/Sales ... OK
225570 : Not enough data to calculate ...
251270 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
317860 : Not enough data to calculate ...
104620 : Calculate EV/Sales ... OK
090350 : Not enough data to calculate ...
Empty DB Incomes
Empty DB Balances
090355 : Not enough data to calculate ...
000320 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB B

041930 : Calculate EV/Sales ... OK
001520 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
001527 : Not enough data to calculate ...
Empty DB Incomes
Empty DB Balances
001529 : Not enough data to calculate ...
084670 : Calculate EV/Sales ... OK
082640 : Calculate EV/Sales ... OK
060380 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
001525 : Not enough data to calculate ...
079960 : Calculate EV/Sales ... OK
008970 : Calculate EV/Sales ... OK
228340 : Calculate EV/Sales ... OK
092780 : Not enough data to calculate ...
088910 : Calculate EV/Sales ... OK
094170 : Calculate EV/Sales ... OK
049770 : Calculate EV/Sales ... OK
013120 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
018500 : Not enough data to calculate ...
006040 : Calculate EV/Sales ... OK
030720 : Calculate EV/Sales ... OK
014820 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
014825 : Not enough data to calculate ...
163560 : Calculate EV/Sales ... OK
109860 : Calc

005360 : Calculate EV/Sales ... OK
080420 : Calculate EV/Sales ... OK
080160 : Calculate EV/Sales ... OK
204210 : Calculate EV/Sales ... OK
100030 : Calculate EV/Sales ... OK
087260 : Calculate EV/Sales ... OK
101330 : Calculate EV/Sales ... OK
012860 : Calculate EV/Sales ... OK
348030 : Calculate EV/Sales ... OK
250060 : Not enough data to calculate ...
288980 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
033200 : Not enough data to calculate ...
009680 : Calculate EV/Sales ... OK
118990 : Calculate EV/Sales ... OK
006920 : Calculate EV/Sales ... OK
009580 : Calculate EV/Sales ... OK
001810 : Calculate EV/Sales ... OK
009200 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
322970 : Not enough data to calculate ...
033920 : Calculate EV/Sales ... OK
008420 : Calculate EV/Sales ... OK
279600 : Calculate EV/Sales ... OK
095500 : Not enough data to calculate ...
025560 : Calculate EV/Sales ... OK
218150 : Calculate EV/Sales ... OK
007120 : Calculate EV/Sales

...

033230 : Calculate EV/Sales ... OK
006490 : Calculate EV/Sales ... OK
037330 : Calculate EV/Sales ... OK
023800 : Calculate EV/Sales ... OK
034590 : Calculate EV/Sales ... OK
211050 : Calculate EV/Sales ... OK
083640 : Calculate EV/Sales ... OK
216050 : Calculate EV/Sales ... OK
049070 : Calculate EV/Sales ... OK
119610 : Calculate EV/Sales ... OK
017250 : Calculate EV/Sales ... OK
129260 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
240340 : Not enough data to calculate ...
035080 : Calculate EV/Sales ... OK
051370 : Calculate EV/Sales ... OK
064290 : Calculate EV/Sales ... OK
189300 : Calculate EV/Sales ... OK
150840 : Calculate EV/Sales ... OK
048530 : Calculate EV/Sales ... OK
023810 : Calculate EV/Sales ... OK
175140 : Calculate EV/Sales ... OK
115310 : Calculate EV/Sales ... OK
039290 : Calculate EV/Sales ... OK
071200 : Calculate EV/Sales ... OK
016670 : Calculate EV/Sales ... OK
101930 : Calculate EV/Sales ... OK
249420 : Calculate EV/Sales ... OK
000230 : Calc

025880 : Calculate EV/Sales ... OK
093320 : Calculate EV/Sales ... OK
073010 : Calculate EV/Sales ... OK
105330 : Calculate EV/Sales ... OK
272110 : Calculate EV/Sales ... OK
039420 : Calculate EV/Sales ... OK
083550 : Calculate EV/Sales ... OK
032500 : Calculate EV/Sales ... OK
225430 : Calculate EV/Sales ... OK
102370 : Calculate EV/Sales ... OK
381970 : Calculate EV/Sales ... OK
145270 : Calculate EV/Sales ... OK
298870 : Calculate EV/Sales ... OK
036030 : Calculate EV/Sales ... OK
064820 : Calculate EV/Sales ... OK
347140 : Not enough data to calculate ...
256940 : Calculate EV/Sales ... OK
024880 : Calculate EV/Sales ... OK
042040 : Calculate EV/Sales ... OK
054410 : Calculate EV/Sales ... OK
274090 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
402420 : Not enough data to calculate ...
217600 : Calculate EV/Sales ... OK
089010 : Calculate EV/Sales ... OK
220260 : Calculate EV/Sales ... OK
052400 : Calculate EV/Sales ... OK
046070 : Calculate EV/Sales ... OK
047770

037070 : Calculate EV/Sales ... OK
150900 : Calculate EV/Sales ... OK
037030 : Calculate EV/Sales ... OK
047310 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
266870 : Not enough data to calculate ...
368770 : Calculate EV/Sales ... OK
170790 : Calculate EV/Sales ... OK
049120 : Calculate EV/Sales ... OK
038950 : Calculate EV/Sales ... OK
106240 : Calculate EV/Sales ... OK
131760 : Not enough data to calculate ...
065690 : Calculate EV/Sales ... OK
140860 : Calculate EV/Sales ... OK
091700 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
202960 : Not enough data to calculate ...
032800 : Calculate EV/Sales ... OK
318010 : Calculate EV/Sales ... OK
036580 : Calculate EV/Sales ... OK
027710 : Calculate EV/Sales ... OK
004720 : Calculate EV/Sales ... OK
225590 : Calculate EV/Sales ... OK
054300 : Calculate EV/Sales ... OK
068050 : Calculate EV/Sales ... OK
028670 : Calculate EV/Sales ... OK
222110 : Calculate EV/Sales ... OK
010820 : Calculate EV/Sales ... OK

070590 : Calculate EV/Sales ... OK
213500 : Calculate EV/Sales ... OK
014680 : Calculate EV/Sales ... OK
004710 : Calculate EV/Sales ... OK
004150 : Calculate EV/Sales ... OK
025750 : Calculate EV/Sales ... OK
226440 : Calculate EV/Sales ... OK
042520 : Calculate EV/Sales ... OK
004960 : Calculate EV/Sales ... OK
011700 : Calculate EV/Sales ... OK
078350 : Calculate EV/Sales ... OK
045100 : Calculate EV/Sales ... OK
001750 : Calculate EV/Sales ... OK
Empty DB Incomes
Empty DB Balances
001755 : Not enough data to calculate ...
018880 : Calculate EV/Sales ... OK
009420 : Calculate EV/Sales ... OK
014130 : Calculate EV/Sales ... OK
046110 : Calculate EV/Sales ... OK
024740 : Calculate EV/Sales ... OK
005860 : Calculate EV/Sales ... OK
300720 : Calculate EV/Sales ... OK
123840 : Calculate EV/Sales ... OK
002220 : Calculate EV/Sales ... OK
006390 : Calculate EV/Sales ... OK
003300 : Calculate EV/Sales ... OK
007770 : Calculate EV/Sales ... OK
051600 : Calculate EV/Sales ... OK
052690 : Calc

,code,name,exchange,period,mktcap,EV/Sales
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,1.242540
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,1.703735
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,11.365733
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.416820
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,2.971939
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,5.871036
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.630853


In [155]:
# EV/Sales 종목 선정
df_factor = ES_list.copy()
factor_list = ['EV/Sales']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # EV/Sales, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # EV/Sales, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # EV/Sales, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,EV/Sales,score
1,049070,인탑스,0.159845,100.000000
2,030520,한글과컴퓨터,0.205659,99.999566
3,205470,휴마시스,0.211803,99.999508
4,095660,네오위즈,0.309096,99.998586
5,060250,NHN한국사이버결제,0.313601,99.998544
6,035600,KG이니시스,0.526223,99.996530
7,056190,에스에프에이,0.567523,99.996139
8,091700,파트론,0.586511,99.995959
9,090460,비에이치,0.831237,99.993641
10,095610,테스,0.858655,99.993381


#### NCAV

In [16]:
# NCAV 구하기
NCAV_list = v.getNCAV()
NCAV_list

060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate NCAV ... OK
006840 : Calculate NCAV ... OK
054620 : Calculate NCAV ... OK
265520 : Calculate NCAV ... OK
211270 : Calculate NCAV ... OK
027410 : Calculate NCAV ... OK
282330 : Calculate NCAV ... OK
032790 : Calculate NCAV ... OK
138930 : Calculate NCAV ... OK
001460 : Calculate NCAV ... OK
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate NCAV ... OK
001040 : Calculate NCAV ... OK
079160 : Calculate NCAV ... OK
035760 : Calculate NCAV ... OK
311690 : Calculate NCAV ... OK
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate NCAV ... OK
011150 : Calculate NCAV ... OK
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Balances
001045 : Not enough data to calculate ...
097950 : Calculate NCAV ... OK
097955 : Empty Balances
097955 : Not enough data to calculate ...
051500 : Calculate NCAV ... OK
058820 : Calcul

361610 : Calculate NCAV ... OK
03473K : Empty Balances
03473K : Not enough data to calculate ...
096770 : Calculate NCAV ... OK
096775 : Empty Balances
096775 : Not enough data to calculate ...
001510 : Calculate NCAV ... OK
001515 : Empty Balances
001515 : Not enough data to calculate ...
285130 : Calculate NCAV ... OK
28513K : Empty Balances
28513K : Not enough data to calculate ...
017670 : Calculate NCAV ... OK
000660 : Calculate NCAV ... OK
048550 : Calculate NCAV ... OK
063440 : Calculate NCAV ... OK
950180 : Empty Balances
950180 : Not enough data to calculate ...
064960 : Calculate NCAV ... OK
100840 : Calculate NCAV ... OK
003570 : Calculate NCAV ... OK
036530 : Calculate NCAV ... OK
005610 : Calculate NCAV ... OK
011810 : Calculate NCAV ... OK
077970 : Calculate NCAV ... OK
071970 : Calculate NCAV ... OK
002820 : Calculate NCAV ... OK
289080 : Empty Balances
289080 : Not enough data to calculate ...
084870 : Not enough data to calculate ...
002710 : Calculate NCAV ... OK
0892

340360 : Calculate NCAV ... OK
039560 : Calculate NCAV ... OK
058730 : Calculate NCAV ... OK
023590 : Calculate NCAV ... OK
032190 : Calculate NCAV ... OK
323350 : Empty Balances
323350 : Not enough data to calculate ...
068240 : Calculate NCAV ... OK
145210 : Calculate NCAV ... OK
271850 : Empty Balances
271850 : Not enough data to calculate ...
343090 : Empty Balances
343090 : Not enough data to calculate ...
019680 : Calculate NCAV ... OK
019685 : Empty Balances
019685 : Not enough data to calculate ...
006370 : Calculate NCAV ... OK
008060 : Calculate NCAV ... OK
00806K : Empty Balances
00806K : Not enough data to calculate ...
353200 : Calculate NCAV ... OK
35320K : Empty Balances
35320K : Not enough data to calculate ...
000490 : Calculate NCAV ... OK
178600 : Empty Balances
178600 : Not enough data to calculate ...
020400 : Calculate NCAV ... OK
008830 : Calculate NCAV ... OK
048470 : Calculate NCAV ... OK
008110 : Empty Balances
008110 : Not enough data to calculate ...
004780 

263020 : Calculate NCAV ... OK
290550 : Calculate NCAV ... OK
007340 : Calculate NCAV ... OK
187220 : Calculate NCAV ... OK
163430 : Empty Balances
163430 : Not enough data to calculate ...
131180 : Not enough data to calculate ...
315640 : Calculate NCAV ... OK
317120 : Calculate NCAV ... OK
042510 : Calculate NCAV ... OK
232680 : Calculate NCAV ... OK
300120 : Calculate NCAV ... OK
171120 : Calculate NCAV ... OK
069540 : Calculate NCAV ... OK
285770 : Empty Balances
285770 : Not enough data to calculate ...
347700 : Calculate NCAV ... OK
214260 : Calculate NCAV ... OK
115390 : Not enough data to calculate ...
200350 : Calculate NCAV ... OK
171010 : Calculate NCAV ... OK
084650 : Calculate NCAV ... OK
217500 : Calculate NCAV ... OK
092590 : Empty Balances
092590 : Not enough data to calculate ...
141080 : Calculate NCAV ... OK
060300 : Calculate NCAV ... OK
038390 : Calculate NCAV ... OK
294140 : Calculate NCAV ... OK
228670 : Calculate NCAV ... OK
228850 : Calculate NCAV ... OK
28174

005030 : Calculate NCAV ... OK
008470 : Calculate NCAV ... OK
338220 : Calculate NCAV ... OK
100120 : Calculate NCAV ... OK
337930 : Calculate NCAV ... OK
099390 : Calculate NCAV ... OK
064480 : Calculate NCAV ... OK
288330 : Calculate NCAV ... OK
365900 : Calculate NCAV ... OK
251630 : Calculate NCAV ... OK
018290 : Calculate NCAV ... OK
044480 : Calculate NCAV ... OK
033560 : Calculate NCAV ... OK
191600 : Empty Balances
191600 : Not enough data to calculate ...
369370 : Not enough data to calculate ...
126340 : Calculate NCAV ... OK
121800 : Calculate NCAV ... OK
148140 : Calculate NCAV ... OK
082800 : Calculate NCAV ... OK
318410 : Calculate NCAV ... OK
002070 : Calculate NCAV ... OK
100220 : Calculate NCAV ... OK
146320 : Calculate NCAV ... OK
200780 : Calculate NCAV ... OK
141000 : Calculate NCAV ... OK
090460 : Calculate NCAV ... OK
083650 : Calculate NCAV ... OK
215050 : Empty Balances
215050 : Not enough data to calculate ...
086670 : Calculate NCAV ... OK
335890 : Calculate N

...

216050 : Calculate NCAV ... OK
049070 : Calculate NCAV ... OK
119610 : Calculate NCAV ... OK
017250 : Calculate NCAV ... OK
129260 : Calculate NCAV ... OK
240340 : Empty Balances
240340 : Not enough data to calculate ...
035080 : Calculate NCAV ... OK
051370 : Calculate NCAV ... OK
064290 : Calculate NCAV ... OK
189300 : Calculate NCAV ... OK
150840 : Calculate NCAV ... OK
048530 : Calculate NCAV ... OK
023810 : Calculate NCAV ... OK
175140 : Calculate NCAV ... OK
115310 : Calculate NCAV ... OK
039290 : Calculate NCAV ... OK
071200 : Calculate NCAV ... OK
016670 : Calculate NCAV ... OK
101930 : Calculate NCAV ... OK
249420 : Calculate NCAV ... OK
000230 : Calculate NCAV ... OK
013360 : Calculate NCAV ... OK
003120 : Calculate NCAV ... OK
333430 : Calculate NCAV ... OK
068330 : Not enough data to calculate ...
003200 : Calculate NCAV ... OK
007110 : Calculate NCAV ... OK
058450 : Calculate NCAV ... OK
007570 : Calculate NCAV ... OK
007575 : Empty Balances
007575 : Not enough data to cal

104540 : Calculate NCAV ... OK
027050 : Calculate NCAV ... OK
290510 : Calculate NCAV ... OK
007815 : Empty Balances
007815 : Not enough data to calculate ...
007810 : Calculate NCAV ... OK
00781K : Empty Balances
00781K : Not enough data to calculate ...
190650 : Empty Balances
190650 : Not enough data to calculate ...
101670 : Not enough data to calculate ...
123410 : Calculate NCAV ... OK
003690 : Calculate NCAV ... OK
036690 : Calculate NCAV ... OK
049430 : Calculate NCAV ... OK
183300 : Calculate NCAV ... OK
041960 : Calculate NCAV ... OK
089890 : Calculate NCAV ... OK
009730 : Calculate NCAV ... OK
189350 : Empty Balances
189350 : Not enough data to calculate ...
082660 : Calculate NCAV ... OK
192820 : Calculate NCAV ... OK
044820 : Calculate NCAV ... OK
222040 : Calculate NCAV ... OK
241710 : Calculate NCAV ... OK
005070 : Calculate NCAV ... OK
005420 : Calculate NCAV ... OK
069110 : Calculate NCAV ... OK
169670 : Empty Balances
169670 : Not enough data to calculate ...
071950 :

950210 : Calculate NCAV ... OK
203690 : Calculate NCAV ... OK
321260 : Not enough data to calculate ...
303360 : Empty Balances
303360 : Not enough data to calculate ...
053610 : Calculate NCAV ... OK
377220 : Calculate NCAV ... OK
335810 : Calculate NCAV ... OK
053160 : Calculate NCAV ... OK
367000 : Calculate NCAV ... OK
075130 : Calculate NCAV ... OK
222670 : Empty Balances
222670 : Not enough data to calculate ...
009810 : Calculate NCAV ... OK
237820 : Calculate NCAV ... OK
023770 : Calculate NCAV ... OK
300080 : Calculate NCAV ... OK
150440 : Empty Balances
150440 : Not enough data to calculate ...
032580 : Calculate NCAV ... OK
051380 : Calculate NCAV ... OK
241820 : Calculate NCAV ... OK
237750 : Calculate NCAV ... OK
024850 : Empty Balances
024850 : Not enough data to calculate ...
319660 : Calculate NCAV ... OK
031980 : Calculate NCAV ... OK
002230 : Calculate NCAV ... OK
057880 : Calculate NCAV ... OK
043370 : Calculate NCAV ... OK
242350 : Empty Balances
242350 : Not enough

322000 : Calculate NCAV ... OK
041440 : Calculate NCAV ... OK
039010 : Calculate NCAV ... OK
017800 : Calculate NCAV ... OK
307950 : Calculate NCAV ... OK
011210 : Calculate NCAV ... OK
090850 : Calculate NCAV ... OK
267260 : Calculate NCAV ... OK
004020 : Calculate NCAV ... OK
329180 : Calculate NCAV ... OK
267250 : Calculate NCAV ... OK
005380 : Calculate NCAV ... OK
005387 : Empty Balances
005387 : Not enough data to calculate ...
005389 : Empty Balances
005389 : Not enough data to calculate ...
005385 : Empty Balances
005385 : Not enough data to calculate ...
001500 : Calculate NCAV ... OK
011760 : Calculate NCAV ... OK
227840 : Calculate NCAV ... OK
126560 : Calculate NCAV ... OK
001450 : Calculate NCAV ... OK
057050 : Calculate NCAV ... OK
092300 : Not enough data to calculate ...
053660 : Empty Balances
053660 : Not enough data to calculate ...
138360 : Calculate NCAV ... OK
011080 : Calculate NCAV ... OK
093240 : Calculate NCAV ... OK
003010 : Calculate NCAV ... OK
111110 : Cal

,code,name,exchange,period,mktcap,NCAV
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,-3.334038e+12
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,-8.340486e+12
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,-6.059542e+11
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,-3.852046e+11
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,-2.754479e+11
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,-2.138500e+11
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,-1.337159e+13
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,-1.417359e+12
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,-1.198883e+11


In [127]:
# NCAV 종목 선정
df_factor = NCAV_list.copy()

factor_list = ['NCAV']
result = stock_select(df_factor, 0.3, 'ALL', 30, factor_list) # NCAV, 시가총액 상위 30%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.3, 'KOSPI', 30, factor_list) # NCAV, 시가총액 상위 30%, KOSPI 종목 중 30개 선택
result

,code,name,NCAV,score
1,006800,미래에셋증권,7.244344e+13,100.000000
2,016360,삼성증권,6.727760e+13,92.863020
3,039490,키움증권,6.033157e+13,83.266587
4,005940,NH투자증권,3.210410e+13,44.268325
5,003540,대신증권,2.444576e+13,33.687777
6,003470,유안타증권,1.994691e+13,27.472293
7,008560,메리츠증권,1.148073e+13,15.775665
8,000720,현대건설,9.469998e+12,12.997692
9,030610,교보증권,5.032844e+12,6.867445
10,001510,SK증권,4.203789e+12,5.722047


# 5. 우량주 종목 추출

## 우량주 지표 계산 클래스

In [95]:
class BLUECHIP:
    period = None # 분기 할당 (str)
    stocks = None # 수집된 종목 할당 (DataFrame)
    ohlcv = None # 분기별 수정주가 할당 (DataFrame)
    fin_unit = 0 # 재무제표 단위 설정
    
    def __init__(self, term):
        print("init 실행 중", datetime.now())
        
        self.stocks = STOCK().getAllStocks() # 수집된 종목 가져오기
        self.period = termToPeriod(term) # 분기 설정
        self.ohlcv = PRICE().getTermOHLCV(term) # 해당 분기 수정주가 가져오기
        self.ohlcv = self.ohlcv.set_index('code') # 종목 코드를 인덱스로 사용
        self.fin_unit = 100000000 # 재무제표 단위 : 억원
        
        print("init 실행 완료", datetime.now())
    
    # 투자 효율이 좋은 기업 (자산 대비 이익)
    def getROA(self):
        # ROA 계산 후 저장할 DataFrame 생성
        df_ROA = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서, 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
            
            # 2. 자산 평균 구하기 : (기초총자산 + 기말총자산) / 2
            # 트레일링 데이터 부족하므로 현재 자산만으로 계산 (balances > asset)
            AVG_ASSET = float(bs_trailing['asset'] * self.fin_unit)
            
            # 3. ROA = 당기순이익/자산평균 계산
            # 당기순이익: incomes > netinc
            if AVG_ASSET <= 0:
                ROA = np.NaN
            else:
                ROA = float((is_trailing['netinc'] * self.fin_unit) / AVG_ASSET )
                
            # 4. 계산한 결과 DataFrame에 추가
            df_ROA = df_ROA.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'ROA': ROA }, ignore_index=True)

            print(code, ": Calculate ROA ... OK")
            
        return df_ROA
    
    # 투자 효율이 좋은 기업 (자본 대비 이익)
    def getROE(self):
        # ROE 계산 후 저장할 DataFrame 생성
        df_ROE = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서, 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
            
            # 2. 자본 평균 구하기 : (기초총자본 + 기말총자본) / 2
            # 트레일링 데이터 부족하므로 현재 자본만으로 계산 (balances > equity)
            AVG_EQUITY = float(bs_trailing['equity'] * self.fin_unit)
            
            # 3. ROE = 당기순이익/자본평균 계산
            # 당기순이익: incomes > netinc
            if AVG_EQUITY <= 0:
                ROE = np.NaN
            else:
                ROE = float((is_trailing['netinc'] * self.fin_unit) / AVG_EQUITY )
                
            # 4. 계산한 결과 DataFrame에 추가
            df_ROE = df_ROE.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'ROE': ROE }, ignore_index=True)

            print(code, ": Calculate ROE ... OK")
            
        return df_ROE
        
    # 회계사가 유행시킨 장기 투자 전략 (잔여 이익 모델)
    def getRIM(self, rate): # rate: 목표수익률 입력받기
        # RIM 계산 후 저장할 DataFrame 생성
        df_RIM = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서, 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
            
            # 2. BPS = 자본/상장주식수 계산하기
            # 자본: balances > equity
            # 상장주식수: stocks > shares
            BPS = float((bs_trailing['equity'] * self.fin_unit) / self.ohlcv.loc[code]['shares'])
            
            # 3. ROE = 당기순이익/자본 계산하기
            # 당기순이익: incomes > netinc
            # 자본: balances > equity
            ROE = float((is_trailing['netinc'] * self.fin_unit) / (bs_trailing['equity'] * self.fin_unit))
            
            # 4. RIM = BPS * (ROE / 목표수익률) 계산하기
            RIM = float(BPS * (ROE / rate))
            
            # 5. P/RIM = 주가 / RIM 계산하기
            if self.ohlcv.loc[code]['shares'] <= 0 or float(bs_trailing['equity']) <= 0 or RIM <= 0:
                PRIM = np.NaN
            else:
                PRIM = float(self.ohlcv.loc[code]['Close'] / RIM)
                
            # 4. 계산한 결과 DataFrame에 추가
            df_RIM = df_RIM.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'RIM': PRIM }, ignore_index=True)

            print(code, ": Calculate RIM ... OK")
            
        return df_RIM
    
    # 영업 효율이 좋은 기업 (GP/A)
    def getGPA(self):
        # GP/A 계산 후 저장할 DataFrame 생성
        df_GPA = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서, 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무제표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
            
            # 2. GP/A = 매출총이익 / 자산평균 계산하기
            # 매출총이익: incomes > gross
            # 자산 평균 = (기초자산 + 기말자산)/2 but 트레일링 데이터 1년치만 존재하므로, 총자산 이용
            if float(bs_trailing['asset'] <= 0):
                GPA = np.NaN
            else:
                GPA = float((is_trailing['gross'] * self.fin_unit) / (bs_trailing['asset'] * self.fin_unit))
                
            # 4. 계산한 결과 DataFrame에 추가
            df_GPA = df_GPA.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'GP/A': GPA }, ignore_index=True)

            print(code, ": Calculate GP/A ... OK")
            
        return df_GPA
        
    # 안정성 지표
    def getStability(self):
        # Stability 계산 후 저장할 DataFrame 생성
        df_ST = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 재무상태표 가져오기
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 재무상태표 trailing 데이터 만들지 못하는 경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Data Not Enough!")
                continue # 해당 종목은 pass
        
            # 2. 부채비율 = (총부채/총자본) 계산
            # 부채비율 Liability/Equity
            # 총부채: balances > liab
            # 총자본: balances > equity
            if float(bs_trailing['equity']) == 0: # 0으로 나누는 경우
                LE = np.NaN
            else:
                LE = float((bs_trailing['liab']*self.fin_unit) / (bs_trailing['equity']*self.fin_unit))

            # 3. 차입금비율 = (장기차입금+단기차입금)/총자본 계산
            # 차입금비율 Debt/Equity
            # 장기/단기차입금: balances > longdebt, curdebt
            # 총자본: balances > equity
            if float(bs_trailing['equity']) == 0: # 0으로 나누는 경우
                DE = np.NaN
            else:
                DE = float(((bs_trailing['longdebt']*self.fin_unit) + (bs_trailing['curdebt']*self.fin_unit)) / (bs_trailing['equity']*self.fin_unit))
            
            # 4. 계산한 결과 DataFrame에 추가
            df_ST = df_ST.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'L/E': LE, # 부채비율
                'D/E': DE }, ignore_index=True) # 차입금비율
            
            print(code, ": Calculate Stability ... OK")
            
        return df_ST
    
    # 쑥쑥 자라는 기업 (성장률 지표)
    def getGrowthRate(self):
        print("growth rate")
        
    # 부지런한 기업 (회전율 지표)
    def getTurnover(self):
        # Turnover 계산 후 저장할 DataFrame 생성
        df_TO = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서, 재무상태표 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            bs_trailing = BALANCES().getTrailingBalances(code, self.period)
            
            # 손익계산서, 재무상태표 trailing 데이터 만들지 못하는  경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or bs_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
                
            # 2. 총자산 회전율 계산
            # 평균 총자산 = (총자산 + 4분기 전 총자산) / 2
            # 총자산: balances > asset
            # 4분기 전 트레일링 데이터 존재하지 않으므로 현재 총자산만 계산에 이용
            AVG_Asset = float(bs_trailing['asset'] * self.fin_unit)
            
            # ASS_TO = 매출액(incomes > rev) / AVG_Asset
            if AVG_Asset <= 0:
                ASS_TO = np.NaN
            else:
                ASS_TO = float((is_trailing['rev'] * self.fin_unit) / AVG_Asset)
        
            # 3. 매출채권 회전율 계산
            # 평균 매출채권 = (매출채권 + 4분기 전 매출채권) / 2
            # 매출채권: balances > rec
            # 4분기 전 트레일링 데이터 존재하지 않으므로 현재 매출채권만 계산에 이용
            AVG_Rec = float(bs_trailing['rec'] * self.fin_unit)
            
            # REC_TO = 매출액(incomes > rev) / AVG_Receivables
            if AVG_Rec <= 0:
                REC_TO = np.NaN
            else:
                REC_TO = float((is_trailing['rev'] * self.fin_unit) / AVG_Rec)
            
            # 4. 재고자산 회전율 계산
            # 평균 재고자산 = (재고자산 + 4분기 전 재고자산) / 2
            # 재고자산: balances > inv
            # 4분기 전 트레일링 데이터 존재하지 않으므로 현재 재고자산만 계산에 이용
            AVG_Inv = float(bs_trailing['inv'] * self.fin_unit)
            
            # INV_TO = 매출원가(incomes cgs) / 평균 재고자산
            if AVG_Inv <= 0:
                INV_TO = np.NaN
            else:
                INV_TO = float((is_trailing['cgs'] * self.fin_unit) / AVG_Inv)
            
            # 3. 계산한 결과 DataFrame에 추가
            df_TO = df_TO.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'ASS turnover': ASS_TO,
                'REC turnover': REC_TO,
                'INV turnover': INV_TO }, ignore_index=True)
            
            print(code, ": Calculate Turnover ... OK")
            
        return df_TO
        
        
    # 해자가 있는 기업 (이익률 지표)
    def getGrossMargin(self):
        # Gross Margin 계산 후 저장할 DataFrame 생성
        df_GM = pd.DataFrame()
        
        # 모든 종목 코드에 대해
        for code, name, exc in zip(self.stocks['code'], self.stocks['name'], self.stocks['exchange']):
            # 1. 분기별 손익계산서 가져오기
            is_trailing = INCOMES().getTrailingIncomes(code, self.period)
            
            # 손익계산서 trailing 데이터 만들지 못하는  경우
            #   또는 ohlcv 데이터가 존재하지 않는 종목인 경우
            if is_trailing is None or code not in self.ohlcv.index:
                print(code, ": Not enough data to calculate ...")
                continue # 해당 종목은 pass
        
            # 2. 매출총이익률(gross margin) = (매출총이익/매출액)*100 계산
            # 매출총이익: incomes > gross
            # 매출액: incomes > rev
            if float(is_trailing['gross']) <= 0 or float(is_trailing['rev']) <= 0: # 매출총이익 음수 or 매출액 음수인 경우
                GM = np.NaN
            else:
                GM = float((is_trailing['gross']*self.fin_unit) / (is_trailing['rev']*self.fin_unit) * 100) 
            
            # 3. 계산한 결과 DataFrame에 추가
            df_GM = df_GM.append({
                'code':code,
                'name':name,
                'exchange': exc,
                'period': self.period,
                'mktcap': self.ohlcv.loc[code]['mktcap'],
                'GrossMargin': GM }, ignore_index=True)
            
            print(code, ": Calculate Gross Margin ... OK")
            
        return df_GM

### BLUECHIP 클래스 사용 예시

In [96]:
b = BLUECHIP('2022Q1')

init 실행 중 2022-06-16 23:51:33.518083
init 실행 완료 2022-06-16 23:57:32.248169


#### ROA

In [52]:
# ROA 구하기
ROA_list = b.getROA()
ROA_list

060310 : Empty Incomes
060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate ROA ... OK
006840 : Calculate ROA ... OK
054620 : Calculate ROA ... OK
265520 : Calculate ROA ... OK
211270 : Calculate ROA ... OK
027410 : Calculate ROA ... OK
282330 : Calculate ROA ... OK
032790 : Calculate ROA ... OK
138930 : Calculate ROA ... OK
001460 : Calculate ROA ... OK
001465 : Empty Incomes
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate ROA ... OK
001040 : Calculate ROA ... OK
079160 : Calculate ROA ... OK
035760 : Calculate ROA ... OK
311690 : Calculate ROA ... OK
00104K : Empty Incomes
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate ROA ... OK
011150 : Calculate ROA ... OK
011155 : Empty Incomes
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Incomes
001045 : Empty Balances
001045 : Not enough data to calculate ...
097950 : Calculate ROA ... OK
097955 : Empty 

011790 : Calculate ROA ... OK
018670 : Calculate ROA ... OK
001740 : Calculate ROA ... OK
001745 : Empty Incomes
001745 : Empty Balances
001745 : Not enough data to calculate ...
006120 : Calculate ROA ... OK
006125 : Empty Incomes
006125 : Empty Balances
006125 : Not enough data to calculate ...
210980 : Calculate ROA ... OK
068400 : Calculate ROA ... OK
395400 : Calculate ROA ... OK
302440 : Calculate ROA ... OK
326030 : Not enough data to calculate ...
402340 : Calculate ROA ... OK
361610 : Calculate ROA ... OK
03473K : Empty Incomes
03473K : Empty Balances
03473K : Not enough data to calculate ...
096770 : Calculate ROA ... OK
096775 : Empty Incomes
096775 : Empty Balances
096775 : Not enough data to calculate ...
001510 : Calculate ROA ... OK
001515 : Empty Incomes
001515 : Empty Balances
001515 : Not enough data to calculate ...
285130 : Calculate ROA ... OK
28513K : Empty Incomes
28513K : Empty Balances
28513K : Not enough data to calculate ...
017670 : Calculate ROA ... OK
0006

194700 : Calculate ROA ... OK
285490 : Calculate ROA ... OK
229500 : Empty Incomes
229500 : Empty Balances
229500 : Not enough data to calculate ...
106520 : Calculate ROA ... OK
376930 : Calculate ROA ... OK
278650 : Calculate ROA ... OK
006280 : Calculate ROA ... OK
142280 : Calculate ROA ... OK
234690 : Calculate ROA ... OK
005250 : Calculate ROA ... OK
005257 : Empty Incomes
005257 : Empty Balances
005257 : Not enough data to calculate ...
065560 : Calculate ROA ... OK
004370 : Calculate ROA ... OK
072710 : Calculate ROA ... OK
054050 : Calculate ROA ... OK
069140 : Calculate ROA ... OK
040160 : Calculate ROA ... OK
332290 : Calculate ROA ... OK
126870 : Calculate ROA ... OK
060260 : Calculate ROA ... OK
012340 : Calculate ROA ... OK
214870 : Calculate ROA ... OK
270870 : Calculate ROA ... OK
144960 : Calculate ROA ... OK
085670 : Calculate ROA ... OK
119860 : Calculate ROA ... OK
064260 : Calculate ROA ... OK
093640 : Calculate ROA ... OK
340360 : Calculate ROA ... OK
039560 : Cal

034020 : Calculate ROA ... OK
336260 : Calculate ROA ... OK
33626K : Empty Incomes
33626K : Empty Balances
33626K : Not enough data to calculate ...
33626L : Empty Incomes
33626L : Empty Balances
33626L : Not enough data to calculate ...
016740 : Calculate ROA ... OK
073190 : Calculate ROA ... OK
176750 : Empty Incomes
176750 : Empty Balances
176750 : Not enough data to calculate ...
030350 : Calculate ROA ... OK
203650 : Calculate ROA ... OK
223250 : Calculate ROA ... OK
060570 : Calculate ROA ... OK
192650 : Calculate ROA ... OK
217620 : Calculate ROA ... OK
187870 : Calculate ROA ... OK
066670 : Calculate ROA ... OK
024090 : Calculate ROA ... OK
003160 : Calculate ROA ... OK
092200 : Calculate ROA ... OK
110990 : Calculate ROA ... OK
078590 : Calculate ROA ... OK
263690 : Calculate ROA ... OK
214680 : Calculate ROA ... OK
377190 : Calculate ROA ... OK
263720 : Calculate ROA ... OK
376300 : Calculate ROA ... OK
109740 : Calculate ROA ... OK
131030 : Calculate ROA ... OK
196490 : Calc

214610 : Calculate ROA ... OK
201490 : Calculate ROA ... OK
950190 : Calculate ROA ... OK
214180 : Calculate ROA ... OK
155900 : Not enough data to calculate ...
206640 : Calculate ROA ... OK
018700 : Calculate ROA ... OK
035620 : Calculate ROA ... OK
064520 : Calculate ROA ... OK
053030 : Calculate ROA ... OK
301300 : Calculate ROA ... OK
064550 : Calculate ROA ... OK
314930 : Calculate ROA ... OK
208710 : Calculate ROA ... OK
142760 : Calculate ROA ... OK
086820 : Calculate ROA ... OK
038460 : Calculate ROA ... OK
281310 : Empty Incomes
281310 : Empty Balances
281310 : Not enough data to calculate ...
251120 : Calculate ROA ... OK
266470 : Empty Incomes
266470 : Empty Balances
266470 : Not enough data to calculate ...
086040 : Calculate ROA ... OK
199290 : Empty Incomes
199290 : Empty Balances
199290 : Not enough data to calculate ...
099430 : Calculate ROA ... OK
032980 : Calculate ROA ... OK
222160 : Empty Incomes
222160 : Empty Balances
222160 : Not enough data to calculate ...
30

...

191410 : Calculate ROA ... OK
008730 : Calculate ROA ... OK
072770 : Calculate ROA ... OK
008250 : Calculate ROA ... OK
039020 : Calculate ROA ... OK
025820 : Calculate ROA ... OK
067920 : Calculate ROA ... OK
044960 : Calculate ROA ... OK
303530 : Calculate ROA ... OK
302430 : Calculate ROA ... OK
279060 : Empty Incomes
279060 : Empty Balances
279060 : Not enough data to calculate ...
214320 : Calculate ROA ... OK
073490 : Calculate ROA ... OK
215790 : Not enough data to calculate ...
344860 : Empty Incomes
344860 : Empty Balances
344860 : Not enough data to calculate ...
246960 : Calculate ROA ... OK
088390 : Calculate ROA ... OK
272290 : Calculate ROA ... OK
053350 : Calculate ROA ... OK
258610 : Calculate ROA ... OK
041520 : Calculate ROA ... OK
264850 : Calculate ROA ... OK
054210 : Calculate ROA ... OK
164060 : Calculate ROA ... OK
065440 : Calculate ROA ... OK
088260 : Not enough data to calculate ...
139480 : Calculate ROA ... OK
115610 : Calculate ROA ... OK
349720 : Not enoug

311320 : Calculate ROA ... OK
010580 : Calculate ROA ... OK
035000 : Calculate ROA ... OK
219750 : Calculate ROA ... OK
088790 : Calculate ROA ... OK
018120 : Calculate ROA ... OK
109820 : Calculate ROA ... OK
086060 : Calculate ROA ... OK
036890 : Calculate ROA ... OK
363250 : Calculate ROA ... OK
003780 : Calculate ROA ... OK
007370 : Calculate ROA ... OK
010640 : Calculate ROA ... OK
100250 : Calculate ROA ... OK
051630 : Calculate ROA ... OK
272450 : Calculate ROA ... OK
011000 : Calculate ROA ... OK
250030 : Calculate ROA ... OK
002780 : Calculate ROA ... OK
002787 : Empty Incomes
002787 : Empty Balances
002787 : Not enough data to calculate ...
002785 : Empty Incomes
002785 : Empty Balances
002785 : Not enough data to calculate ...
233990 : Empty Incomes
233990 : Empty Balances
233990 : Not enough data to calculate ...
085660 : Calculate ROA ... OK
261780 : Calculate ROA ... OK
009310 : Calculate ROA ... OK
158310 : Calculate ROA ... OK
094850 : Calculate ROA ... OK
004650 : Calc

054450 : Calculate ROA ... OK
091440 : Calculate ROA ... OK
078000 : Calculate ROA ... OK
200230 : Calculate ROA ... OK
214420 : Calculate ROA ... OK
393210 : Empty Incomes
393210 : Empty Balances
393210 : Not enough data to calculate ...
215480 : Calculate ROA ... OK
051360 : Calculate ROA ... OK
045340 : Calculate ROA ... OK
108230 : Calculate ROA ... OK
079970 : Calculate ROA ... OK
199800 : Calculate ROA ... OK
105550 : Calculate ROA ... OK
290090 : Calculate ROA ... OK
026150 : Calculate ROA ... OK
322180 : Calculate ROA ... OK
117730 : Calculate ROA ... OK
033830 : Calculate ROA ... OK
057680 : Calculate ROA ... OK
064760 : Calculate ROA ... OK
246710 : Calculate ROA ... OK
340570 : Calculate ROA ... OK
277880 : Calculate ROA ... OK
131290 : Calculate ROA ... OK
019180 : Calculate ROA ... OK
356860 : Calculate ROA ... OK
062860 : Calculate ROA ... OK
363280 : Calculate ROA ... OK
36328K : Empty Incomes
36328K : Empty Balances
36328K : Not enough data to calculate ...
321550 : Cal

021650 : Calculate ROA ... OK
161390 : Calculate ROA ... OK
053590 : Calculate ROA ... OK
034830 : Calculate ROA ... OK
007280 : Calculate ROA ... OK
032300 : Calculate ROA ... OK
168490 : Empty Incomes
168490 : Empty Balances
168490 : Not enough data to calculate ...
037230 : Calculate ROA ... OK
010100 : Calculate ROA ... OK
047810 : Calculate ROA ... OK
123690 : Calculate ROA ... OK
003350 : Calculate ROA ... OK
030520 : Calculate ROA ... OK
052600 : Calculate ROA ... OK
011500 : Calculate ROA ... OK
002390 : Calculate ROA ... OK
256150 : Calculate ROA ... OK
014790 : Calculate ROA ... OK
092460 : Calculate ROA ... OK
060980 : Calculate ROA ... OK
053690 : Calculate ROA ... OK
042700 : Calculate ROA ... OK
008930 : Calculate ROA ... OK
128940 : Not enough data to calculate ...
047080 : Calculate ROA ... OK
009240 : Calculate ROA ... OK
020000 : Calculate ROA ... OK
003680 : Calculate ROA ... OK
105630 : Calculate ROA ... OK
069640 : Calculate ROA ... OK
016450 : Calculate ROA ... OK

,code,name,exchange,period,mktcap,ROA
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.015228
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,-0.011198
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.016370
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.032554
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,0.000000
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.011092
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,0.000000
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,0.016718
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.009988


In [128]:
# ROA 종목 선정
df_factor = ROA_list.copy()
factor_list = ['ROA']

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # ROA, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # ROA, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # ROA, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,ROA,score
1,205470,휴마시스,0.258004,100.000000
2,950130,엑세스바이오,0.166796,64.648605
3,287410,제이시스메디칼,0.098082,38.015787
4,096530,씨젠,0.095140,36.875304
5,112040,위메이드,0.087798,34.029896
6,108320,LX세미콘,0.075499,29.262726
7,348210,넥스틴,0.071924,27.877020
8,089970,에이피티씨,0.069239,26.836576
9,035900,JYP Ent.,0.061419,23.805523
10,036930,주성엔지니어링,0.057817,22.409466


#### ROE

In [59]:
# ROE 구하기
ROE_list = b.getROE()
ROE_list

060310 : Empty Incomes
060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate ROE ... OK
006840 : Calculate ROE ... OK
054620 : Calculate ROE ... OK
265520 : Calculate ROE ... OK
211270 : Calculate ROE ... OK
027410 : Calculate ROE ... OK
282330 : Calculate ROE ... OK
032790 : Calculate ROE ... OK
138930 : Calculate ROE ... OK
001460 : Calculate ROE ... OK
001465 : Empty Incomes
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate ROE ... OK
001040 : Calculate ROE ... OK
079160 : Calculate ROE ... OK
035760 : Calculate ROE ... OK
311690 : Calculate ROE ... OK
00104K : Empty Incomes
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate ROE ... OK
011150 : Calculate ROE ... OK
011155 : Empty Incomes
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Incomes
001045 : Empty Balances
001045 : Not enough data to calculate ...
097950 : Calculate ROE ... OK
097955 : Empty 

011790 : Calculate ROE ... OK
018670 : Calculate ROE ... OK
001740 : Calculate ROE ... OK
001745 : Empty Incomes
001745 : Empty Balances
001745 : Not enough data to calculate ...
006120 : Calculate ROE ... OK
006125 : Empty Incomes
006125 : Empty Balances
006125 : Not enough data to calculate ...
210980 : Calculate ROE ... OK
068400 : Calculate ROE ... OK
395400 : Calculate ROE ... OK
302440 : Calculate ROE ... OK
326030 : Not enough data to calculate ...
402340 : Calculate ROE ... OK
361610 : Calculate ROE ... OK
03473K : Empty Incomes
03473K : Empty Balances
03473K : Not enough data to calculate ...
096770 : Calculate ROE ... OK
096775 : Empty Incomes
096775 : Empty Balances
096775 : Not enough data to calculate ...
001510 : Calculate ROE ... OK
001515 : Empty Incomes
001515 : Empty Balances
001515 : Not enough data to calculate ...
285130 : Calculate ROE ... OK
28513K : Empty Incomes
28513K : Empty Balances
28513K : Not enough data to calculate ...
017670 : Calculate ROE ... OK
0006

194700 : Calculate ROE ... OK
285490 : Calculate ROE ... OK
229500 : Empty Incomes
229500 : Empty Balances
229500 : Not enough data to calculate ...
106520 : Calculate ROE ... OK
376930 : Calculate ROE ... OK
278650 : Calculate ROE ... OK
006280 : Calculate ROE ... OK
142280 : Calculate ROE ... OK
234690 : Calculate ROE ... OK
005250 : Calculate ROE ... OK
005257 : Empty Incomes
005257 : Empty Balances
005257 : Not enough data to calculate ...
065560 : Calculate ROE ... OK
004370 : Calculate ROE ... OK
072710 : Calculate ROE ... OK
054050 : Calculate ROE ... OK
069140 : Calculate ROE ... OK
040160 : Calculate ROE ... OK
332290 : Calculate ROE ... OK
126870 : Calculate ROE ... OK
060260 : Calculate ROE ... OK
012340 : Calculate ROE ... OK
214870 : Calculate ROE ... OK
270870 : Calculate ROE ... OK
144960 : Calculate ROE ... OK
085670 : Calculate ROE ... OK
119860 : Calculate ROE ... OK
064260 : Calculate ROE ... OK
093640 : Calculate ROE ... OK
340360 : Calculate ROE ... OK
039560 : Cal

034020 : Calculate ROE ... OK
336260 : Calculate ROE ... OK
33626K : Empty Incomes
33626K : Empty Balances
33626K : Not enough data to calculate ...
33626L : Empty Incomes
33626L : Empty Balances
33626L : Not enough data to calculate ...
016740 : Calculate ROE ... OK
073190 : Calculate ROE ... OK
176750 : Empty Incomes
176750 : Empty Balances
176750 : Not enough data to calculate ...
030350 : Calculate ROE ... OK
203650 : Calculate ROE ... OK
223250 : Calculate ROE ... OK
060570 : Calculate ROE ... OK
192650 : Calculate ROE ... OK
217620 : Calculate ROE ... OK
187870 : Calculate ROE ... OK
066670 : Calculate ROE ... OK
024090 : Calculate ROE ... OK
003160 : Calculate ROE ... OK
092200 : Calculate ROE ... OK
110990 : Calculate ROE ... OK
078590 : Calculate ROE ... OK
263690 : Calculate ROE ... OK
214680 : Calculate ROE ... OK
377190 : Calculate ROE ... OK
263720 : Calculate ROE ... OK
376300 : Calculate ROE ... OK
109740 : Calculate ROE ... OK
131030 : Calculate ROE ... OK
196490 : Calc

214610 : Calculate ROE ... OK
201490 : Calculate ROE ... OK
950190 : Calculate ROE ... OK
214180 : Calculate ROE ... OK
155900 : Not enough data to calculate ...
206640 : Calculate ROE ... OK
018700 : Calculate ROE ... OK
035620 : Calculate ROE ... OK
064520 : Calculate ROE ... OK
053030 : Calculate ROE ... OK
301300 : Calculate ROE ... OK
064550 : Calculate ROE ... OK
314930 : Calculate ROE ... OK
208710 : Calculate ROE ... OK
142760 : Calculate ROE ... OK
086820 : Calculate ROE ... OK
038460 : Calculate ROE ... OK
281310 : Empty Incomes
281310 : Empty Balances
281310 : Not enough data to calculate ...
251120 : Calculate ROE ... OK
266470 : Empty Incomes
266470 : Empty Balances
266470 : Not enough data to calculate ...
086040 : Calculate ROE ... OK
199290 : Empty Incomes
199290 : Empty Balances
199290 : Not enough data to calculate ...
099430 : Calculate ROE ... OK
032980 : Calculate ROE ... OK
222160 : Empty Incomes
222160 : Empty Balances
222160 : Not enough data to calculate ...
30

...

191410 : Calculate ROE ... OK
008730 : Calculate ROE ... OK
072770 : Calculate ROE ... OK
008250 : Calculate ROE ... OK
039020 : Calculate ROE ... OK
025820 : Calculate ROE ... OK
067920 : Calculate ROE ... OK
044960 : Calculate ROE ... OK
303530 : Calculate ROE ... OK
302430 : Calculate ROE ... OK
279060 : Empty Incomes
279060 : Empty Balances
279060 : Not enough data to calculate ...
214320 : Calculate ROE ... OK
073490 : Calculate ROE ... OK
215790 : Not enough data to calculate ...
344860 : Empty Incomes
344860 : Empty Balances
344860 : Not enough data to calculate ...
246960 : Calculate ROE ... OK
088390 : Calculate ROE ... OK
272290 : Calculate ROE ... OK
053350 : Calculate ROE ... OK
258610 : Calculate ROE ... OK
041520 : Calculate ROE ... OK
264850 : Calculate ROE ... OK
054210 : Calculate ROE ... OK
164060 : Calculate ROE ... OK
065440 : Calculate ROE ... OK
088260 : Not enough data to calculate ...
139480 : Calculate ROE ... OK
115610 : Calculate ROE ... OK
349720 : Not enoug

311320 : Calculate ROE ... OK
010580 : Calculate ROE ... OK
035000 : Calculate ROE ... OK
219750 : Calculate ROE ... OK
088790 : Calculate ROE ... OK
018120 : Calculate ROE ... OK
109820 : Calculate ROE ... OK
086060 : Calculate ROE ... OK
036890 : Calculate ROE ... OK
363250 : Calculate ROE ... OK
003780 : Calculate ROE ... OK
007370 : Calculate ROE ... OK
010640 : Calculate ROE ... OK
100250 : Calculate ROE ... OK
051630 : Calculate ROE ... OK
272450 : Calculate ROE ... OK
011000 : Calculate ROE ... OK
250030 : Calculate ROE ... OK
002780 : Calculate ROE ... OK
002787 : Empty Incomes
002787 : Empty Balances
002787 : Not enough data to calculate ...
002785 : Empty Incomes
002785 : Empty Balances
002785 : Not enough data to calculate ...
233990 : Empty Incomes
233990 : Empty Balances
233990 : Not enough data to calculate ...
085660 : Calculate ROE ... OK
261780 : Calculate ROE ... OK
009310 : Calculate ROE ... OK
158310 : Calculate ROE ... OK
094850 : Calculate ROE ... OK
004650 : Calc

054450 : Calculate ROE ... OK
091440 : Calculate ROE ... OK
078000 : Calculate ROE ... OK
200230 : Calculate ROE ... OK
214420 : Calculate ROE ... OK
393210 : Empty Incomes
393210 : Empty Balances
393210 : Not enough data to calculate ...
215480 : Calculate ROE ... OK
051360 : Calculate ROE ... OK
045340 : Calculate ROE ... OK
108230 : Calculate ROE ... OK
079970 : Calculate ROE ... OK
199800 : Calculate ROE ... OK
105550 : Calculate ROE ... OK
290090 : Calculate ROE ... OK
026150 : Calculate ROE ... OK
322180 : Calculate ROE ... OK
117730 : Calculate ROE ... OK
033830 : Calculate ROE ... OK
057680 : Calculate ROE ... OK
064760 : Calculate ROE ... OK
246710 : Calculate ROE ... OK
340570 : Calculate ROE ... OK
277880 : Calculate ROE ... OK
131290 : Calculate ROE ... OK
019180 : Calculate ROE ... OK
356860 : Calculate ROE ... OK
062860 : Calculate ROE ... OK
363280 : Calculate ROE ... OK
36328K : Empty Incomes
36328K : Empty Balances
36328K : Not enough data to calculate ...
321550 : Cal

021650 : Calculate ROE ... OK
161390 : Calculate ROE ... OK
053590 : Calculate ROE ... OK
034830 : Calculate ROE ... OK
007280 : Calculate ROE ... OK
032300 : Calculate ROE ... OK
168490 : Empty Incomes
168490 : Empty Balances
168490 : Not enough data to calculate ...
037230 : Calculate ROE ... OK
010100 : Calculate ROE ... OK
047810 : Calculate ROE ... OK
123690 : Calculate ROE ... OK
003350 : Calculate ROE ... OK
030520 : Calculate ROE ... OK
052600 : Calculate ROE ... OK
011500 : Calculate ROE ... OK
002390 : Calculate ROE ... OK
256150 : Calculate ROE ... OK
014790 : Calculate ROE ... OK
092460 : Calculate ROE ... OK
060980 : Calculate ROE ... OK
053690 : Calculate ROE ... OK
042700 : Calculate ROE ... OK
008930 : Calculate ROE ... OK
128940 : Not enough data to calculate ...
047080 : Calculate ROE ... OK
009240 : Calculate ROE ... OK
020000 : Calculate ROE ... OK
003680 : Calculate ROE ... OK
105630 : Calculate ROE ... OK
069640 : Calculate ROE ... OK
016450 : Calculate ROE ... OK

,code,name,exchange,period,mktcap,ROE
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.059318
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,-0.043428
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.028466
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.076769
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,0.000000
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.018331
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,0.000000
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,0.040262
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.028064


In [129]:
# ROE 종목 선정
df_factor = ROE_list.copy()
factor_list = ['ROE']

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # ROA, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # ROA, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # ROA, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,ROE,score
1,205470,휴마시스,0.402038,100.000000
2,950130,엑세스바이오,0.275704,68.576711
3,287410,제이시스메디칼,0.158077,39.318940
4,112040,위메이드,0.143906,35.794254
5,096530,씨젠,0.130827,32.540986
6,036930,주성엔지니어링,0.119420,29.703599
7,222800,심텍,0.117104,29.127687
8,108320,LX세미콘,0.114927,28.586160
9,106240,파인테크닉스,0.104701,26.042623
10,067160,아프리카TV,0.094589,23.527436


#### RIM

In [74]:
# RIM 구하기
RIM_list = b.getRIM(0.1) # 목표 수익률 10%
RIM_list

060310 : Empty Incomes
060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate RIM ... OK
006840 : Calculate RIM ... OK
054620 : Calculate RIM ... OK
265520 : Calculate RIM ... OK
211270 : Calculate RIM ... OK
027410 : Calculate RIM ... OK
282330 : Calculate RIM ... OK
032790 : Calculate RIM ... OK
138930 : Calculate RIM ... OK
001460 : Calculate RIM ... OK
001465 : Empty Incomes
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate RIM ... OK
001040 : Calculate RIM ... OK
079160 : Calculate RIM ... OK
035760 : Calculate RIM ... OK
311690 : Calculate RIM ... OK
00104K : Empty Incomes
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate RIM ... OK
011150 : Calculate RIM ... OK
011155 : Empty Incomes
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Incomes
001045 : Empty Balances
001045 : Not enough data to calculate ...
097950 : Calculate RIM ... OK
097955 : Empty 

011790 : Calculate RIM ... OK
018670 : Calculate RIM ... OK
001740 : Calculate RIM ... OK
001745 : Empty Incomes
001745 : Empty Balances
001745 : Not enough data to calculate ...
006120 : Calculate RIM ... OK
006125 : Empty Incomes
006125 : Empty Balances
006125 : Not enough data to calculate ...
210980 : Calculate RIM ... OK
068400 : Calculate RIM ... OK
395400 : Calculate RIM ... OK
302440 : Calculate RIM ... OK
326030 : Not enough data to calculate ...
402340 : Calculate RIM ... OK
361610 : Calculate RIM ... OK
03473K : Empty Incomes
03473K : Empty Balances
03473K : Not enough data to calculate ...
096770 : Calculate RIM ... OK
096775 : Empty Incomes
096775 : Empty Balances
096775 : Not enough data to calculate ...
001510 : Calculate RIM ... OK
001515 : Empty Incomes
001515 : Empty Balances
001515 : Not enough data to calculate ...
285130 : Calculate RIM ... OK
28513K : Empty Incomes
28513K : Empty Balances
28513K : Not enough data to calculate ...
017670 : Calculate RIM ... OK
0006

194700 : Calculate RIM ... OK
285490 : Calculate RIM ... OK
229500 : Empty Incomes
229500 : Empty Balances
229500 : Not enough data to calculate ...
106520 : Calculate RIM ... OK
376930 : Calculate RIM ... OK
278650 : Calculate RIM ... OK
006280 : Calculate RIM ... OK
142280 : Calculate RIM ... OK
234690 : Calculate RIM ... OK
005250 : Calculate RIM ... OK
005257 : Empty Incomes
005257 : Empty Balances
005257 : Not enough data to calculate ...
065560 : Calculate RIM ... OK
004370 : Calculate RIM ... OK
072710 : Calculate RIM ... OK
054050 : Calculate RIM ... OK
069140 : Calculate RIM ... OK
040160 : Calculate RIM ... OK
332290 : Calculate RIM ... OK
126870 : Calculate RIM ... OK
060260 : Calculate RIM ... OK
012340 : Calculate RIM ... OK
214870 : Calculate RIM ... OK
270870 : Calculate RIM ... OK
144960 : Calculate RIM ... OK
085670 : Calculate RIM ... OK
119860 : Calculate RIM ... OK
064260 : Calculate RIM ... OK
093640 : Calculate RIM ... OK
340360 : Calculate RIM ... OK
039560 : Cal

034020 : Calculate RIM ... OK
336260 : Calculate RIM ... OK
33626K : Empty Incomes
33626K : Empty Balances
33626K : Not enough data to calculate ...
33626L : Empty Incomes
33626L : Empty Balances
33626L : Not enough data to calculate ...
016740 : Calculate RIM ... OK
073190 : Calculate RIM ... OK
176750 : Empty Incomes
176750 : Empty Balances
176750 : Not enough data to calculate ...
030350 : Calculate RIM ... OK
203650 : Calculate RIM ... OK
223250 : Calculate RIM ... OK
060570 : Calculate RIM ... OK
192650 : Calculate RIM ... OK
217620 : Calculate RIM ... OK
187870 : Calculate RIM ... OK
066670 : Calculate RIM ... OK
024090 : Calculate RIM ... OK
003160 : Calculate RIM ... OK
092200 : Calculate RIM ... OK
110990 : Calculate RIM ... OK
078590 : Calculate RIM ... OK
263690 : Calculate RIM ... OK
214680 : Calculate RIM ... OK
377190 : Calculate RIM ... OK
263720 : Calculate RIM ... OK
376300 : Calculate RIM ... OK
109740 : Calculate RIM ... OK
131030 : Calculate RIM ... OK
196490 : Calc

214610 : Calculate RIM ... OK
201490 : Calculate RIM ... OK
950190 : Calculate RIM ... OK
214180 : Calculate RIM ... OK
155900 : Not enough data to calculate ...
206640 : Calculate RIM ... OK
018700 : Calculate RIM ... OK
035620 : Calculate RIM ... OK
064520 : Calculate RIM ... OK
053030 : Calculate RIM ... OK
301300 : Calculate RIM ... OK
064550 : Calculate RIM ... OK
314930 : Calculate RIM ... OK
208710 : Calculate RIM ... OK
142760 : Calculate RIM ... OK
086820 : Calculate RIM ... OK
038460 : Calculate RIM ... OK
281310 : Empty Incomes
281310 : Empty Balances
281310 : Not enough data to calculate ...
251120 : Calculate RIM ... OK
266470 : Empty Incomes
266470 : Empty Balances
266470 : Not enough data to calculate ...
086040 : Calculate RIM ... OK
199290 : Empty Incomes
199290 : Empty Balances
199290 : Not enough data to calculate ...
099430 : Calculate RIM ... OK
032980 : Calculate RIM ... OK
222160 : Empty Incomes
222160 : Empty Balances
222160 : Not enough data to calculate ...
30

...

003460 : Calculate RIM ... OK
003465 : Empty Incomes
003465 : Empty Balances
003465 : Not enough data to calculate ...
191410 : Calculate RIM ... OK
008730 : Calculate RIM ... OK
072770 : Calculate RIM ... OK
008250 : Calculate RIM ... OK
039020 : Calculate RIM ... OK
025820 : Calculate RIM ... OK
067920 : Calculate RIM ... OK
044960 : Calculate RIM ... OK
303530 : Calculate RIM ... OK
302430 : Calculate RIM ... OK
279060 : Empty Incomes
279060 : Empty Balances
279060 : Not enough data to calculate ...
214320 : Calculate RIM ... OK
073490 : Calculate RIM ... OK
215790 : Not enough data to calculate ...
344860 : Empty Incomes
344860 : Empty Balances
344860 : Not enough data to calculate ...
246960 : Calculate RIM ... OK
088390 : Calculate RIM ... OK
272290 : Calculate RIM ... OK
053350 : Calculate RIM ... OK
258610 : Calculate RIM ... OK
041520 : Calculate RIM ... OK
264850 : Calculate RIM ... OK
054210 : Calculate RIM ... OK
164060 : Calculate RIM ... OK
065440 : Calculate RIM ... OK
0

065060 : Calculate RIM ... OK
204840 : Calculate RIM ... OK
013870 : Calculate RIM ... OK
071320 : Calculate RIM ... OK
135160 : Empty Incomes
135160 : Empty Balances
135160 : Not enough data to calculate ...
311320 : Calculate RIM ... OK
010580 : Calculate RIM ... OK
035000 : Calculate RIM ... OK
219750 : Calculate RIM ... OK
088790 : Calculate RIM ... OK
018120 : Calculate RIM ... OK
109820 : Calculate RIM ... OK
086060 : Calculate RIM ... OK
036890 : Calculate RIM ... OK
363250 : Calculate RIM ... OK
003780 : Calculate RIM ... OK
007370 : Calculate RIM ... OK
010640 : Calculate RIM ... OK
100250 : Calculate RIM ... OK
051630 : Calculate RIM ... OK
272450 : Calculate RIM ... OK
011000 : Calculate RIM ... OK
250030 : Calculate RIM ... OK
002780 : Calculate RIM ... OK
002787 : Empty Incomes
002787 : Empty Balances
002787 : Not enough data to calculate ...
002785 : Empty Incomes
002785 : Empty Balances
002785 : Not enough data to calculate ...
233990 : Empty Incomes
233990 : Empty Balan

055490 : Calculate RIM ... OK
308700 : Empty Incomes
308700 : Empty Balances
308700 : Not enough data to calculate ...
089030 : Calculate RIM ... OK
258050 : Empty Incomes
258050 : Empty Balances
258050 : Not enough data to calculate ...
054450 : Calculate RIM ... OK
091440 : Calculate RIM ... OK
078000 : Calculate RIM ... OK
200230 : Calculate RIM ... OK
214420 : Calculate RIM ... OK
393210 : Empty Incomes
393210 : Empty Balances
393210 : Not enough data to calculate ...
215480 : Calculate RIM ... OK
051360 : Calculate RIM ... OK
045340 : Calculate RIM ... OK
108230 : Calculate RIM ... OK
079970 : Calculate RIM ... OK
199800 : Calculate RIM ... OK
105550 : Calculate RIM ... OK
290090 : Calculate RIM ... OK
026150 : Calculate RIM ... OK
322180 : Calculate RIM ... OK
117730 : Calculate RIM ... OK
033830 : Calculate RIM ... OK
057680 : Calculate RIM ... OK
064760 : Calculate RIM ... OK
246710 : Calculate RIM ... OK
340570 : Calculate RIM ... OK
277880 : Calculate RIM ... OK
131290 : Calc

000970 : Calculate RIM ... OK
104700 : Calculate RIM ... OK
017960 : Calculate RIM ... OK
023760 : Calculate RIM ... OK
054040 : Calculate RIM ... OK
050540 : Calculate RIM ... OK
161890 : Not enough data to calculate ...
024720 : Calculate RIM ... OK
021650 : Calculate RIM ... OK
161390 : Calculate RIM ... OK
053590 : Calculate RIM ... OK
034830 : Calculate RIM ... OK
007280 : Calculate RIM ... OK
032300 : Calculate RIM ... OK
168490 : Empty Incomes
168490 : Empty Balances
168490 : Not enough data to calculate ...
037230 : Calculate RIM ... OK
010100 : Calculate RIM ... OK
047810 : Calculate RIM ... OK
123690 : Calculate RIM ... OK
003350 : Calculate RIM ... OK
030520 : Calculate RIM ... OK
052600 : Calculate RIM ... OK
011500 : Calculate RIM ... OK
002390 : Calculate RIM ... OK
256150 : Calculate RIM ... OK
014790 : Calculate RIM ... OK
092460 : Calculate RIM ... OK
060980 : Calculate RIM ... OK
053690 : Calculate RIM ... OK
042700 : Calculate RIM ... OK
008930 : Calculate RIM ... OK

,code,name,exchange,period,mktcap,RIM
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.349986
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,NaN
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,1.224396
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.646768
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,2.708472
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,3.821897
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,1.942766


In [75]:
# RIM 종목 선정
df_factor = RIM_list.copy()
factor_list = ['RIM']

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # RIM, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # RIM, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # RIM, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,RIM,score
1,032190,다우데이타,0.071406,100.000000
2,950130,엑세스바이오,0.185005,99.767705
3,003380,하림지주,0.194586,99.748113
4,205470,휴마시스,0.205836,99.725109
5,096530,씨젠,0.495106,99.133590
6,049070,인탑스,0.545486,99.030569
7,090460,비에이치,0.685142,98.744990
8,108320,LX세미콘,0.689252,98.736587
9,086520,에코프로,0.710849,98.692423
10,121800,비덴트,0.780221,98.550567


#### GP/A

In [84]:
# GP/A 구하기
GPA_list = b.getGPA()
GPA_list

060310 : Empty Incomes
060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate GP/A ... OK
006840 : Calculate GP/A ... OK
054620 : Calculate GP/A ... OK
265520 : Calculate GP/A ... OK
211270 : Calculate GP/A ... OK
027410 : Calculate GP/A ... OK
282330 : Calculate GP/A ... OK
032790 : Calculate GP/A ... OK
138930 : Calculate GP/A ... OK
001460 : Calculate GP/A ... OK
001465 : Empty Incomes
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate GP/A ... OK
001040 : Calculate GP/A ... OK
079160 : Calculate GP/A ... OK
035760 : Calculate GP/A ... OK
311690 : Calculate GP/A ... OK
00104K : Empty Incomes
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate GP/A ... OK
011150 : Calculate GP/A ... OK
011155 : Empty Incomes
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Incomes
001045 : Empty Balances
001045 : Not enough data to calculate ...
097950 : Calculate GP/A ... 

123700 : Not enough data to calculate ...
025530 : Calculate GP/A ... OK
034730 : Calculate GP/A ... OK
337450 : Not enough data to calculate ...
340350 : Not enough data to calculate ...
011790 : Calculate GP/A ... OK
018670 : Calculate GP/A ... OK
001740 : Calculate GP/A ... OK
001745 : Empty Incomes
001745 : Empty Balances
001745 : Not enough data to calculate ...
006120 : Calculate GP/A ... OK
006125 : Empty Incomes
006125 : Empty Balances
006125 : Not enough data to calculate ...
210980 : Calculate GP/A ... OK
068400 : Calculate GP/A ... OK
395400 : Calculate GP/A ... OK
302440 : Calculate GP/A ... OK
326030 : Not enough data to calculate ...
402340 : Calculate GP/A ... OK
361610 : Calculate GP/A ... OK
03473K : Empty Incomes
03473K : Empty Balances
03473K : Not enough data to calculate ...
096770 : Calculate GP/A ... OK
096775 : Empty Incomes
096775 : Empty Balances
096775 : Not enough data to calculate ...
001510 : Calculate GP/A ... OK
001515 : Empty Incomes
001515 : Empty Bala

317860 : Empty Balances
317860 : Not enough data to calculate ...
104620 : Calculate GP/A ... OK
090350 : Not enough data to calculate ...
090355 : Empty Incomes
090355 : Empty Balances
090355 : Not enough data to calculate ...
000320 : Calculate GP/A ... OK
000325 : Empty Incomes
000325 : Empty Balances
000325 : Not enough data to calculate ...
194700 : Calculate GP/A ... OK
285490 : Calculate GP/A ... OK
229500 : Empty Incomes
229500 : Empty Balances
229500 : Not enough data to calculate ...
106520 : Calculate GP/A ... OK
376930 : Calculate GP/A ... OK
278650 : Calculate GP/A ... OK
006280 : Calculate GP/A ... OK
142280 : Calculate GP/A ... OK
234690 : Calculate GP/A ... OK
005250 : Calculate GP/A ... OK
005257 : Empty Incomes
005257 : Empty Balances
005257 : Not enough data to calculate ...
065560 : Calculate GP/A ... OK
004370 : Calculate GP/A ... OK
072710 : Calculate GP/A ... OK
054050 : Calculate GP/A ... OK
069140 : Calculate GP/A ... OK
040160 : Calculate GP/A ... OK
332290 : 

163560 : Calculate GP/A ... OK
109860 : Calculate GP/A ... OK
032960 : Calculate GP/A ... OK
004890 : Calculate GP/A ... OK
002690 : Calculate GP/A ... OK
023790 : Calculate GP/A ... OK
005290 : Calculate GP/A ... OK
025900 : Calculate GP/A ... OK
000020 : Calculate GP/A ... OK
000150 : Calculate GP/A ... OK
000157 : Empty Incomes
000157 : Empty Balances
000157 : Not enough data to calculate ...
241560 : Calculate GP/A ... OK
000155 : Empty Incomes
000155 : Empty Balances
000155 : Not enough data to calculate ...
034020 : Calculate GP/A ... OK
336260 : Calculate GP/A ... OK
33626K : Empty Incomes
33626K : Empty Balances
33626K : Not enough data to calculate ...
33626L : Empty Incomes
33626L : Empty Balances
33626L : Not enough data to calculate ...
016740 : Calculate GP/A ... OK
073190 : Calculate GP/A ... OK
176750 : Empty Incomes
176750 : Empty Balances
176750 : Not enough data to calculate ...
030350 : Calculate GP/A ... OK
203650 : Calculate GP/A ... OK
223250 : Calculate GP/A ... 

085620 : Calculate GP/A ... OK
006800 : Calculate GP/A ... OK
00680K : Empty Incomes
00680K : Empty Balances
00680K : Not enough data to calculate ...
006805 : Empty Incomes
006805 : Empty Balances
006805 : Not enough data to calculate ...
208890 : Empty Incomes
208890 : Empty Balances
208890 : Not enough data to calculate ...
049950 : Calculate GP/A ... OK
207760 : Calculate GP/A ... OK
225850 : Empty Incomes
225850 : Empty Balances
225850 : Not enough data to calculate ...
002840 : Calculate GP/A ... OK
268280 : Calculate GP/A ... OK
107590 : Not enough data to calculate ...
134380 : Calculate GP/A ... OK
003650 : Calculate GP/A ... OK
059090 : Calculate GP/A ... OK
214610 : Calculate GP/A ... OK
201490 : Calculate GP/A ... OK
950190 : Calculate GP/A ... OK
214180 : Calculate GP/A ... OK
155900 : Not enough data to calculate ...
206640 : Calculate GP/A ... OK
018700 : Calculate GP/A ... OK
035620 : Calculate GP/A ... OK
064520 : Calculate GP/A ... OK
053030 : Calculate GP/A ... OK
30

...

038010 : Calculate GP/A ... OK
002620 : Calculate GP/A ... OK
276730 : Calculate GP/A ... OK
080220 : Calculate GP/A ... OK
006220 : Calculate GP/A ... OK
089590 : Calculate GP/A ... OK
216080 : Calculate GP/A ... OK
229000 : Calculate GP/A ... OK
248020 : Empty Incomes
248020 : Empty Balances
248020 : Not enough data to calculate ...
082270 : Calculate GP/A ... OK
064800 : Calculate GP/A ... OK
041590 : Calculate GP/A ... OK
044060 : Calculate GP/A ... OK
004910 : Calculate GP/A ... OK
004700 : Calculate GP/A ... OK
001550 : Calculate GP/A ... OK
000480 : Calculate GP/A ... OK
120030 : Calculate GP/A ... OK
034940 : Calculate GP/A ... OK
067000 : Calculate GP/A ... OK
018470 : Calculate GP/A ... OK
002600 : Calculate GP/A ... OK
185750 : Calculate GP/A ... OK
063160 : Calculate GP/A ... OK
001630 : Calculate GP/A ... OK
033340 : Calculate GP/A ... OK
036930 : Calculate GP/A ... OK
044380 : Calculate GP/A ... OK
239340 : Calculate GP/A ... OK
051980 : Calculate GP/A ... OK
072020 : Cal

259960 : Calculate GP/A ... OK
096240 : Calculate GP/A ... OK
215570 : Empty Incomes
215570 : Empty Balances
215570 : Not enough data to calculate ...
043590 : Calculate GP/A ... OK
114120 : Calculate GP/A ... OK
110790 : Calculate GP/A ... OK
900250 : Calculate GP/A ... OK
083790 : Calculate GP/A ... OK
045520 : Calculate GP/A ... OK
036170 : Calculate GP/A ... OK
214150 : Calculate GP/A ... OK
352770 : Calculate GP/A ... OK
237880 : Calculate GP/A ... OK
139670 : Calculate GP/A ... OK
020120 : Calculate GP/A ... OK
039490 : Calculate GP/A ... OK
054780 : Calculate GP/A ... OK
336040 : Empty Incomes
336040 : Empty Balances
336040 : Not enough data to calculate ...
219130 : Calculate GP/A ... OK
318660 : Empty Incomes
318660 : Empty Balances
318660 : Not enough data to calculate ...
180060 : Empty Incomes
180060 : Empty Balances
180060 : Not enough data to calculate ...
065130 : Calculate GP/A ... OK
134580 : Calculate GP/A ... OK
014580 : Calculate GP/A ... OK
015890 : Calculate GP/A 

153360 : Empty Balances
153360 : Not enough data to calculate ...
149980 : Calculate GP/A ... OK
013030 : Calculate GP/A ... OK
352820 : Calculate GP/A ... OK
126700 : Calculate GP/A ... OK
106080 : Calculate GP/A ... OK
071090 : Calculate GP/A ... OK
340120 : Not enough data to calculate ...
377400 : Not enough data to calculate ...
400840 : Not enough data to calculate ...
221840 : Calculate GP/A ... OK
106190 : Calculate GP/A ... OK
019490 : Calculate GP/A ... OK
000080 : Calculate GP/A ... OK
000087 : Empty Incomes
000087 : Empty Balances
000087 : Not enough data to calculate ...
000140 : Calculate GP/A ... OK
000145 : Empty Incomes
000145 : Empty Balances
000145 : Not enough data to calculate ...
373200 : Calculate GP/A ... OK
066130 : Calculate GP/A ... OK
152550 : Empty Incomes
152550 : Empty Balances
152550 : Not enough data to calculate ...
004590 : Calculate GP/A ... OK
036460 : Calculate GP/A ... OK
039340 : Calculate GP/A ... OK
005430 : Calculate GP/A ... OK
071050 : Calcu

126640 : Calculate GP/A ... OK
133820 : Calculate GP/A ... OK
061250 : Calculate GP/A ... OK
010660 : Calculate GP/A ... OK
000850 : Calculate GP/A ... OK
016580 : Calculate GP/A ... OK
032560 : Calculate GP/A ... OK
004800 : Calculate GP/A ... OK
094280 : Calculate GP/A ... OK
097870 : Not enough data to calculate ...
298040 : Calculate GP/A ... OK
298050 : Calculate GP/A ... OK
298020 : Calculate GP/A ... OK
298000 : Calculate GP/A ... OK
093370 : Calculate GP/A ... OK
050090 : Calculate GP/A ... OK
081660 : Calculate GP/A ... OK
290270 : Calculate GP/A ... OK
005870 : Calculate GP/A ... OK
353190 : Not enough data to calculate ...
090710 : Calculate GP/A ... OK
205470 : Calculate GP/A ... OK
115160 : Calculate GP/A ... OK
028080 : Calculate GP/A ... OK
032860 : Calculate GP/A ... OK
200670 : Calculate GP/A ... OK
212310 : Empty Incomes
212310 : Empty Balances
212310 : Not enough data to calculate ...
079980 : Calculate GP/A ... OK
065510 : Calculate GP/A ... OK
215090 : Calculate GP

,code,name,exchange,period,mktcap,GP/A
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.198893
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.028616
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.015375
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.067810
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,0.000000
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.051224
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,0.000000
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,0.007344
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.035921


In [130]:
# GP/A 종목 선정
df_factor = GPA_list.copy()
factor_list = ['GP/A']

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # ROA, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # ROA, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # ROA, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,GP/A,score
1,950130,엑세스바이오,0.396320,100.000000
2,205470,휴마시스,0.350303,88.388839
3,194480,데브시스터즈,0.334512,84.404361
4,030190,NICE평가정보,0.300702,75.873559
5,287410,제이시스메디칼,0.230406,58.136185
6,067000,조이시티,0.225498,56.897897
7,230360,에코마케팅,0.188562,47.578057
8,101730,위메이드맥스,0.187661,47.350892
9,067160,아프리카TV,0.186337,47.016859
10,096530,씨젠,0.182530,46.056238


#### 안정성 지표

In [181]:
# 안정성지표 구하기
ld_list = b.getStability()
ld_list

060310 : Empty Balances
060310 : Data Not Enough!
095570 : Calculate Stability ... OK
006840 : Calculate Stability ... OK
054620 : Calculate Stability ... OK
265520 : Calculate Stability ... OK
211270 : Calculate Stability ... OK
027410 : Calculate Stability ... OK
282330 : Calculate Stability ... OK
032790 : Calculate Stability ... OK
138930 : Calculate Stability ... OK
001460 : Calculate Stability ... OK
001465 : Empty Balances
001465 : Data Not Enough!
013720 : Calculate Stability ... OK
001040 : Calculate Stability ... OK
079160 : Calculate Stability ... OK
035760 : Calculate Stability ... OK
311690 : Calculate Stability ... OK
00104K : Empty Balances
00104K : Data Not Enough!
000120 : Calculate Stability ... OK
011150 : Calculate Stability ... OK
011155 : Empty Balances
011155 : Data Not Enough!
001045 : Empty Balances
001045 : Data Not Enough!
097950 : Calculate Stability ... OK
097955 : Empty Balances
097955 : Data Not Enough!
051500 : Calculate Stability ... OK
058820 : Calcula

395400 : Calculate Stability ... OK
302440 : Calculate Stability ... OK
326030 : Data Not Enough!
402340 : Calculate Stability ... OK
361610 : Calculate Stability ... OK
03473K : Empty Balances
03473K : Data Not Enough!
096770 : Calculate Stability ... OK
096775 : Empty Balances
096775 : Data Not Enough!
001510 : Calculate Stability ... OK
001515 : Empty Balances
001515 : Data Not Enough!
285130 : Calculate Stability ... OK
28513K : Empty Balances
28513K : Data Not Enough!
017670 : Calculate Stability ... OK
000660 : Calculate Stability ... OK
048550 : Calculate Stability ... OK
063440 : Calculate Stability ... OK
950180 : Empty Balances
950180 : Data Not Enough!
064960 : Calculate Stability ... OK
100840 : Calculate Stability ... OK
003570 : Calculate Stability ... OK
036530 : Calculate Stability ... OK
005610 : Calculate Stability ... OK
011810 : Calculate Stability ... OK
077970 : Calculate Stability ... OK
071970 : Calculate Stability ... OK
002820 : Calculate Stability ... OK
2890

040160 : Calculate Stability ... OK
332290 : Calculate Stability ... OK
126870 : Calculate Stability ... OK
060260 : Calculate Stability ... OK
012340 : Calculate Stability ... OK
214870 : Calculate Stability ... OK
270870 : Calculate Stability ... OK
144960 : Calculate Stability ... OK
085670 : Calculate Stability ... OK
119860 : Calculate Stability ... OK
064260 : Calculate Stability ... OK
093640 : Calculate Stability ... OK
340360 : Calculate Stability ... OK
039560 : Calculate Stability ... OK
058730 : Calculate Stability ... OK
023590 : Calculate Stability ... OK
032190 : Calculate Stability ... OK
323350 : Empty Balances
323350 : Data Not Enough!
068240 : Calculate Stability ... OK
145210 : Calculate Stability ... OK
271850 : Empty Balances
271850 : Data Not Enough!
343090 : Empty Balances
343090 : Data Not Enough!
019680 : Calculate Stability ... OK
019685 : Empty Balances
019685 : Data Not Enough!
006370 : Calculate Stability ... OK
008060 : Calculate Stability ... OK
00806K :

196490 : Calculate Stability ... OK
066900 : Calculate Stability ... OK
180400 : Calculate Stability ... OK
127120 : Calculate Stability ... OK
092070 : Calculate Stability ... OK
039840 : Calculate Stability ... OK
013570 : Calculate Stability ... OK
219550 : Calculate Stability ... OK
310870 : Calculate Stability ... OK
210540 : Calculate Stability ... OK
104460 : Calculate Stability ... OK
079810 : Calculate Stability ... OK
113810 : Calculate Stability ... OK
043360 : Calculate Stability ... OK
197140 : Calculate Stability ... OK
068930 : Calculate Stability ... OK
033130 : Calculate Stability ... OK
105740 : Calculate Stability ... OK
263020 : Calculate Stability ... OK
290550 : Calculate Stability ... OK
007340 : Calculate Stability ... OK
187220 : Calculate Stability ... OK
163430 : Empty Balances
163430 : Data Not Enough!
131180 : Data Not Enough!
315640 : Calculate Stability ... OK
317120 : Calculate Stability ... OK
042510 : Calculate Stability ... OK
232680 : Calculate Stabi

177350 : Calculate Stability ... OK
299910 : Calculate Stability ... OK
096300 : Empty Balances
096300 : Data Not Enough!
007210 : Calculate Stability ... OK
225530 : Calculate Stability ... OK
250000 : Calculate Stability ... OK
002760 : Calculate Stability ... OK
003850 : Calculate Stability ... OK
006910 : Calculate Stability ... OK
000890 : Calculate Stability ... OK
226340 : Data Not Enough!
206950 : Empty Balances
206950 : Data Not Enough!
003000 : Calculate Stability ... OK
001270 : Calculate Stability ... OK
001275 : Empty Balances
001275 : Data Not Enough!
026940 : Calculate Stability ... OK
014470 : Calculate Stability ... OK
011390 : Calculate Stability ... OK
005030 : Calculate Stability ... OK
008470 : Calculate Stability ... OK
338220 : Calculate Stability ... OK
100120 : Calculate Stability ... OK
337930 : Calculate Stability ... OK
099390 : Calculate Stability ... OK
064480 : Calculate Stability ... OK
288330 : Calculate Stability ... OK
365900 : Calculate Stability ...

...

139480 : Calculate Stability ... OK
115610 : Calculate Stability ... OK
349720 : Data Not Enough!
078020 : Calculate Stability ... OK
131400 : Calculate Stability ... OK
208850 : Empty Balances
208850 : Data Not Enough!
351330 : Calculate Stability ... OK
080010 : Calculate Stability ... OK
379390 : Empty Balances
379390 : Data Not Enough!
086890 : Calculate Stability ... OK
007660 : Calculate Stability ... OK
005950 : Calculate Stability ... OK
015020 : Calculate Stability ... OK
047560 : Data Not Enough!
900110 : Calculate Stability ... OK
270020 : Empty Balances
270020 : Data Not Enough!
067010 : Empty Balances
067010 : Data Not Enough!
093230 : Calculate Stability ... OK
341310 : Empty Balances
341310 : Data Not Enough!
101360 : Calculate Stability ... OK
102710 : Calculate Stability ... OK
066980 : Calculate Stability ... OK
074610 : Calculate Stability ... OK
063760 : Calculate Stability ... OK
123570 : Calculate Stability ... OK
083470 : Calculate Stability ... OK
095190 : Calcu

100250 : Calculate Stability ... OK
051630 : Calculate Stability ... OK
272450 : Calculate Stability ... OK
011000 : Calculate Stability ... OK
250030 : Calculate Stability ... OK
002780 : Calculate Stability ... OK
002787 : Empty Balances
002787 : Data Not Enough!
002785 : Empty Balances
002785 : Data Not Enough!
233990 : Empty Balances
233990 : Data Not Enough!
085660 : Calculate Stability ... OK
261780 : Calculate Stability ... OK
009310 : Calculate Stability ... OK
158310 : Calculate Stability ... OK
094850 : Calculate Stability ... OK
004650 : Calculate Stability ... OK
278280 : Calculate Stability ... OK
000650 : Calculate Stability ... OK
140290 : Empty Balances
140290 : Data Not Enough!
066360 : Calculate Stability ... OK
033250 : Calculate Stability ... OK
047820 : Calculate Stability ... OK
052300 : Calculate Stability ... OK
094360 : Calculate Stability ... OK
016920 : Calculate Stability ... OK
284620 : Calculate Stability ... OK
035720 : Calculate Stability ... OK
293490 :

117730 : Calculate Stability ... OK
033830 : Calculate Stability ... OK
057680 : Calculate Stability ... OK
064760 : Calculate Stability ... OK
246710 : Calculate Stability ... OK
340570 : Calculate Stability ... OK
277880 : Calculate Stability ... OK
131290 : Calculate Stability ... OK
019180 : Calculate Stability ... OK
356860 : Calculate Stability ... OK
062860 : Calculate Stability ... OK
363280 : Calculate Stability ... OK
36328K : Empty Balances
36328K : Data Not Enough!
321550 : Calculate Stability ... OK
091810 : Calculate Stability ... OK
004870 : Calculate Stability ... OK
104480 : Calculate Stability ... OK
309900 : Empty Balances
309900 : Data Not Enough!
081150 : Calculate Stability ... OK
130740 : Data Not Enough!
217880 : Empty Balances
217880 : Data Not Enough!
084730 : Calculate Stability ... OK
046210 : Calculate Stability ... OK
058530 : Calculate Stability ... OK
034230 : Calculate Stability ... OK
033540 : Calculate Stability ... OK
043200 : Calculate Stability ...

256150 : Calculate Stability ... OK
014790 : Calculate Stability ... OK
092460 : Calculate Stability ... OK
060980 : Calculate Stability ... OK
053690 : Calculate Stability ... OK
042700 : Calculate Stability ... OK
008930 : Calculate Stability ... OK
128940 : Data Not Enough!
047080 : Calculate Stability ... OK
009240 : Calculate Stability ... OK
020000 : Calculate Stability ... OK
003680 : Calculate Stability ... OK
105630 : Calculate Stability ... OK
069640 : Calculate Stability ... OK
016450 : Calculate Stability ... OK
010420 : Calculate Stability ... OK
009180 : Calculate Stability ... OK
070590 : Calculate Stability ... OK
213500 : Calculate Stability ... OK
014680 : Calculate Stability ... OK
004710 : Calculate Stability ... OK
004150 : Calculate Stability ... OK
025750 : Calculate Stability ... OK
226440 : Calculate Stability ... OK
042520 : Calculate Stability ... OK
004960 : Calculate Stability ... OK
011700 : Calculate Stability ... OK
078350 : Calculate Stability ... OK
04

,code,name,exchange,period,mktcap,L/E,D/E
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,2.895370,1.011022
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,2.878094,1.551008
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.738874,0.424143
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,1.358210,0.490054
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,0.370914,0.000000
...,...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.652682,0.471724
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,41.657558,0.000000
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,1.408216,0.006598
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,1.809843,0.527872


In [187]:
# 안정성지표 종목 선정 (부채비율)
df_factor = ld_list.copy()
factor_list = ['L/E'] # 부채비율

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 부채비율, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 부채비율, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # 부채비율, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,L/E,score
1,041190,우리기술투자,0.000000,100.000000
2,199800,툴젠,0.035553,99.593128
3,323990,박셀바이오,0.039761,99.544972
4,376300,디어유,0.080760,99.075768
5,214370,케어젠,0.094721,98.915998
6,067630,HLB생명과학,0.109883,98.742477
7,082270,젬백스,0.115339,98.680040
8,058470,리노공업,0.116513,98.666599
9,950220,네오이뮨텍,0.132905,98.479012
10,141080,레고켐바이오,0.133162,98.476070


In [190]:
# 안정성지표 종목 선정 (차입금비율)
df_factor = ld_list.copy()
factor_list = ['D/E'] # 차입금비율
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 차입금비율, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 차입금비율, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSDAQ', 30, factor_list) # 차입금비율, 시가총액 상위 20%, KOSDAQ 종목 중 30개 선택
result

,code,name,D/E,score
1,348210,넥스틴,0.000000,100.000000
2,053800,안랩,0.000000,100.000000
3,145020,휴젤,0.000000,100.000000
4,089970,에이피티씨,0.000000,100.000000
5,064760,티씨케이,0.000000,100.000000
6,069080,웹젠,0.000000,100.000000
7,376300,디어유,0.000000,100.000000
8,215600,신라젠,0.000000,100.000000
9,298380,에이비엘바이오,0.000000,100.000000
10,095660,네오위즈,0.000000,100.000000


#### PER + 차입금비율 합성

In [192]:
# 안정성지표 종목 선정 (PER + 차입금비율 합성)
df_factor = pd.merge(per_list, ld_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])
factor_list = ['PER', 'D/E']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PER+차입금비율, 시가총액 상위 20%, 전체 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # PER+차입금비율, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,PER,D/E,score
1,001530,DI동일,1.279302,0.456876,99.999898
2,000880,한화,1.443846,0.395390,99.999833
3,011200,HMM,1.714468,0.026868,99.999779
4,139480,이마트,1.714299,0.151358,99.999754
5,024110,기업은행,1.697331,0.791234,99.999637
6,001120,LX인터내셔널,2.005910,0.386508,99.999572
7,000210,DL,1.947041,0.573978,99.999563
8,001230,동국제강,1.989159,0.696803,99.999520
9,006120,SK디스커버리,2.187466,0.364186,99.999492
10,034730,SK,2.203551,0.336127,99.999490


#### PBR + 차입금비율 합성

In [194]:
# 안정성지표 종목 선정 (PBR + 차입금비율 합성)
df_factor = pd.merge(pbr_list, ld_list, how='left', on=['code', 'name', 'exchange', 'period', 'mktcap'])
factor_list = ['PBR', 'D/E']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # PBR+차입금비율, 시가총액 상위 20%, 전체 종목 중 30개 선택
result

,code,name,PBR,D/E,score
1,004170,신세계,0.112386,0.119755,99.999990
2,029780,삼성카드,0.126229,0.000000,99.999989
3,011170,롯데케미칼,0.117520,0.107647,99.999988
4,003550,LG,0.132706,0.001911,99.999986
5,006650,대한유화,0.134723,0.025555,99.999984
6,005490,POSCO홀딩스,0.118080,0.179439,99.999984
7,161390,한국타이어앤테크놀로지,0.128108,0.089454,99.999984
8,002790,아모레G,0.138439,0.042340,99.999981
9,000720,현대건설,0.143751,0.034465,99.999979
10,030200,KT,0.143582,0.044937,99.999978


### 성장률 지표

### 회전률 지표

In [97]:
# Turnover 구하기
to_list = b.getTurnover()
to_list

060310 : Empty Incomes
060310 : Empty Balances
060310 : Not enough data to calculate ...
095570 : Calculate Turnover ... OK
006840 : Calculate Turnover ... OK
054620 : Calculate Turnover ... OK
265520 : Calculate Turnover ... OK
211270 : Calculate Turnover ... OK
027410 : Calculate Turnover ... OK
282330 : Calculate Turnover ... OK
032790 : Calculate Turnover ... OK
138930 : Calculate Turnover ... OK
001460 : Calculate Turnover ... OK
001465 : Empty Incomes
001465 : Empty Balances
001465 : Not enough data to calculate ...
013720 : Calculate Turnover ... OK
001040 : Calculate Turnover ... OK
079160 : Calculate Turnover ... OK
035760 : Calculate Turnover ... OK
311690 : Calculate Turnover ... OK
00104K : Empty Incomes
00104K : Empty Balances
00104K : Not enough data to calculate ...
000120 : Calculate Turnover ... OK
011150 : Calculate Turnover ... OK
011155 : Empty Incomes
011155 : Empty Balances
011155 : Not enough data to calculate ...
001045 : Empty Incomes
001045 : Empty Balances
00

019550 : Calculate Turnover ... OK
950110 : Empty Incomes
950110 : Empty Balances
950110 : Not enough data to calculate ...
034120 : Calculate Turnover ... OK
046140 : Calculate Turnover ... OK
036120 : Calculate Turnover ... OK
099220 : Calculate Turnover ... OK
036540 : Calculate Turnover ... OK
255220 : Calculate Turnover ... OK
040610 : Calculate Turnover ... OK
049470 : Calculate Turnover ... OK
184230 : Calculate Turnover ... OK
005090 : Calculate Turnover ... OK
016250 : Calculate Turnover ... OK
001380 : Calculate Turnover ... OK
004060 : Calculate Turnover ... OK
001770 : Calculate Turnover ... OK
002360 : Calculate Turnover ... OK
009160 : Calculate Turnover ... OK
123700 : Not enough data to calculate ...
025530 : Calculate Turnover ... OK
034730 : Calculate Turnover ... OK
337450 : Not enough data to calculate ...
340350 : Not enough data to calculate ...
011790 : Calculate Turnover ... OK
018670 : Calculate Turnover ... OK
001740 : Calculate Turnover ... OK
001745 : Empty 

111710 : Calculate Turnover ... OK
091590 : Calculate Turnover ... OK
168330 : Calculate Turnover ... OK
094860 : Calculate Turnover ... OK
253590 : Calculate Turnover ... OK
212560 : Calculate Turnover ... OK
095660 : Calculate Turnover ... OK
042420 : Calculate Turnover ... OK
950220 : Calculate Turnover ... OK
311390 : Calculate Turnover ... OK
085910 : Calculate Turnover ... OK
092730 : Calculate Turnover ... OK
290660 : Calculate Turnover ... OK
306620 : Not enough data to calculate ...
153460 : Calculate Turnover ... OK
007390 : Calculate Turnover ... OK
086220 : Empty Incomes
086220 : Empty Balances
086220 : Not enough data to calculate ...
033640 : Calculate Turnover ... OK
330860 : Calculate Turnover ... OK
005720 : Calculate Turnover ... OK
005725 : Empty Incomes
005725 : Empty Balances
005725 : Not enough data to calculate ...
002350 : Calculate Turnover ... OK
002355 : Empty Incomes
002355 : Empty Balances
002355 : Not enough data to calculate ...
089140 : Calculate Turnove

067990 : Calculate Turnover ... OK
002150 : Calculate Turnover ... OK
006620 : Calculate Turnover ... OK
100130 : Calculate Turnover ... OK
005160 : Calculate Turnover ... OK
075970 : Calculate Turnover ... OK
001230 : Calculate Turnover ... OK
086450 : Calculate Turnover ... OK
023450 : Calculate Turnover ... OK
004140 : Calculate Turnover ... OK
099410 : Calculate Turnover ... OK
007590 : Calculate Turnover ... OK
005960 : Calculate Turnover ... OK
005965 : Empty Incomes
005965 : Empty Balances
005965 : Not enough data to calculate ...
026960 : Calculate Turnover ... OK
002210 : Calculate Turnover ... OK
102260 : Not enough data to calculate ...
033500 : Calculate Turnover ... OK
025950 : Calculate Turnover ... OK
000640 : Calculate Turnover ... OK
170900 : Calculate Turnover ... OK
088130 : Calculate Turnover ... OK
028100 : Calculate Turnover ... OK
282690 : Calculate Turnover ... OK
041930 : Calculate Turnover ... OK
001520 : Calculate Turnover ... OK
001527 : Empty Incomes
001527

072870 : Calculate Turnover ... OK
215200 : Calculate Turnover ... OK
133750 : Calculate Turnover ... OK
235980 : Calculate Turnover ... OK
200580 : Empty Incomes
200580 : Empty Balances
200580 : Not enough data to calculate ...
041920 : Calculate Turnover ... OK
233250 : Empty Incomes
233250 : Empty Balances
233250 : Not enough data to calculate ...
014100 : Calculate Turnover ... OK
236340 : Empty Incomes
236340 : Empty Balances
236340 : Not enough data to calculate ...
054180 : Calculate Turnover ... OK
086900 : Calculate Turnover ... OK
078160 : Calculate Turnover ... OK
065650 : Calculate Turnover ... OK
138040 : Calculate Turnover ... OK
008560 : Calculate Turnover ... OK
000060 : Calculate Turnover ... OK
021880 : Empty Incomes
021880 : Empty Balances
021880 : Not enough data to calculate ...
140410 : Not enough data to calculate ...
241770 : Calculate Turnover ... OK
090370 : Calculate Turnover ... OK
059210 : Calculate Turnover ... OK
058110 : Calculate Turnover ... OK
096640 

...

204840 : Calculate Turnover ... OK
013870 : Calculate Turnover ... OK
071320 : Calculate Turnover ... OK
135160 : Empty Incomes
135160 : Empty Balances
135160 : Not enough data to calculate ...
311320 : Calculate Turnover ... OK
010580 : Calculate Turnover ... OK
035000 : Calculate Turnover ... OK
219750 : Calculate Turnover ... OK
088790 : Calculate Turnover ... OK
018120 : Calculate Turnover ... OK
109820 : Calculate Turnover ... OK
086060 : Calculate Turnover ... OK
036890 : Calculate Turnover ... OK
363250 : Calculate Turnover ... OK
003780 : Calculate Turnover ... OK
007370 : Calculate Turnover ... OK
010640 : Calculate Turnover ... OK
100250 : Calculate Turnover ... OK
051630 : Calculate Turnover ... OK
272450 : Calculate Turnover ... OK
011000 : Calculate Turnover ... OK
250030 : Calculate Turnover ... OK
002780 : Calculate Turnover ... OK
002787 : Empty Incomes
002787 : Empty Balances
002787 : Not enough data to calculate ...
002785 : Empty Incomes
002785 : Empty Balances
00278

023160 : Calculate Turnover ... OK
003240 : Calculate Turnover ... OK
011280 : Calculate Turnover ... OK
053620 : Calculate Turnover ... OK
004100 : Calculate Turnover ... OK
004105 : Empty Incomes
004105 : Empty Balances
004105 : Not enough data to calculate ...
116100 : Empty Incomes
116100 : Empty Balances
116100 : Not enough data to calculate ...
009410 : Calculate Turnover ... OK
009415 : Empty Incomes
009415 : Empty Balances
009415 : Not enough data to calculate ...
044490 : Calculate Turnover ... OK
124560 : Calculate Turnover ... OK
001420 : Calculate Turnover ... OK
007980 : Calculate Turnover ... OK
191420 : Calculate Turnover ... OK
073640 : Calculate Turnover ... OK
182690 : Empty Incomes
182690 : Empty Balances
182690 : Not enough data to calculate ...
066700 : Calculate Turnover ... OK
095610 : Calculate Turnover ... OK
131970 : Calculate Turnover ... OK
055490 : Calculate Turnover ... OK
308700 : Empty Incomes
308700 : Empty Balances
308700 : Not enough data to calculate

221840 : Calculate Turnover ... OK
106190 : Calculate Turnover ... OK
019490 : Calculate Turnover ... OK
000080 : Calculate Turnover ... OK
000087 : Empty Incomes
000087 : Empty Balances
000087 : Not enough data to calculate ...
000140 : Calculate Turnover ... OK
000145 : Empty Incomes
000145 : Empty Balances
000145 : Not enough data to calculate ...
373200 : Calculate Turnover ... OK
066130 : Calculate Turnover ... OK
152550 : Empty Incomes
152550 : Empty Balances
152550 : Not enough data to calculate ...
004590 : Calculate Turnover ... OK
036460 : Calculate Turnover ... OK
039340 : Calculate Turnover ... OK
005430 : Calculate Turnover ... OK
071050 : Calculate Turnover ... OK
071055 : Empty Incomes
071055 : Empty Balances
071055 : Not enough data to calculate ...
034950 : Calculate Turnover ... OK
010040 : Calculate Turnover ... OK
025540 : Calculate Turnover ... OK
222980 : Calculate Turnover ... OK
331660 : Empty Incomes
331660 : Empty Balances
331660 : Not enough data to calculate

060560 : Calculate Turnover ... OK
064240 : Calculate Turnover ... OK
039610 : Calculate Turnover ... OK
002460 : Calculate Turnover ... OK
378850 : Calculate Turnover ... OK
241590 : Not enough data to calculate ...
006060 : Calculate Turnover ... OK
013520 : Calculate Turnover ... OK
010690 : Calculate Turnover ... OK
126640 : Calculate Turnover ... OK
133820 : Calculate Turnover ... OK
061250 : Calculate Turnover ... OK
010660 : Calculate Turnover ... OK
000850 : Calculate Turnover ... OK
016580 : Calculate Turnover ... OK
032560 : Calculate Turnover ... OK
004800 : Calculate Turnover ... OK
094280 : Calculate Turnover ... OK
097870 : Not enough data to calculate ...
298040 : Calculate Turnover ... OK
298050 : Calculate Turnover ... OK
298020 : Calculate Turnover ... OK
298000 : Calculate Turnover ... OK
093370 : Calculate Turnover ... OK
050090 : Calculate Turnover ... OK
081660 : Calculate Turnover ... OK
290270 : Calculate Turnover ... OK
005870 : Calculate Turnover ... OK
353190

,code,name,exchange,period,mktcap,ASS turnover,REC turnover,INV turnover
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,0.198893,3.003746,0.000000
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,0.186153,2.158318,1.951752
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,0.031175,1.319794,2.448134
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,0.287868,2.076579,1.490167
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,0.148889,2.261761,0.879840
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,0.000000,0.000000,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,0.081981,3.490803,2.358013
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,0.314829,0.687308,NaN


In [132]:
# Turnover 종목 선정
df_factor = to_list.copy()
factor_list = ['ASS turnover'] # 총자산 회전률

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 총자산회전률, 시가총액 상위 20%, 모든 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 총자산회전률, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,ASS turnover,score
1,047050,포스코인터내셔널,0.800875,100.000000
2,282330,BGF리테일,0.668493,83.470313
3,001120,LX인터내셔널,0.649872,81.145345
4,005610,SPC삼립,0.622233,77.694243
5,298020,효성티앤씨,0.537908,67.165116
6,011070,LG이노텍,0.514506,64.242974
7,097520,엠씨넥스,0.491214,61.334730
8,049770,동원F&B,0.477799,59.659637
9,003070,코오롱글로벌,0.471305,58.848734
10,086280,현대글로비스,0.469469,58.619548


In [133]:
# Turnover 종목 선정
df_factor = to_list.copy()
factor_list = ['REC turnover'] # 매출채권 회전률

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 매출채권 회전률, 시가총액 상위 20%, 모든 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 매출채권 회전률, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,REC turnover,score
1,003690,코리안리,1.614743e+06,100.000000
2,000060,메리츠화재,6.602758e+04,4.089046
3,001450,현대해상,1.950733e+04,1.208077
4,005830,DB손해보험,1.029182e+04,0.637366
5,000810,삼성화재,2.293216e+03,0.142017
6,082640,동양생명,2.043524e+02,0.012655
7,032830,삼성생명,1.838877e+02,0.011388
8,023590,다우기술,1.558393e+02,0.009651
9,383800,LX홀딩스,1.542008e+02,0.009550
10,088350,한화생명,5.115037e+01,0.003168


In [134]:
# Turnover 종목 선정
df_factor = to_list.copy()
factor_list = ['INV turnover'] # 재고자산 회전률

result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 재고자산 회전률, 시가총액 상위 20%, 모든 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 재고자산 회전률, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,INV turnover,score
1,023590,다우기술,30659.756250,100.000000
2,088350,한화생명,2252.750238,7.347580
3,005830,DB손해보험,276.409822,0.901540
4,017390,서울가스,222.782121,0.726627
5,307950,현대오토에버,124.445747,0.405893
6,016710,대성홀딩스,112.883663,0.368182
7,000120,CJ대한통운,98.280595,0.320552
8,018260,삼성에스디에스,88.035079,0.287136
9,034120,SBS,85.203550,0.277900
10,035250,강원랜드,64.406229,0.210068


#### 이익률 지표

In [195]:
# GrossMargin 구하기
gm_list = b.getGrossMargin()
gm_list

Empty DB Incomes
060310 : Not enough data to calculate ...
095570 : Calculate Gross Margin ... OK
006840 : Calculate Gross Margin ... OK
054620 : Calculate Gross Margin ... OK
265520 : Calculate Gross Margin ... OK
211270 : Calculate Gross Margin ... OK
027410 : Calculate Gross Margin ... OK
282330 : Calculate Gross Margin ... OK
032790 : Calculate Gross Margin ... OK
138930 : Calculate Gross Margin ... OK
001460 : Calculate Gross Margin ... OK
Empty DB Incomes
001465 : Not enough data to calculate ...
013720 : Calculate Gross Margin ... OK
001040 : Calculate Gross Margin ... OK
079160 : Calculate Gross Margin ... OK
035760 : Calculate Gross Margin ... OK
311690 : Calculate Gross Margin ... OK
Empty DB Incomes
00104K : Not enough data to calculate ...
000120 : Calculate Gross Margin ... OK
011150 : Calculate Gross Margin ... OK
Empty DB Incomes
011155 : Not enough data to calculate ...
Empty DB Incomes
001045 : Not enough data to calculate ...
097950 : Calculate Gross Margin ... OK
Emp

099220 : Calculate Gross Margin ... OK
036540 : Calculate Gross Margin ... OK
255220 : Calculate Gross Margin ... OK
040610 : Calculate Gross Margin ... OK
049470 : Calculate Gross Margin ... OK
184230 : Calculate Gross Margin ... OK
005090 : Calculate Gross Margin ... OK
016250 : Calculate Gross Margin ... OK
001380 : Calculate Gross Margin ... OK
004060 : Calculate Gross Margin ... OK
001770 : Calculate Gross Margin ... OK
002360 : Calculate Gross Margin ... OK
009160 : Calculate Gross Margin ... OK
123700 : Not enough data to calculate ...
025530 : Calculate Gross Margin ... OK
034730 : Calculate Gross Margin ... OK
337450 : Not enough data to calculate ...
340350 : Not enough data to calculate ...
011790 : Calculate Gross Margin ... OK
018670 : Calculate Gross Margin ... OK
001740 : Calculate Gross Margin ... OK
Empty DB Incomes
001745 : Not enough data to calculate ...
006120 : Calculate Gross Margin ... OK
Empty DB Incomes
006125 : Not enough data to calculate ...
210980 : Calcul

095660 : Calculate Gross Margin ... OK
042420 : Calculate Gross Margin ... OK
950220 : Calculate Gross Margin ... OK
311390 : Calculate Gross Margin ... OK
085910 : Calculate Gross Margin ... OK
092730 : Calculate Gross Margin ... OK
290660 : Calculate Gross Margin ... OK
306620 : Not enough data to calculate ...
153460 : Calculate Gross Margin ... OK
007390 : Calculate Gross Margin ... OK
Empty DB Incomes
086220 : Not enough data to calculate ...
033640 : Calculate Gross Margin ... OK
330860 : Calculate Gross Margin ... OK
005720 : Calculate Gross Margin ... OK
Empty DB Incomes
005725 : Not enough data to calculate ...
002350 : Calculate Gross Margin ... OK
Empty DB Incomes
002355 : Not enough data to calculate ...
089140 : Calculate Gross Margin ... OK
065170 : Calculate Gross Margin ... OK
003580 : Calculate Gross Margin ... OK
137940 : Not enough data to calculate ...
348210 : Calculate Gross Margin ... OK
Empty DB Incomes
041140 : Not enough data to calculate ...
217270 : Calculat

026960 : Calculate Gross Margin ... OK
002210 : Calculate Gross Margin ... OK
102260 : Not enough data to calculate ...
033500 : Calculate Gross Margin ... OK
025950 : Calculate Gross Margin ... OK
000640 : Calculate Gross Margin ... OK
170900 : Calculate Gross Margin ... OK
088130 : Calculate Gross Margin ... OK
028100 : Calculate Gross Margin ... OK
282690 : Calculate Gross Margin ... OK
041930 : Calculate Gross Margin ... OK
001520 : Calculate Gross Margin ... OK
Empty DB Incomes
001527 : Not enough data to calculate ...
Empty DB Incomes
001529 : Not enough data to calculate ...
084670 : Calculate Gross Margin ... OK
082640 : Calculate Gross Margin ... OK
060380 : Calculate Gross Margin ... OK
Empty DB Incomes
001525 : Not enough data to calculate ...
079960 : Calculate Gross Margin ... OK
008970 : Calculate Gross Margin ... OK
228340 : Calculate Gross Margin ... OK
092780 : Not enough data to calculate ...
088910 : Calculate Gross Margin ... OK
094170 : Calculate Gross Margin ... O

140410 : Not enough data to calculate ...
241770 : Calculate Gross Margin ... OK
090370 : Calculate Gross Margin ... OK
059210 : Calculate Gross Margin ... OK
058110 : Calculate Gross Margin ... OK
096640 : Calculate Gross Margin ... OK
017180 : Calculate Gross Margin ... OK
257370 : Calculate Gross Margin ... OK
009900 : Calculate Gross Margin ... OK
Empty DB Incomes
267060 : Not enough data to calculate ...
012690 : Calculate Gross Margin ... OK
005360 : Calculate Gross Margin ... OK
080420 : Calculate Gross Margin ... OK
080160 : Calculate Gross Margin ... OK
204210 : Calculate Gross Margin ... OK
100030 : Calculate Gross Margin ... OK
087260 : Calculate Gross Margin ... OK
101330 : Calculate Gross Margin ... OK
012860 : Calculate Gross Margin ... OK
348030 : Calculate Gross Margin ... OK
250060 : Not enough data to calculate ...
288980 : Calculate Gross Margin ... OK
Empty DB Incomes
033200 : Not enough data to calculate ...
009680 : Calculate Gross Margin ... OK
118990 : Calculate

...

311320 : Calculate Gross Margin ... OK
010580 : Calculate Gross Margin ... OK
035000 : Calculate Gross Margin ... OK
219750 : Calculate Gross Margin ... OK
088790 : Calculate Gross Margin ... OK
018120 : Calculate Gross Margin ... OK
109820 : Calculate Gross Margin ... OK
086060 : Calculate Gross Margin ... OK
036890 : Calculate Gross Margin ... OK
363250 : Calculate Gross Margin ... OK
003780 : Calculate Gross Margin ... OK
007370 : Calculate Gross Margin ... OK
010640 : Calculate Gross Margin ... OK
100250 : Calculate Gross Margin ... OK
051630 : Calculate Gross Margin ... OK
272450 : Calculate Gross Margin ... OK
011000 : Calculate Gross Margin ... OK
250030 : Calculate Gross Margin ... OK
002780 : Calculate Gross Margin ... OK
Empty DB Incomes
002787 : Not enough data to calculate ...
Empty DB Incomes
002785 : Not enough data to calculate ...
Empty DB Incomes
233990 : Not enough data to calculate ...
085660 : Calculate Gross Margin ... OK
261780 : Calculate Gross Margin ... OK
0093

Empty DB Incomes
116100 : Not enough data to calculate ...
009410 : Calculate Gross Margin ... OK
Empty DB Incomes
009415 : Not enough data to calculate ...
044490 : Calculate Gross Margin ... OK
124560 : Calculate Gross Margin ... OK
001420 : Calculate Gross Margin ... OK
007980 : Calculate Gross Margin ... OK
191420 : Calculate Gross Margin ... OK
073640 : Calculate Gross Margin ... OK
Empty DB Incomes
182690 : Not enough data to calculate ...
066700 : Calculate Gross Margin ... OK
095610 : Calculate Gross Margin ... OK
131970 : Calculate Gross Margin ... OK
055490 : Calculate Gross Margin ... OK
Empty DB Incomes
308700 : Not enough data to calculate ...
089030 : Calculate Gross Margin ... OK
Empty DB Incomes
258050 : Not enough data to calculate ...
054450 : Calculate Gross Margin ... OK
091440 : Calculate Gross Margin ... OK
078000 : Calculate Gross Margin ... OK
200230 : Calculate Gross Margin ... OK
214420 : Calculate Gross Margin ... OK
Empty DB Incomes
393210 : Not enough data 

373200 : Calculate Gross Margin ... OK
066130 : Calculate Gross Margin ... OK
Empty DB Incomes
152550 : Not enough data to calculate ...
004590 : Calculate Gross Margin ... OK
036460 : Calculate Gross Margin ... OK
039340 : Calculate Gross Margin ... OK
005430 : Calculate Gross Margin ... OK
071050 : Calculate Gross Margin ... OK
Empty DB Incomes
071055 : Not enough data to calculate ...
034950 : Calculate Gross Margin ... OK
010040 : Calculate Gross Margin ... OK
025540 : Calculate Gross Margin ... OK
222980 : Calculate Gross Margin ... OK
Empty DB Incomes
331660 : Not enough data to calculate ...
318000 : Calculate Gross Margin ... OK
256840 : Calculate Gross Margin ... OK
004090 : Calculate Gross Margin ... OK
025550 : Calculate Gross Margin ... OK
002200 : Calculate Gross Margin ... OK
002960 : Calculate Gross Margin ... OK
017890 : Calculate Gross Margin ... OK
000240 : Calculate Gross Margin ... OK
080720 : Calculate Gross Margin ... OK
123890 : Calculate Gross Margin ... OK
0157

010690 : Calculate Gross Margin ... OK
126640 : Calculate Gross Margin ... OK
133820 : Calculate Gross Margin ... OK
061250 : Calculate Gross Margin ... OK
010660 : Calculate Gross Margin ... OK
000850 : Calculate Gross Margin ... OK
016580 : Calculate Gross Margin ... OK
032560 : Calculate Gross Margin ... OK
004800 : Calculate Gross Margin ... OK
094280 : Calculate Gross Margin ... OK
097870 : Not enough data to calculate ...
298040 : Calculate Gross Margin ... OK
298050 : Calculate Gross Margin ... OK
298020 : Calculate Gross Margin ... OK
298000 : Calculate Gross Margin ... OK
093370 : Calculate Gross Margin ... OK
050090 : Calculate Gross Margin ... OK
081660 : Calculate Gross Margin ... OK
290270 : Calculate Gross Margin ... OK
005870 : Calculate Gross Margin ... OK
353190 : Not enough data to calculate ...
090710 : Calculate Gross Margin ... OK
205470 : Calculate Gross Margin ... OK
115160 : Calculate Gross Margin ... OK
028080 : Calculate Gross Margin ... OK
032860 : Calculate 

,code,name,exchange,period,mktcap,GrossMargin
0,095570,AJ네트웍스,KOSPI,2022/03,2.898300e+11,100.000000
1,006840,AK홀딩스,KOSPI,2022/03,2.907840e+11,15.372372
2,054620,APS홀딩스,KOSDAQ,2022/03,2.488095e+11,49.319104
3,265520,AP시스템,KOSDAQ,2022/03,3.629337e+11,23.556013
4,211270,AP위성,KOSDAQ,2022/03,2.307593e+11,NaN
...,...,...,...,...,...,...
2177,189980,흥국에프엔비,KOSDAQ,2022/03,1.557913e+11,34.403979
2178,000540,흥국화재,KOSPI,2022/03,2.601827e+11,NaN
2179,003280,흥아해운,KOSPI,2022/03,6.996365e+11,8.957735
2180,037440,희림,KOSDAQ,2022/03,1.322635e+11,11.409652


In [199]:
# GorssMargin 종목 선정
df_factor = gm_list.copy()
factor_list = ['GrossMargin']
result = stock_select(df_factor, 0.2, 'ALL', 30, factor_list) # 이익률지표, 시가총액 상위 20%, 모든 종목 중 30개 선택
result = stock_select(df_factor, 0.2, 'KOSPI', 30, factor_list) # 이익률지표, 시가총액 상위 20%, KOSPI 종목 중 30개 선택
result

,code,name,GrossMargin,score
1,097230,HJ중공업,0.406900,100.000000
2,082640,동양생명,1.257922,99.145501
3,032830,삼성생명,1.534628,98.867665
4,085620,미래에셋생명,2.049218,98.350973
5,047050,포스코인터내셔널,3.410383,96.984247
6,003470,유안타증권,3.869809,96.522944
7,138930,BNK금융지주,4.103920,96.287877
8,138040,메리츠금융지주,4.386256,96.004387
9,000670,영풍,4.501193,95.888981
10,088350,한화생명,4.938273,95.450115
